In [1]:
# gpu check
! nvidia-smi

Fri Dec  8 14:38:31 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:3A:00.0 Off |                    0 |
| N/A   31C    P0    42W / 300W |      9MiB / 32768MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-SXM2...  On   | 00000000:3B:00.0 Off |                    0 |
| N/A   

In [2]:
# check number of cores
import multiprocessing

cores = multiprocessing.cpu_count() 
cores

80

In [3]:
! python --version

Python 3.9.7


## Environment Prep

In [4]:
cd /pfs/data5/home/hd/hd_hd/hd_nf283/MA_Thesis/

/pfs/data5/home/hd/hd_hd/hd_nf283/MA_Thesis


In [5]:
# from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization, Lambda
# # from tensorflow.keras.models import Model
# from tensorflow.keras import models
import pickle
import numpy as np
from tqdm import tqdm
tqdm.pandas()
# from sklearn.utils.class_weight import compute_class_weight
# from tensorflow.keras.optimizers import Adam
# from sklearn.metrics import roc_auc_score, precision_recall_curve, auc
# import tensorflow.keras.backend as K
# from tensorflow.keras.callbacks import Callback, EarlyStopping
import pandas as pd
import json
from torch.utils.data import Dataset
# from transformers import AutoTokenizer, pipeline, AutoModel
# import resources.smart_cond as sc
# from google.colab import files
import gc

## Train/Val

In [6]:
# path to glove data
data_path = 'Data/glove_kws_reserved.pkl'
pkl = pickle.load(open(data_path, 'rb'))
data = pkl[0]
oc = pkl[1]
train_ind = pkl[2]
valid_ind = pkl[3]
test_ind = pkl[4]

In [7]:
# embs = pkl[5]
del pkl
data

,ts_ind,hour,variable,value,TABLE,mean,std
0,10223,467.816667,Text,Admission Date : 2119 - 5 - 4 Discharge Date :...,noteevents,1.000000,1.000000
1,18407,28.016667,Text,Admission Date : 2112 - 12 - 8 Discharge Date ...,noteevents,1.000000,1.000000
2,40300,155.166667,Text,Admission Date : 2194 - 7 - 18 Discharge Date ...,noteevents,1.000000,1.000000
3,23747,52.383333,Text,Admission Date : 2194 - 1 - 7 Discharge Date :...,noteevents,1.000000,1.000000
4,2357,73.133333,Text,Admission Date : 2186 - 6 - 7 Discharge Date :...,noteevents,1.000000,1.000000
...,...,...,...,...,...,...,...
82886223,57281,20.400000,MBP,0.195381,chart,78.552377,17.645628
82886224,57281,20.400000,O2 Saturation,-0.678068,chart,96.820961,4.160290
82886225,57281,20.400000,RR,0.179866,chart,26.278501,15.130729
82886226,57281,20.400000,SBP,-0.404061,chart,120.239648,25.341836


In [8]:
# Filter labeled data in first 24h.
data = data.loc[data.ts_ind.isin(np.concatenate((train_ind, valid_ind, test_ind), axis=-1))]
data = data.loc[(data.hour>=0)&(data.hour<=24)]
oc = oc.loc[oc.ts_ind.isin(np.concatenate((train_ind, valid_ind, test_ind), axis=-1))]
# Get y and N.
y = np.array(oc.sort_values(by='ts_ind')['in_hospital_sepsis']).astype('float32')
N = data.ts_ind.max() + 1
# Get static data with mean fill and missingness indicator.
static_varis = ['Age', 'Gender']
ii = data.variable.isin(static_varis)
static_data = data.loc[ii]
data = data.loc[~ii]
def inv_list(l, start=0):
    d = {}
    for i in range(len(l)):
        d[l[i]] = i+start
    return d
static_var_to_ind = inv_list(static_varis)
D = len(static_varis)
demo = np.zeros((N, D))
for row in tqdm(static_data.itertuples()):
    demo[row.ts_ind, static_var_to_ind[row.variable]] = row.value
# Normalize static data.
means = demo.mean(axis=0, keepdims=True)
stds = demo.std(axis=0, keepdims=True)
stds = (stds==0)*1 + (stds!=0)*stds
demo = (demo-means)/stds
# Trim to max len.
data = data.sample(frac=1)
data = data.groupby('ts_ind').head(880)
# Get N, V, var_to_ind.
N = data.ts_ind.max() + 1
varis = sorted(list(set(data.variable)))
V = len(varis)
def inv_list(l, start=0):
    d = {}
    for i in range(len(l)):
        d[l[i]] = i+start
    return d
var_to_ind = inv_list(varis, start=1)
data['vind'] = data.variable.map(var_to_ind)
data = data[['ts_ind', 'vind', 'hour', 'value']].sort_values(by=['ts_ind', 'vind', 'hour'])
# Add obs index.
data = data.sort_values(by=['ts_ind']).reset_index(drop=True)
data = data.reset_index().rename(columns={'index':'obs_ind'})
data = data.merge(data.groupby('ts_ind').agg({'obs_ind':'min'}).reset_index().rename(columns={ \
                                                            'obs_ind':'first_obs_ind'}), on='ts_ind')
data['obs_ind'] = data['obs_ind'] - data['first_obs_ind']
# Find max_len.
max_len = data.obs_ind.max()+1
print ('max_len', max_len)
# Generate times_ip and values_ip matrices.
# times_inp = np.zeros((N, max_len), dtype='float32')
# values_inp = np.zeros((N, max_len), dtype='float32')
# varis_inp = np.zeros((N, max_len), dtype='int32')

114564it [00:00, 793061.96it/s]


max_len 880


In [9]:
texts_inp = np.empty([N, max_len], dtype='object')
for row in tqdm(data.itertuples()):
    ts_ind = row.ts_ind
    l = row.obs_ind
    # times_inp[ts_ind, l] = row.hour
    # if isinstance(row.value, str):
    if row.vind == 124:
        # values_inp[ts_ind, l] = 1.0
        # print(row.value)
        texts_inp[ts_ind, l] = row.value
        # print(row.value)
    else:{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {
    "id": "ECKAJiLXiTlN"
   },
   "source": [
    "## Hardware check"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 1,
   "metadata": {
    "id": "e7odgm5VsOsb"
   },
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "Wed Dec 13 12:34:42 2023       \n",
      "+-----------------------------------------------------------------------------+\n",
      "| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |\n",
      "|-------------------------------+----------------------+----------------------+\n",
      "| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |\n",
      "| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |\n",
      "|                               |                      |               MIG M. |\n",
      "|===============================+======================+======================|\n",
      "|   0  Tesla V100-SXM2...  On   | 00000000:3B:00.0 Off |                    0 |\n",
      "| N/A   39C    P0    65W / 300W |      9MiB / 32768MiB |      0%      Default |\n",
      "|                               |                      |                  N/A |\n",
      "+-------------------------------+----------------------+----------------------+\n",
      "|   1  Tesla V100-SXM2...  On   | 00000000:B2:00.0 Off |                    0 |\n",
      "| N/A   35C    P0    39W / 300W |      9MiB / 32768MiB |      0%      Default |\n",
      "|                               |                      |                  N/A |\n",
      "+-------------------------------+----------------------+----------------------+\n",
      "                                                                               \n",
      "+-----------------------------------------------------------------------------+\n",
      "| Processes:                                                                  |\n",
      "|  GPU   GI   CI        PID   Type   Process name                  GPU Memory |\n",
      "|        ID   ID                                                   Usage      |\n",
      "|=============================================================================|\n",
      "|    0   N/A  N/A      2031      G   /usr/libexec/Xorg                   8MiB |\n",
      "|    1   N/A  N/A      2031      G   /usr/libexec/Xorg                   8MiB |\n",
      "+-----------------------------------------------------------------------------+\n"
     ]
    }
   ],
   "source": [
    "# gpu check\n",
    "!nvidia-smi"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 2,
   "metadata": {
    "id": "oEEUeIW3sOsc"
   },
   "outputs": [
    {
     "data": {
      "text/plain": [
       "80"
      ]
     },
     "execution_count": 2,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "# check number of cores\n",
    "import multiprocessing\n",
    "\n",
    "cores = multiprocessing.cpu_count() \n",
    "cores"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {
    "id": "vY9I5eXWeOom"
   },
   "source": [
    "## Environment Prep"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 3,
   "metadata": {
    "tags": []
   },
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "/pfs/data5/home/hd/hd_hd/hd_nf283/MA_Thesis\n"
     ]
    }
   ],
   "source": [
    "cd /pfs/data5/home/hd/hd_hd/hd_nf283/MA_Thesis/"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 4,
   "metadata": {
    "tags": []
   },
   "outputs": [
    {
     "name": "stderr",
     "output_type": "stream",
     "text": [
      "2023-12-13 12:34:46.149794: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.\n",
      "2023-12-13 12:34:46.851064: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.\n",
      "To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.\n",
      "2023-12-13 12:35:17.279566: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT\n"
     ]
    }
   ],
   "source": [
    "from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization, Lambda, GlobalAveragePooling1D\n",
    "# from tensorflow.keras.models import Model\n",
    "from tensorflow.keras import models\n",
    "import pickle\n",
    "import numpy as np\n",
    "from tqdm import tqdm\n",
    "tqdm.pandas()\n",
    "from sklearn.utils.class_weight import compute_class_weight\n",
    "from tensorflow.keras.optimizers import Adam\n",
    "from sklearn.metrics import roc_auc_score, precision_recall_curve, auc\n",
    "import tensorflow.keras.backend as K\n",
    "from tensorflow.keras.callbacks import Callback, EarlyStopping\n",
    "import pandas as pd\n",
    "import json\n",
    "from torch.utils.data import Dataset\n",
    "from transformers import AutoTokenizer, pipeline, AutoModel\n",
    "import resources.smart_cond as sc\n",
    "# from google.colab import files\n",
    "from keras.utils import pad_sequences\n",
    "from keras.preprocessing.text import Tokenizer"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 5,
   "metadata": {
    "tags": []
   },
   "outputs": [
    {
     "name": "stderr",
     "output_type": "stream",
     "text": [
      "36551it [00:00, 75135.93it/s]\n"
     ]
    }
   ],
   "source": [
    "train_path = 'Exp_GloVe/texts_ip/classification_train.txt'\n",
    "train_text = []\n",
    "\n",
    "with open(train_path) as file:\n",
    "    for line in tqdm(file):\n",
    "        train_text.append(line.rstrip())\n",
    "\n",
    "# train_text = np.array(train_text)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 6,
   "metadata": {
    "tags": []
   },
   "outputs": [
    {
     "name": "stderr",
     "output_type": "stream",
     "text": [
      "9262it [00:00, 47483.29it/s]\n"
     ]
    }
   ],
   "source": [
    "valid_path = 'Exp_GloVe/texts_ip/classification_valid.txt'\n",
    "valid_text = []\n",
    "\n",
    "with open(valid_path) as file:\n",
    "    for line in tqdm(file):\n",
    "        valid_text.append(line.rstrip())\n",
    "\n",
    "# train_text = np.array(train_text)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 7,
   "metadata": {
    "tags": []
   },
   "outputs": [
    {
     "name": "stderr",
     "output_type": "stream",
     "text": [
      "11469it [00:00, 51768.44it/s]\n"
     ]
    }
   ],
   "source": [
    "test_path = 'Exp_GloVe/texts_ip/classification_test.txt'\n",
    "test_text = []\n",
    "\n",
    "with open(test_path) as file:\n",
    "    for line in tqdm(file):\n",
    "        test_text.append(line.rstrip())\n",
    "\n",
    "# train_text = np.array(train_text)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 8,
   "metadata": {
    "tags": []
   },
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "vocabulary size: 297321\n"
     ]
    }
   ],
   "source": [
    "# prepare tokenizer\n",
    "t = Tokenizer()\n",
    "train_val_ip = train_text+valid_text\n",
    "t.fit_on_texts(train_val_ip)\n",
    "# vocab_size = len(t.word_index) + 1\n",
    "vocab_size = 297321\n",
    "print(f'vocabulary size: {vocab_size}')\n",
    "del train_val_ip\n",
    "\n",
    "# encode\n",
    "encoded_train = t.texts_to_sequences(train_text)\n",
    "encoded_valid = t.texts_to_sequences(valid_text)\n",
    "encoded_test = t.texts_to_sequences(test_text)\n",
    "\n",
    "\n",
    "# pad to max_len\n",
    "max_length = 24135\n",
    "padded_train = pad_sequences(encoded_train, maxlen=max_length, padding='post')\n",
    "padded_valid = pad_sequences(encoded_valid, maxlen=max_length, padding='post')\n",
    "padded_test = pad_sequences(encoded_test, maxlen=max_length, padding='post')"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 9,
   "metadata": {
    "tags": []
   },
   "outputs": [],
   "source": [
    "data_path = 'Data/sepsis_removed_0.pkl'\n",
    "pkl = pickle.load(open(data_path, 'rb'))\n",
    "data = pkl[0]\n",
    "oc = pkl[1]\n",
    "train_ind = pkl[2]\n",
    "valid_ind = pkl[3]\n",
    "test_ind = pkl[4]\n",
    "del pkl"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 10,
   "metadata": {
    "tags": []
   },
   "outputs": [
    {
     "data": {
      "text/html": [
       "<div>\n",
       "<style scoped>\n",
       "    .dataframe tbody tr th:only-of-type {\n",
       "        vertical-align: middle;\n",
       "    }\n",
       "\n",
       "    .dataframe tbody tr th {\n",
       "        vertical-align: top;\n",
       "    }\n",
       "\n",
       "    .dataframe thead th {\n",
       "        text-align: right;\n",
       "    }\n",
       "</style>\n",
       "<table border=\"1\" class=\"dataframe\">\n",
       "  <thead>\n",
       "    <tr style=\"text-align: right;\">\n",
       "      <th></th>\n",
       "      <th>ts_ind</th>\n",
       "      <th>hour</th>\n",
       "      <th>variable</th>\n",
       "      <th>value</th>\n",
       "      <th>TABLE</th>\n",
       "      <th>mean</th>\n",
       "      <th>std</th>\n",
       "    </tr>\n",
       "  </thead>\n",
       "  <tbody>\n",
       "    <tr>\n",
       "      <th>0</th>\n",
       "      <td>10223</td>\n",
       "      <td>467.816667</td>\n",
       "      <td>Text</td>\n",
       "      <td>1</td>\n",
       "      <td>noteevents</td>\n",
       "      <td>1.000000</td>\n",
       "      <td>1.000000</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>1</th>\n",
       "      <td>18407</td>\n",
       "      <td>28.016667</td>\n",
       "      <td>Text</td>\n",
       "      <td>1</td>\n",
       "      <td>noteevents</td>\n",
       "      <td>1.000000</td>\n",
       "      <td>1.000000</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>2</th>\n",
       "      <td>40300</td>\n",
       "      <td>155.166667</td>\n",
       "      <td>Text</td>\n",
       "      <td>1</td>\n",
       "      <td>noteevents</td>\n",
       "      <td>1.000000</td>\n",
       "      <td>1.000000</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>3</th>\n",
       "      <td>23747</td>\n",
       "      <td>52.383333</td>\n",
       "      <td>Text</td>\n",
       "      <td>1</td>\n",
       "      <td>noteevents</td>\n",
       "      <td>1.000000</td>\n",
       "      <td>1.000000</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>4</th>\n",
       "      <td>2357</td>\n",
       "      <td>73.133333</td>\n",
       "      <td>Text</td>\n",
       "      <td>1</td>\n",
       "      <td>noteevents</td>\n",
       "      <td>1.000000</td>\n",
       "      <td>1.000000</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>...</th>\n",
       "      <td>...</td>\n",
       "      <td>...</td>\n",
       "      <td>...</td>\n",
       "      <td>...</td>\n",
       "      <td>...</td>\n",
       "      <td>...</td>\n",
       "      <td>...</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>82886223</th>\n",
       "      <td>57281</td>\n",
       "      <td>20.400000</td>\n",
       "      <td>MBP</td>\n",
       "      <td>0.195381</td>\n",
       "      <td>chart</td>\n",
       "      <td>78.552377</td>\n",
       "      <td>17.645628</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>82886224</th>\n",
       "      <td>57281</td>\n",
       "      <td>20.400000</td>\n",
       "      <td>O2 Saturation</td>\n",
       "      <td>-0.678068</td>\n",
       "      <td>chart</td>\n",
       "      <td>96.820961</td>\n",
       "      <td>4.160290</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>82886225</th>\n",
       "      <td>57281</td>\n",
       "      <td>20.400000</td>\n",
       "      <td>RR</td>\n",
       "      <td>0.179866</td>\n",
       "      <td>chart</td>\n",
       "      <td>26.278501</td>\n",
       "      <td>15.130729</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>82886226</th>\n",
       "      <td>57281</td>\n",
       "      <td>20.400000</td>\n",
       "      <td>SBP</td>\n",
       "      <td>-0.404061</td>\n",
       "      <td>chart</td>\n",
       "      <td>120.239648</td>\n",
       "      <td>25.341836</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>82886227</th>\n",
       "      <td>57281</td>\n",
       "      <td>20.400000</td>\n",
       "      <td>Urine</td>\n",
       "      <td>-0.24296</td>\n",
       "      <td>output</td>\n",
       "      <td>123.393012</td>\n",
       "      <td>137.442433</td>\n",
       "    </tr>\n",
       "  </tbody>\n",
       "</table>\n",
       "<p>82447326 rows × 7 columns</p>\n",
       "</div>"
      ],
      "text/plain": [
       "          ts_ind        hour       variable     value       TABLE        mean  \\\n",
       "0          10223  467.816667           Text         1  noteevents    1.000000   \n",
       "1          18407   28.016667           Text         1  noteevents    1.000000   \n",
       "2          40300  155.166667           Text         1  noteevents    1.000000   \n",
       "3          23747   52.383333           Text         1  noteevents    1.000000   \n",
       "4           2357   73.133333           Text         1  noteevents    1.000000   \n",
       "...          ...         ...            ...       ...         ...         ...   \n",
       "82886223   57281   20.400000            MBP  0.195381       chart   78.552377   \n",
       "82886224   57281   20.400000  O2 Saturation -0.678068       chart   96.820961   \n",
       "82886225   57281   20.400000             RR  0.179866       chart   26.278501   \n",
       "82886226   57281   20.400000            SBP -0.404061       chart  120.239648   \n",
       "82886227   57281   20.400000          Urine  -0.24296      output  123.393012   \n",
       "\n",
       "                 std  \n",
       "0           1.000000  \n",
       "1           1.000000  \n",
       "2           1.000000  \n",
       "3           1.000000  \n",
       "4           1.000000  \n",
       "...              ...  \n",
       "82886223   17.645628  \n",
       "82886224    4.160290  \n",
       "82886225   15.130729  \n",
       "82886226   25.341836  \n",
       "82886227  137.442433  \n",
       "\n",
       "[82447326 rows x 7 columns]"
      ]
     },
     "execution_count": 10,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "data.loc[data['variable'] == 'Text', 'value'] = 1\n",
    "data"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 11,
   "metadata": {
    "tags": []
   },
   "outputs": [
    {
     "name": "stderr",
     "output_type": "stream",
     "text": [
      "114564it [00:00, 799659.58it/s]\n"
     ]
    },
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "max_len 880\n"
     ]
    },
    {
     "name": "stderr",
     "output_type": "stream",
     "text": [
      "19267073it [00:27, 701695.13it/s]\n"
     ]
    }
   ],
   "source": [
    "# Filter labeled data in first 24h.\n",
    "data = data.loc[data.ts_ind.isin(np.concatenate((train_ind, valid_ind, test_ind), axis=-1))]\n",
    "data = data.loc[(data.hour>=0)&(data.hour<=24)]\n",
    "oc = oc.loc[oc.ts_ind.isin(np.concatenate((train_ind, valid_ind, test_ind), axis=-1))]\n",
    "# Fix age.\n",
    "data.loc[(data.variable=='Age')&(data.value>200), 'value'] = 91.4\n",
    "# Get y and N.\n",
    "y = np.array(oc.sort_values(by='ts_ind')['in_hospital_sepsis']).astype('float32')\n",
    "N = data.ts_ind.max() + 1\n",
    "# Get static data with mean fill and missingness indicator.\n",
    "static_varis = ['Age', 'Gender']\n",
    "ii = data.variable.isin(static_varis)\n",
    "static_data = data.loc[ii]\n",
    "data = data.loc[~ii]\n",
    "def inv_list(l, start=0):\n",
    "    d = {}\n",
    "    for i in range(len(l)):\n",
    "        d[l[i]] = i+start\n",
    "    return d\n",
    "static_var_to_ind = inv_list(static_varis)\n",
    "D = len(static_varis)\n",
    "demo = np.zeros((N, D))\n",
    "for row in tqdm(static_data.itertuples()):\n",
    "    demo[row.ts_ind, static_var_to_ind[row.variable]] = row.value\n",
    "# Normalize static data.\n",
    "means = demo.mean(axis=0, keepdims=True)\n",
    "stds = demo.std(axis=0, keepdims=True)\n",
    "stds = (stds==0)*1 + (stds!=0)*stds\n",
    "demo = (demo-means)/stds\n",
    "# Trim to max len.\n",
    "data = data.sample(frac=1)\n",
    "data = data.groupby('ts_ind').head(880)\n",
    "# Get N, V, var_to_ind.\n",
    "N = data.ts_ind.max() + 1\n",
    "varis = sorted(list(set(data.variable)))\n",
    "V = len(varis)\n",
    "def inv_list(l, start=0):\n",
    "    d = {}\n",
    "    for i in range(len(l)):\n",
    "        d[l[i]] = i+start\n",
    "    return d\n",
    "var_to_ind = inv_list(varis, start=1)\n",
    "data['vind'] = data.variable.map(var_to_ind)\n",
    "data = data[['ts_ind', 'vind', 'hour', 'value']].sort_values(by=['ts_ind', 'vind', 'hour'])\n",
    "# Add obs index.\n",
    "data = data.sort_values(by=['ts_ind']).reset_index(drop=True)\n",
    "data = data.reset_index().rename(columns={'index':'obs_ind'})\n",
    "data = data.merge(data.groupby('ts_ind').agg({'obs_ind':'min'}).reset_index().rename(columns={ \\\n",
    "                                                            'obs_ind':'first_obs_ind'}), on='ts_ind')\n",
    "data['obs_ind'] = data['obs_ind'] - data['first_obs_ind']\n",
    "# Find max_len.\n",
    "max_len = data.obs_ind.max()+1\n",
    "print ('max_len', max_len)\n",
    "# Generate times_ip and values_ip matrices.\n",
    "times_inp = np.zeros((N, max_len), dtype='float32')\n",
    "values_inp = np.zeros((N, max_len), dtype='float32')\n",
    "varis_inp = np.zeros((N, max_len), dtype='int32')\n",
    "for row in tqdm(data.itertuples()):\n",
    "    ts_ind = row.ts_ind\n",
    "    l = row.obs_ind\n",
    "    times_inp[ts_ind, l] = row.hour\n",
    "    values_inp[ts_ind, l] = row.value\n",
    "    varis_inp[ts_ind, l] = row.vind\n",
    "data.drop(columns=['obs_ind', 'first_obs_ind'], inplace=True)\n",
    "# Generate 3 sets of inputs and outputs.\n",
    "train_ip = [ip[train_ind] for ip in [demo, times_inp, values_inp, varis_inp]]\n",
    "valid_ip = [ip[valid_ind] for ip in [demo, times_inp, values_inp, varis_inp]]\n",
    "test_ip = [ip[test_ind] for ip in [demo, times_inp, values_inp, varis_inp]]\n",
    "del times_inp, values_inp, varis_inp\n",
    "# train_op = y[train_ind]\n",
    "# valid_op = y[valid_ind]\n",
    "# test_op = y[test_ind]\n",
    "# del y"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 12,
   "metadata": {
    "tags": []
   },
   "outputs": [],
   "source": [
    "train_ip.append(padded_train)\n",
    "valid_ip.append(padded_valid)\n",
    "test_ip.append(padded_test)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 13,
   "metadata": {
    "tags": []
   },
   "outputs": [],
   "source": [
    "# data_path = 'Data/sepsis_removed_0.pkl'\n",
    "pkl = pickle.load(open(data_path, 'rb'))\n",
    "data = pkl[0]\n",
    "oc = pkl[1]\n",
    "train_ind = pkl[2]\n",
    "valid_ind = pkl[3]\n",
    "test_ind = pkl[4]\n",
    "del pkl"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 14,
   "metadata": {
    "tags": []
   },
   "outputs": [
    {
     "data": {
      "text/html": [
       "<div>\n",
       "<style scoped>\n",
       "    .dataframe tbody tr th:only-of-type {\n",
       "        vertical-align: middle;\n",
       "    }\n",
       "\n",
       "    .dataframe tbody tr th {\n",
       "        vertical-align: top;\n",
       "    }\n",
       "\n",
       "    .dataframe thead th {\n",
       "        text-align: right;\n",
       "    }\n",
       "</style>\n",
       "<table border=\"1\" class=\"dataframe\">\n",
       "  <thead>\n",
       "    <tr style=\"text-align: right;\">\n",
       "      <th></th>\n",
       "      <th>ts_ind</th>\n",
       "      <th>HADM_ID</th>\n",
       "      <th>SUBJECT_ID</th>\n",
       "      <th>in_hospital_sepsis</th>\n",
       "    </tr>\n",
       "  </thead>\n",
       "  <tbody>\n",
       "    <tr>\n",
       "      <th>0</th>\n",
       "      <td>0</td>\n",
       "      <td>110404</td>\n",
       "      <td>268</td>\n",
       "      <td>1</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>1</th>\n",
       "      <td>1</td>\n",
       "      <td>188028</td>\n",
       "      <td>270</td>\n",
       "      <td>0</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>2</th>\n",
       "      <td>2</td>\n",
       "      <td>173727</td>\n",
       "      <td>271</td>\n",
       "      <td>0</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>3</th>\n",
       "      <td>3</td>\n",
       "      <td>164716</td>\n",
       "      <td>272</td>\n",
       "      <td>0</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>4</th>\n",
       "      <td>4</td>\n",
       "      <td>158689</td>\n",
       "      <td>273</td>\n",
       "      <td>0</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>...</th>\n",
       "      <td>...</td>\n",
       "      <td>...</td>\n",
       "      <td>...</td>\n",
       "      <td>...</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>54862</th>\n",
       "      <td>57277</td>\n",
       "      <td>182476</td>\n",
       "      <td>83967</td>\n",
       "      <td>0</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>29333</th>\n",
       "      <td>57278</td>\n",
       "      <td>118320</td>\n",
       "      <td>23200</td>\n",
       "      <td>0</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>54573</th>\n",
       "      <td>57279</td>\n",
       "      <td>146497</td>\n",
       "      <td>73807</td>\n",
       "      <td>0</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>14317</th>\n",
       "      <td>57280</td>\n",
       "      <td>118512</td>\n",
       "      <td>10762</td>\n",
       "      <td>0</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>4430</th>\n",
       "      <td>57281</td>\n",
       "      <td>122737</td>\n",
       "      <td>4094</td>\n",
       "      <td>0</td>\n",
       "    </tr>\n",
       "  </tbody>\n",
       "</table>\n",
       "<p>57282 rows × 4 columns</p>\n",
       "</div>"
      ],
      "text/plain": [
       "       ts_ind  HADM_ID  SUBJECT_ID  in_hospital_sepsis\n",
       "0           0   110404         268                   1\n",
       "1           1   188028         270                   0\n",
       "2           2   173727         271                   0\n",
       "3           3   164716         272                   0\n",
       "4           4   158689         273                   0\n",
       "...       ...      ...         ...                 ...\n",
       "54862   57277   182476       83967                   0\n",
       "29333   57278   118320       23200                   0\n",
       "54573   57279   146497       73807                   0\n",
       "14317   57280   118512       10762                   0\n",
       "4430    57281   122737        4094                   0\n",
       "\n",
       "[57282 rows x 4 columns]"
      ]
     },
     "execution_count": 14,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "oc = oc.sort_values(by='ts_ind')\n",
    "oc"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 15,
   "metadata": {
    "tags": []
   },
   "outputs": [
    {
     "data": {
      "text/html": [
       "<div>\n",
       "<style scoped>\n",
       "    .dataframe tbody tr th:only-of-type {\n",
       "        vertical-align: middle;\n",
       "    }\n",
       "\n",
       "    .dataframe tbody tr th {\n",
       "        vertical-align: top;\n",
       "    }\n",
       "\n",
       "    .dataframe thead th {\n",
       "        text-align: right;\n",
       "    }\n",
       "</style>\n",
       "<table border=\"1\" class=\"dataframe\">\n",
       "  <thead>\n",
       "    <tr style=\"text-align: right;\">\n",
       "      <th></th>\n",
       "      <th>index</th>\n",
       "      <th>ts_ind</th>\n",
       "      <th>HADM_ID</th>\n",
       "      <th>SUBJECT_ID</th>\n",
       "      <th>in_hospital_sepsis</th>\n",
       "    </tr>\n",
       "  </thead>\n",
       "  <tbody>\n",
       "    <tr>\n",
       "      <th>0</th>\n",
       "      <td>0</td>\n",
       "      <td>0</td>\n",
       "      <td>110404</td>\n",
       "      <td>268</td>\n",
       "      <td>1</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>1</th>\n",
       "      <td>1</td>\n",
       "      <td>1</td>\n",
       "      <td>188028</td>\n",
       "      <td>270</td>\n",
       "      <td>0</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>2</th>\n",
       "      <td>2</td>\n",
       "      <td>2</td>\n",
       "      <td>173727</td>\n",
       "      <td>271</td>\n",
       "      <td>0</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>3</th>\n",
       "      <td>3</td>\n",
       "      <td>3</td>\n",
       "      <td>164716</td>\n",
       "      <td>272</td>\n",
       "      <td>0</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>4</th>\n",
       "      <td>4</td>\n",
       "      <td>4</td>\n",
       "      <td>158689</td>\n",
       "      <td>273</td>\n",
       "      <td>0</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>...</th>\n",
       "      <td>...</td>\n",
       "      <td>...</td>\n",
       "      <td>...</td>\n",
       "      <td>...</td>\n",
       "      <td>...</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>57277</th>\n",
       "      <td>54862</td>\n",
       "      <td>57277</td>\n",
       "      <td>182476</td>\n",
       "      <td>83967</td>\n",
       "      <td>0</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>57278</th>\n",
       "      <td>29333</td>\n",
       "      <td>57278</td>\n",
       "      <td>118320</td>\n",
       "      <td>23200</td>\n",
       "      <td>0</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>57279</th>\n",
       "      <td>54573</td>\n",
       "      <td>57279</td>\n",
       "      <td>146497</td>\n",
       "      <td>73807</td>\n",
       "      <td>0</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>57280</th>\n",
       "      <td>14317</td>\n",
       "      <td>57280</td>\n",
       "      <td>118512</td>\n",
       "      <td>10762</td>\n",
       "      <td>0</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>57281</th>\n",
       "      <td>4430</td>\n",
       "      <td>57281</td>\n",
       "      <td>122737</td>\n",
       "      <td>4094</td>\n",
       "      <td>0</td>\n",
       "    </tr>\n",
       "  </tbody>\n",
       "</table>\n",
       "<p>57282 rows × 5 columns</p>\n",
       "</div>"
      ],
      "text/plain": [
       "       index  ts_ind  HADM_ID  SUBJECT_ID  in_hospital_sepsis\n",
       "0          0       0   110404         268                   1\n",
       "1          1       1   188028         270                   0\n",
       "2          2       2   173727         271                   0\n",
       "3          3       3   164716         272                   0\n",
       "4          4       4   158689         273                   0\n",
       "...      ...     ...      ...         ...                 ...\n",
       "57277  54862   57277   182476       83967                   0\n",
       "57278  29333   57278   118320       23200                   0\n",
       "57279  54573   57279   146497       73807                   0\n",
       "57280  14317   57280   118512       10762                   0\n",
       "57281   4430   57281   122737        4094                   0\n",
       "\n",
       "[57282 rows x 5 columns]"
      ]
     },
     "execution_count": 15,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "reset_oc = oc.reset_index()\n",
    "reset_oc"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 16,
   "metadata": {
    "tags": []
   },
   "outputs": [],
   "source": [
    "train_op = reset_oc['in_hospital_sepsis'][train_ind]\n",
    "valid_op = reset_oc['in_hospital_sepsis'][valid_ind]\n",
    "test_op = reset_oc['in_hospital_sepsis'][test_ind]"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 17,
   "metadata": {
    "tags": []
   },
   "outputs": [],
   "source": [
    "train_op = train_op.to_numpy(dtype='float32')\n",
    "valid_op = valid_op.to_numpy(dtype='float32')\n",
    "test_op = test_op.to_numpy(dtype='float32')"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 18,
   "metadata": {
    "tags": []
   },
   "outputs": [
    {
     "name": "stderr",
     "output_type": "stream",
     "text": [
      "2196017it [01:27, 25193.76it/s]\n"
     ]
    }
   ],
   "source": [
    "from numpy import array, asarray, zeros\n",
    "\n",
    "# GloVe\n",
    "glove_path = 'resources/glove.840B.300d.txt'\n",
    "\n",
    "embedding_model = {}\n",
    "f = open('resources/glove.840B.300d.txt', 'r', encoding=\"utf8\")\n",
    "for line in tqdm(f):\n",
    "    values = line.split()\n",
    "    word = ''.join(values[:-300])\n",
    "    coefs = np.asarray(values[-300:], dtype='float32')\n",
    "    embedding_model[word] = coefs\n",
    "f.close()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 19,
   "metadata": {
    "tags": []
   },
   "outputs": [
    {
     "name": "stderr",
     "output_type": "stream",
     "text": [
      "100%|██████████| 289088/289088 [00:00<00:00, 559318.96it/s]\n"
     ]
    }
   ],
   "source": [
    "# create a weight matrix for words in training docs\n",
    "embedding_matrix = zeros((vocab_size, 300))\n",
    "for word, i in tqdm(t.word_index.items()):\n",
    "    embedding_vector = embedding_model.get(word)\n",
    "    if embedding_vector is not None:\n",
    "        embedding_matrix[i] = embedding_vector"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 20,
   "metadata": {
    "tags": []
   },
   "outputs": [],
   "source": [
    "def get_res(y_true, y_pred):\n",
    "    precision, recall, thresholds = precision_recall_curve(y_true, y_pred)\n",
    "    pr_auc = auc(recall, precision)\n",
    "    minrp = np.minimum(precision, recall).max()\n",
    "    roc_auc = roc_auc_score(y_true, y_pred)\n",
    "    return [roc_auc, pr_auc, minrp]\n",
    "\n",
    "######################################################################################################## \n",
    "######################################################################################################## \n",
    "class_weights = compute_class_weight(class_weight='balanced', classes=[0,1], y=train_op)\n",
    "def mortality_loss(y_true, y_pred):\n",
    "    sample_weights = (1-y_true)*class_weights[0] + y_true*class_weights[1]\n",
    "    bce = K.binary_crossentropy(y_true, y_pred)\n",
    "    return K.mean(sample_weights*bce, axis=-1)\n",
    "######################################################################################################## \n",
    "######################################################################################################## \n",
    "\n",
    "# var_weights = np.sum(fore_train_op[:, V:], axis=0)\n",
    "# var_weights[var_weights==0] = var_weights.max()\n",
    "# var_weights = var_weights.max()/var_weights\n",
    "# var_weights = var_weights.reshape((1, V))\n",
    "def forecast_loss(y_true, y_pred):\n",
    "    return K.sum(y_true[:,V:]*(y_true[:,:V]-y_pred)**2, axis=-1)\n",
    "\n",
    "def get_min_loss(weight):\n",
    "    def min_loss(y_true, y_pred):\n",
    "        return weight*y_pred\n",
    "    return min_loss\n",
    "\n",
    "class CustomCallback(Callback):\n",
    "    def __init__(self, validation_data, batch_size):\n",
    "        self.val_x, self.val_y = validation_data\n",
    "        self.batch_size = batch_size\n",
    "        super(Callback, self).__init__()\n",
    "\n",
    "    def on_epoch_end(self, epoch, logs={}):\n",
    "        y_pred = self.model.predict(self.val_x, verbose=0, batch_size=self.batch_size)\n",
    "        if type(y_pred)==type([]):\n",
    "            y_pred = y_pred[0]\n",
    "        precision, recall, thresholds = precision_recall_curve(self.val_y, y_pred)\n",
    "        pr_auc = auc(recall, precision)\n",
    "        roc_auc = roc_auc_score(self.val_y, y_pred)\n",
    "        logs['custom_metric'] = pr_auc + roc_auc\n",
    "        print ('val_aucs:', pr_auc, roc_auc)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 21,
   "metadata": {
    "tags": []
   },
   "outputs": [],
   "source": [
    "import tensorflow as tf\n",
    "import numpy as np\n",
    "import tensorflow.keras.backend as K\n",
    "from tensorflow.keras.layers import Embedding, Activation, Dropout, Softmax, Layer, InputSpec, Input, Dense, Lambda, TimeDistributed, Concatenate, Add\n",
    "from tensorflow.keras import initializers, regularizers, constraints, Model\n",
    "from tensorflow.python.keras.utils import tf_utils\n",
    "from tensorflow.python.ops import array_ops\n",
    "from tensorflow import nn\n",
    "\n",
    "    \n",
    "class CVE(Layer):\n",
    "    def __init__(self, hid_units, output_dim):\n",
    "        self.hid_units = hid_units\n",
    "        self.output_dim = output_dim\n",
    "        super(CVE, self).__init__()\n",
    "        \n",
    "    def build(self, input_shape): \n",
    "        self.W1 = self.add_weight(name='CVE_W1',\n",
    "                            shape=(1, self.hid_units),\n",
    "                            initializer='glorot_uniform',\n",
    "                            trainable=True)\n",
    "        self.b1 = self.add_weight(name='CVE_b1',\n",
    "                            shape=(self.hid_units,),\n",
    "                            initializer='zeros',\n",
    "                            trainable=True)\n",
    "        self.W2 = self.add_weight(name='CVE_W2',\n",
    "                            shape=(self.hid_units, self.output_dim),\n",
    "                            initializer='glorot_uniform',\n",
    "                            trainable=True)\n",
    "        super(CVE, self).build(input_shape)\n",
    "        \n",
    "    def call(self, x):\n",
    "        x = K.expand_dims(x, axis=-1)\n",
    "        x = K.dot(K.tanh(K.bias_add(K.dot(x, self.W1), self.b1)), self.W2)\n",
    "        return x\n",
    "        \n",
    "    def compute_output_shape(self, input_shape):\n",
    "        return input_shape + (self.output_dim,)\n",
    "    \n",
    "    \n",
    "class Attention(Layer):\n",
    "    \n",
    "    def __init__(self, hid_dim):\n",
    "        self.hid_dim = hid_dim\n",
    "        super(Attention, self).__init__()\n",
    "\n",
    "    def build(self, input_shape):\n",
    "        d = input_shape.as_list()[-1]\n",
    "        self.W = self.add_weight(shape=(d, self.hid_dim), name='Att_W',\n",
    "                                 initializer='glorot_uniform',\n",
    "                                 trainable=True)\n",
    "        self.b = self.add_weight(shape=(self.hid_dim,), name='Att_b',\n",
    "                                 initializer='zeros',\n",
    "                                 trainable=True)\n",
    "        self.u = self.add_weight(shape=(self.hid_dim,1), name='Att_u',\n",
    "                                 initializer='glorot_uniform',\n",
    "                                 trainable=True)\n",
    "        super(Attention, self).build(input_shape)\n",
    "        \n",
    "    def call(self, x, mask, mask_value=-1e30):\n",
    "        attn_weights = K.dot(K.tanh(K.bias_add(K.dot(x,self.W), self.b)), self.u)\n",
    "        mask = K.expand_dims(mask, axis=-1)\n",
    "        attn_weights = mask*attn_weights + (1-mask)*mask_value\n",
    "        attn_weights = K.softmax(attn_weights, axis=-2)\n",
    "        return attn_weights\n",
    "        \n",
    "    def compute_output_shape(self, input_shape):\n",
    "        return input_shape[:-1] + (1,)\n",
    "    \n",
    "    \n",
    "class Transformer(Layer):\n",
    "    \n",
    "    def __init__(self, N=2, h=8, dk=None, dv=None, dff=None, dropout=0):\n",
    "        self.N, self.h, self.dk, self.dv, self.dff, self.dropout = N, h, dk, dv, dff, dropout\n",
    "        self.epsilon = K.epsilon() * K.epsilon()\n",
    "        super(Transformer, self).__init__()\n",
    "\n",
    "    def build(self, input_shape):\n",
    "        d = input_shape.as_list()[-1]\n",
    "        if self.dk==None:\n",
    "            self.dk = d//self.h\n",
    "        if self.dv==None:\n",
    "            self.dv = d//self.h\n",
    "        if self.dff==None:\n",
    "            self.dff = 2*d\n",
    "        self.Wq = self.add_weight(shape=(self.N, self.h, d, self.dk), name='Wq',\n",
    "                                 initializer='glorot_uniform', trainable=True)\n",
    "        self.Wk = self.add_weight(shape=(self.N, self.h, d, self.dk), name='Wk',\n",
    "                                 initializer='glorot_uniform', trainable=True)\n",
    "        self.Wv = self.add_weight(shape=(self.N, self.h, d, self.dv), name='Wv',\n",
    "                                 initializer='glorot_uniform', trainable=True)\n",
    "        self.Wo = self.add_weight(shape=(self.N, self.dv*self.h, d), name='Wo',\n",
    "                                 initializer='glorot_uniform', trainable=True)\n",
    "        self.W1 = self.add_weight(shape=(self.N, d, self.dff), name='W1',\n",
    "                                 initializer='glorot_uniform', trainable=True)\n",
    "        self.b1 = self.add_weight(shape=(self.N, self.dff), name='b1',\n",
    "                                 initializer='zeros', trainable=True)\n",
    "        self.W2 = self.add_weight(shape=(self.N, self.dff, d), name='W2',\n",
    "                                 initializer='glorot_uniform', trainable=True)\n",
    "        self.b2 = self.add_weight(shape=(self.N, d), name='b2',\n",
    "                                 initializer='zeros', trainable=True)\n",
    "        self.gamma = self.add_weight(shape=(2*self.N,), name='gamma',\n",
    "                                 initializer='ones', trainable=True)\n",
    "        self.beta = self.add_weight(shape=(2*self.N,), name='beta',\n",
    "                                 initializer='zeros', trainable=True)\n",
    "        super(Transformer, self).build(input_shape)\n",
    "        \n",
    "    def call(self, x, mask, mask_value=-1e-30):\n",
    "        mask = K.expand_dims(mask, axis=-2)\n",
    "        for i in range(self.N):\n",
    "            # MHA\n",
    "            mha_ops = []\n",
    "            for j in range(self.h):\n",
    "                q = K.dot(x, self.Wq[i,j,:,:])\n",
    "                k = K.permute_dimensions(K.dot(x, self.Wk[i,j,:,:]), (0,2,1))\n",
    "                v = K.dot(x, self.Wv[i,j,:,:])\n",
    "                A = K.batch_dot(q,k)\n",
    "                # Mask unobserved steps.\n",
    "                A = mask*A + (1-mask)*mask_value\n",
    "                # Mask for attention dropout.\n",
    "                def dropped_A():\n",
    "                    dp_mask = K.cast((K.random_uniform(shape=array_ops.shape(A))>=self.dropout), K.floatx())\n",
    "                    return A*dp_mask + (1-dp_mask)*mask_value\n",
    "                A = sc.smart_cond(K.learning_phase(), dropped_A, lambda: array_ops.identity(A))\n",
    "                A = K.softmax(A, axis=-1)\n",
    "                mha_ops.append(K.batch_dot(A,v))\n",
    "            conc = K.concatenate(mha_ops, axis=-1)\n",
    "            proj = K.dot(conc, self.Wo[i,:,:])\n",
    "            # Dropout.\n",
    "            proj = sc.smart_cond(K.learning_phase(), lambda: array_ops.identity(nn.dropout(proj, rate=self.dropout)),\\\n",
    "                                       lambda: array_ops.identity(proj))\n",
    "            # Add & LN\n",
    "            x = x+proj\n",
    "            mean = K.mean(x, axis=-1, keepdims=True)\n",
    "            variance = K.mean(K.square(x - mean), axis=-1, keepdims=True)\n",
    "            std = K.sqrt(variance + self.epsilon)\n",
    "            x = (x - mean) / std\n",
    "            x = x*self.gamma[2*i] + self.beta[2*i]\n",
    "            # FFN\n",
    "            ffn_op = K.bias_add(K.dot(K.relu(K.bias_add(K.dot(x, self.W1[i,:,:]), self.b1[i,:])), \n",
    "                           self.W2[i,:,:]), self.b2[i,:,])\n",
    "            # Dropout.\n",
    "            ffn_op = sc.smart_cond(K.learning_phase(), lambda: array_ops.identity(nn.dropout(ffn_op, rate=self.dropout)),\\\n",
    "                                       lambda: array_ops.identity(ffn_op))\n",
    "            # Add & LN\n",
    "            x = x+ffn_op\n",
    "            mean = K.mean(x, axis=-1, keepdims=True)\n",
    "            variance = K.mean(K.square(x - mean), axis=-1, keepdims=True)\n",
    "            std = K.sqrt(variance + self.epsilon)\n",
    "            x = (x - mean) / std\n",
    "            x = x*self.gamma[2*i+1] + self.beta[2*i+1]            \n",
    "        return x\n",
    "        \n",
    "    def compute_output_shape(self, input_shape):\n",
    "        return input_shape\n",
    "\n",
    "\n",
    "def build_strats(D, max_len, V, d, N, he, dropout, forecast=False):\n",
    "    demo = Input(shape=(D,))\n",
    "    demo_enc = Dense(2*d, activation='tanh')(demo)\n",
    "    demo_enc = Dense(d, activation='tanh')(demo_enc)\n",
    "    varis = Input(shape=(max_len,))\n",
    "    values = Input(shape=(max_len,))\n",
    "    times = Input(shape=(max_len,))\n",
    "    varis_emb = Embedding(V+1, d)(varis)\n",
    "    cve_units = int(np.sqrt(d))\n",
    "    values_emb = CVE(cve_units, d)(values)\n",
    "    times_emb = CVE(cve_units, d)(times)\n",
    "    comb_emb = Add()([varis_emb, values_emb, times_emb]) # b, L, d\n",
    "#     demo_enc = Lambda(lambda x:K.expand_dims(x, axis=-2))(demo_enc) # b, 1, d\n",
    "#     comb_emb = Concatenate(axis=-2)([demo_enc, comb_emb]) # b, L+1, d\n",
    "    mask = Lambda(lambda x:K.clip(x,0,1))(varis) # b, L\n",
    "#     mask = Lambda(lambda x:K.concatenate((K.ones_like(x)[:,0:1], x), axis=-1))(mask) # b, L+1\n",
    "    cont_emb = Transformer(N, he, dk=None, dv=None, dff=None, dropout=dropout)(comb_emb, mask=mask)\n",
    "    attn_weights = Attention(2*d)(cont_emb, mask=mask)\n",
    "    fused_emb = Lambda(lambda x:K.sum(x[0]*x[1], axis=-2))([cont_emb, attn_weights])\n",
    "    \n",
    "    \n",
    "    ## GloVe text embedding starts here\n",
    "    max_length = 24135\n",
    "    # embed text input\n",
    "    texts = Input(shape=(max_length,))\n",
    "    embedded_text = Embedding(vocab_size, 300, weights=[\n",
    "    embedding_matrix], input_length=max_length, trainable=False)(texts)\n",
    "    pooled = GlobalAveragePooling1D()(embedded_text)\n",
    "    # flattened = Flatten()(embedded_text)\n",
    "    \n",
    "    text_0 = Dense(100, activation='relu')(pooled)\n",
    "    \n",
    "    \n",
    "    # hidden_states = LSTM(64, return_sequences=True, name='lstm_layer')(embedded_text)\n",
    "    # pooled = AveragePooling1D()(hidden_states)\n",
    "    # text_0 = Dense(100, activation='relu')(hidden_states)\n",
    "    \n",
    "    # text_1 = Flatten()(hidden_states)\n",
    "    \n",
    "    # text_enc = Dense(22528, activation='relu')(texts)\n",
    "    # text_enc = Dense(10000, activation='relu')(texts)\n",
    "    # text_enc = Dense(5000, activation='relu')(texts)\n",
    "    # text_enc = Dense(1000, activation='relu')(texts)\n",
    "    \n",
    "    # text_enc = Dense(d, activation='relu')(text_1)\n",
    "    \n",
    "    text_enc = Dense(d, activation='relu')(text_0)\n",
    "    \n",
    "    # text_enc = Dense(d * max_length, activation='relu')(text_0)\n",
    "    # reshaped_text_enc = Reshape((max_length, d))(text_enc)\n",
    "    \n",
    "    # reshaped_text_enc = Reshape((d,))(text_enc)\n",
    "    \n",
    "    # conc = Concatenate(axis=-1)([fused_emb, text_enc, demo_enc])\n",
    "    conc = Concatenate(axis=-1)([fused_emb, text_enc, demo_enc])\n",
    "    fore_op = Dense(V)(conc)\n",
    "    op = Dense(1, activation='sigmoid')(fore_op)\n",
    "    model = Model([demo, times, values, varis, texts], op)\n",
    "    if forecast:\n",
    "        fore_model = Model([demo, times, values, varis, texts], fore_op)\n",
    "        return [model, fore_model]\n",
    "    return model\n",
    "\n",
    "# To tune:\n",
    "# 1. Transformer parameters. (N, h, dropout)\n",
    "# 2. Normalization"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "tags": []
   },
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "Repeat 0 ld 10\n",
      "Num train: 3655 Num valid: 926\n",
      "Exp_GloVe/M/models/classification/new_mimic_iii_24hm_strats_no_interp_with_ss_repeat0_10ld.h5\n"
     ]
    },
    {
     "name": "stderr",
     "output_type": "stream",
     "text": [
      "2023-12-13 12:42:38.015998: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 30503 MB memory:  -> device: 0, name: Tesla V100-SXM2-32GB, pci bus id: 0000:3b:00.0, compute capability: 7.0\n",
      "2023-12-13 12:42:38.017144: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 30503 MB memory:  -> device: 1, name: Tesla V100-SXM2-32GB, pci bus id: 0000:b2:00.0, compute capability: 7.0\n"
     ]
    },
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "Epoch 1/1000\n"
     ]
    },
    {
     "name": "stderr",
     "output_type": "stream",
     "text": [
      "2023-12-13 12:42:55.730173: I tensorflow/compiler/xla/service/service.cc:169] XLA service 0x5643cc519970 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:\n",
      "2023-12-13 12:42:55.757924: I tensorflow/compiler/xla/service/service.cc:177]   StreamExecutor device (0): Tesla V100-SXM2-32GB, Compute Capability 7.0\n",
      "2023-12-13 12:42:55.757951: I tensorflow/compiler/xla/service/service.cc:177]   StreamExecutor device (1): Tesla V100-SXM2-32GB, Compute Capability 7.0\n",
      "2023-12-13 12:42:59.864342: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.\n",
      "2023-12-13 12:43:03.723629: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8700\n",
      "2023-12-13 12:43:04.629189: I ./tensorflow/compiler/jit/device_compiler.h:180] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.\n"
     ]
    },
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "115/115 [==============================] - ETA: 0s - loss: 0.5665val_aucs: 0.3840374912538482 0.8424136167725911\n",
      "115/115 [==============================] - 34s 85ms/step - loss: 0.5665 - custom_metric: 1.2265\n",
      "Epoch 2/1000\n",
      "115/115 [==============================] - ETA: 0s - loss: 0.4606val_aucs: 0.42458854956214503 0.8516034772445028\n",
      "115/115 [==============================] - 6s 54ms/step - loss: 0.4606 - custom_metric: 1.2762\n",
      "Epoch 3/1000\n",
      "115/115 [==============================] - ETA: 0s - loss: 0.4216val_aucs: 0.40067753219141927 0.8450434655562861\n",
      "115/115 [==============================] - 6s 51ms/step - loss: 0.4216 - custom_metric: 1.2457\n",
      "Epoch 4/1000\n",
      "115/115 [==============================] - ETA: 0s - loss: 0.3771val_aucs: 0.45092563377993866 0.8504200452918401\n",
      "115/115 [==============================] - 6s 55ms/step - loss: 0.3771 - custom_metric: 1.3013\n",
      "Epoch 5/1000\n",
      "115/115 [==============================] - ETA: 0s - loss: 0.3489val_aucs: 0.4444748184520283 0.8407334356052305\n",
      "115/115 [==============================] - 6s 51ms/step - loss: 0.3489 - custom_metric: 1.2852\n",
      "Epoch 6/1000\n",
      "115/115 [==============================] - ETA: 0s - loss: 0.3317val_aucs: 0.46548953841903473 0.8479216889473299\n",
      "115/115 [==============================] - 6s 54ms/step - loss: 0.3317 - custom_metric: 1.3134\n",
      "Epoch 7/1000\n",
      "115/115 [==============================] - ETA: 0s - loss: 0.2686val_aucs: 0.46047570685318323 0.8468551391628315\n",
      "115/115 [==============================] - 6s 50ms/step - loss: 0.2686 - custom_metric: 1.3073\n",
      "Epoch 8/1000\n",
      "115/115 [==============================] - ETA: 0s - loss: 0.2364val_aucs: 0.4456516437440775 0.8463583899481336\n",
      "115/115 [==============================] - 6s 51ms/step - loss: 0.2364 - custom_metric: 1.2920\n",
      "Epoch 9/1000\n",
      "115/115 [==============================] - ETA: 0s - loss: 0.2173val_aucs: 0.433344648794848 0.8452041785375117\n",
      "115/115 [==============================] - 6s 51ms/step - loss: 0.2173 - custom_metric: 1.2785\n",
      "Epoch 10/1000\n",
      "115/115 [==============================] - ETA: 0s - loss: 0.1684val_aucs: 0.4529329875053493 0.8487252538534589\n",
      "115/115 [==============================] - 6s 52ms/step - loss: 0.1684 - custom_metric: 1.3017\n",
      "Epoch 11/1000\n",
      "115/115 [==============================] - ETA: 0s - loss: 0.1382val_aucs: 0.41837906095894906 0.8257725180802103\n",
      "115/115 [==============================] - 6s 50ms/step - loss: 0.1382 - custom_metric: 1.2442\n",
      "Epoch 12/1000\n",
      "115/115 [==============================] - ETA: 0s - loss: 0.1549val_aucs: 0.43118760746991036 0.8474103294616114\n",
      "115/115 [==============================] - 6s 51ms/step - loss: 0.1549 - custom_metric: 1.2786\n",
      "Epoch 13/1000\n",
      "115/115 [==============================] - ETA: 0s - loss: 0.1254val_aucs: 0.4526458982130295 0.8405142815399226\n",
      "115/115 [==============================] - 6s 51ms/step - loss: 0.1254 - custom_metric: 1.2932\n",
      "Epoch 14/1000\n",
      "115/115 [==============================] - ETA: 0s - loss: 0.1120val_aucs: 0.4166070034828417 0.8440937979399519\n",
      "115/115 [==============================] - 6s 50ms/step - loss: 0.1120 - custom_metric: 1.2607\n",
      "Epoch 15/1000\n",
      "115/115 [==============================] - ETA: 0s - loss: 0.1483val_aucs: 0.39839629971739726 0.8339104390386443\n",
      "115/115 [==============================] - 6s 51ms/step - loss: 0.1483 - custom_metric: 1.2323\n",
      "Epoch 16/1000\n",
      "115/115 [==============================] - ETA: 0s - loss: 0.1389val_aucs: 0.4140026856732869 0.8378113814011249\n",
      "115/115 [==============================] - 6s 53ms/step - loss: 0.1389 - custom_metric: 1.2518\n",
      "Test res 0.853824534765438 0.39085240003175376 0.4296799224054316\n",
      "Repeat 1 ld 10\n",
      "Num train: 3655 Num valid: 926\n",
      "Exp_GloVe/M/models/classification/new_mimic_iii_24hm_strats_no_interp_with_ss_repeat1_10ld.h5\n",
      "Epoch 1/1000\n",
      "115/115 [==============================] - ETA: 0s - loss: 0.5471val_aucs: 0.5222253596168209 0.906710747636448\n",
      "115/115 [==============================] - 17s 72ms/step - loss: 0.5471 - custom_metric: 1.4289\n",
      "Epoch 2/1000\n",
      "115/115 [==============================] - ETA: 0s - loss: 0.4508val_aucs: 0.5397621719335528 0.905446319819036\n",
      "115/115 [==============================] - 6s 54ms/step - loss: 0.4508 - custom_metric: 1.4452\n",
      "Epoch 3/1000\n",
      "115/115 [==============================] - ETA: 0s - loss: 0.4039val_aucs: 0.5904321908920523 0.9094600081201787\n",
      "115/115 [==============================] - 6s 54ms/step - loss: 0.4039 - custom_metric: 1.4999\n",
      "Epoch 4/1000\n",
      "115/115 [==============================] - ETA: 0s - loss: 0.3628val_aucs: 0.5646873604316707 0.9083811843860564\n",
      "115/115 [==============================] - 6s 52ms/step - loss: 0.3628 - custom_metric: 1.4731\n",
      "Epoch 5/1000\n",
      "115/115 [==============================] - ETA: 0s - loss: 0.3159val_aucs: 0.5750286494824984 0.9013166289658373\n",
      "115/115 [==============================] - 6s 52ms/step - loss: 0.3159 - custom_metric: 1.4763\n",
      "Epoch 6/1000\n",
      "115/115 [==============================] - ETA: 0s - loss: 0.2653val_aucs: 0.5471722312219293 0.8924888347543647\n",
      "115/115 [==============================] - 6s 52ms/step - loss: 0.2653 - custom_metric: 1.4397\n",
      "Epoch 7/1000\n",
      "115/115 [==============================] - ETA: 0s - loss: 0.2383val_aucs: 0.5442244937156782 0.8907255959631112\n",
      "115/115 [==============================] - 6s 53ms/step - loss: 0.2383 - custom_metric: 1.4350\n",
      "Epoch 8/1000\n",
      "115/115 [==============================] - ETA: 0s - loss: 0.2215val_aucs: 0.533308291664558 0.8809929818456006\n",
      "115/115 [==============================] - 6s 50ms/step - loss: 0.2215 - custom_metric: 1.4143\n",
      "Epoch 9/1000\n",
      "115/115 [==============================] - ETA: 0s - loss: 0.1957val_aucs: 0.4898128168761978 0.8649150281306188\n",
      "115/115 [==============================] - 6s 51ms/step - loss: 0.1957 - custom_metric: 1.3547\n",
      "Epoch 10/1000\n",
      "115/115 [==============================] - ETA: 0s - loss: 0.1653val_aucs: 0.4917445080319134 0.8772925004350095\n",
      "115/115 [==============================] - 6s 51ms/step - loss: 0.1653 - custom_metric: 1.3690\n",
      "Epoch 11/1000\n",
      "115/115 [==============================] - ETA: 0s - loss: 0.1355val_aucs: 0.5036053314766045 0.878603329273244\n",
      "115/115 [==============================] - 6s 50ms/step - loss: 0.1355 - custom_metric: 1.3822\n",
      "Epoch 12/1000\n",
      "115/115 [==============================] - ETA: 0s - loss: 0.0952val_aucs: 0.46541783721284025 0.8486630705875531\n",
      "115/115 [==============================] - 6s 51ms/step - loss: 0.0952 - custom_metric: 1.3141\n",
      "Epoch 13/1000\n",
      "115/115 [==============================] - ETA: 0s - loss: 0.0683val_aucs: 0.4981409501715448 0.8701003422075286\n",
      "115/115 [==============================] - 6s 54ms/step - loss: 0.0683 - custom_metric: 1.3682\n",
      "Test res 0.8637962242699857 0.43044859584223094 0.4605137963843958\n",
      "Repeat 2 ld 10\n",
      "Num train: 3655 Num valid: 926\n",
      "Exp_GloVe/M/models/classification/new_mimic_iii_24hm_strats_no_interp_with_ss_repeat2_10ld.h5\n",
      "Epoch 1/1000\n",
      "115/115 [==============================] - ETA: 0s - loss: 0.5335val_aucs: 0.3679748353546483 0.8542405437352246\n",
      "115/115 [==============================] - 15s 72ms/step - loss: 0.5335 - custom_metric: 1.2222\n",
      "Epoch 2/1000\n",
      "115/115 [==============================] - ETA: 0s - loss: 0.4464val_aucs: 0.40323144649959697 0.8737440898345155\n",
      "115/115 [==============================] - 6s 54ms/step - loss: 0.4464 - custom_metric: 1.2770\n",
      "Epoch 3/1000\n",
      "114/115 [============================>.] - ETA: 0s - loss: 0.4038val_aucs: 0.4173519095756304 0.8676418439716312\n",
      "115/115 [==============================] - 6s 54ms/step - loss: 0.4033 - custom_metric: 1.2850\n",
      "Epoch 4/1000\n",
      "115/115 [==============================] - ETA: 0s - loss: 0.3596val_aucs: 0.41624827683812193 0.8552748226950354\n",
      "115/115 [==============================] - 6s 52ms/step - loss: 0.3596 - custom_metric: 1.2715\n",
      "Epoch 5/1000\n",
      "115/115 [==============================] - ETA: 0s - loss: 0.3298val_aucs: 0.42439166159054725 0.854580378250591\n",
      "115/115 [==============================] - 6s 51ms/step - loss: 0.3298 - custom_metric: 1.2790\n",
      "Epoch 6/1000\n",
      "115/115 [==============================] - ETA: 0s - loss: 0.2855val_aucs: 0.42312984900760064 0.8408540189125294\n",
      "115/115 [==============================] - 6s 51ms/step - loss: 0.2855 - custom_metric: 1.2640\n",
      "Epoch 7/1000\n",
      "115/115 [==============================] - ETA: 0s - loss: 0.2409val_aucs: 0.36356188994302624 0.8291962174940898\n",
      "115/115 [==============================] - 6s 51ms/step - loss: 0.2409 - custom_metric: 1.1928\n",
      "Epoch 8/1000\n",
      "115/115 [==============================] - ETA: 0s - loss: 0.1950val_aucs: 0.4012332876267302 0.838105791962175\n",
      "115/115 [==============================] - 6s 51ms/step - loss: 0.1950 - custom_metric: 1.2393\n",
      "Epoch 9/1000\n",
      "115/115 [==============================] - ETA: 0s - loss: 0.1790val_aucs: 0.4157550495489071 0.8311022458628842\n",
      "115/115 [==============================] - 6s 51ms/step - loss: 0.1790 - custom_metric: 1.2469\n",
      "Epoch 10/1000\n",
      "115/115 [==============================] - ETA: 0s - loss: 0.1582val_aucs: 0.38870821443969744 0.8223256501182032\n",
      "115/115 [==============================] - 6s 52ms/step - loss: 0.1582 - custom_metric: 1.2110\n",
      "Epoch 11/1000\n",
      "115/115 [==============================] - ETA: 0s - loss: 0.1368val_aucs: 0.37485451994582747 0.8177304964539007\n",
      "115/115 [==============================] - 6s 51ms/step - loss: 0.1368 - custom_metric: 1.1926\n",
      "Epoch 12/1000\n",
      "115/115 [==============================] - ETA: 0s - loss: 0.1346val_aucs: 0.3352512684135528 0.8263297872340426\n",
      "115/115 [==============================] - 6s 51ms/step - loss: 0.1346 - custom_metric: 1.1616\n",
      "Epoch 13/1000\n",
      "115/115 [==============================] - ETA: 0s - loss: 0.0995val_aucs: 0.3711396632573894 0.8384308510638298\n",
      "115/115 [==============================] - 6s 53ms/step - loss: 0.0995 - custom_metric: 1.2096\n",
      "Test res 0.8618150505624702 0.4080965004927668 0.4400386847195358\n",
      "Repeat 3 ld 10\n",
      "Num train: 3655 Num valid: 926\n",
      "Exp_GloVe/M/models/classification/new_mimic_iii_24hm_strats_no_interp_with_ss_repeat3_10ld.h5\n",
      "Epoch 1/1000\n",
      "115/115 [==============================] - ETA: 0s - loss: 0.5262val_aucs: 0.510059562779666 0.8673799665335337\n",
      "115/115 [==============================] - 16s 72ms/step - loss: 0.5262 - custom_metric: 1.3774\n",
      "Epoch 2/1000\n",
      "114/115 [============================>.] - ETA: 0s - loss: 0.4112val_aucs: 0.5062308008286047 0.8794718649615868\n",
      "115/115 [==============================] - 6s 54ms/step - loss: 0.4107 - custom_metric: 1.3857\n",
      "Epoch 3/1000\n",
      "115/115 [==============================] - ETA: 0s - loss: 0.3771val_aucs: 0.545413355791289 0.885102536855862\n",
      "115/115 [==============================] - 6s 53ms/step - loss: 0.3771 - custom_metric: 1.4305\n",
      "Epoch 4/1000\n",
      "115/115 [==============================] - ETA: 0s - loss: 0.3304val_aucs: 0.526156963914963 0.8897316575672077\n",
      "115/115 [==============================] - 6s 51ms/step - loss: 0.3304 - custom_metric: 1.4159\n",
      "Epoch 5/1000\n",
      "115/115 [==============================] - ETA: 0s - loss: 0.2862val_aucs: 0.5406359886497026 0.8852246772440243\n",
      "115/115 [==============================] - 6s 51ms/step - loss: 0.2862 - custom_metric: 1.4259\n",
      "Epoch 6/1000\n",
      "115/115 [==============================] - ETA: 0s - loss: 0.2631val_aucs: 0.5250519285923424 0.8795329351556678\n",
      "115/115 [==============================] - 6s 52ms/step - loss: 0.2631 - custom_metric: 1.4046\n",
      "Epoch 7/1000\n",
      "115/115 [==============================] - ETA: 0s - loss: 0.2231val_aucs: 0.509054198277871 0.8717159503132902\n",
      "115/115 [==============================] - 6s 51ms/step - loss: 0.2231 - custom_metric: 1.3808\n",
      "Epoch 8/1000\n",
      "115/115 [==============================] - ETA: 0s - loss: 0.1959val_aucs: 0.5028568080500727 0.874769460017344\n",
      "115/115 [==============================] - 6s 52ms/step - loss: 0.1959 - custom_metric: 1.3776\n",
      "Epoch 9/1000\n",
      "115/115 [==============================] - ETA: 0s - loss: 0.1946val_aucs: 0.5477581535035492 0.8725220768751604\n",
      "115/115 [==============================] - 6s 51ms/step - loss: 0.1946 - custom_metric: 1.4203\n",
      "Epoch 10/1000\n",
      "115/115 [==============================] - ETA: 0s - loss: 0.1393val_aucs: 0.4878932103372106 0.872277796098836\n",
      "115/115 [==============================] - 6s 50ms/step - loss: 0.1393 - custom_metric: 1.3602\n",
      "Epoch 11/1000\n",
      "115/115 [==============================] - ETA: 0s - loss: 0.1102val_aucs: 0.48139307542259674 0.8659997801473013\n",
      "115/115 [==============================] - 6s 52ms/step - loss: 0.1102 - custom_metric: 1.3474\n",
      "Epoch 12/1000\n",
      "115/115 [==============================] - ETA: 0s - loss: 0.0945val_aucs: 0.44959238544240204 0.8487535573388052\n",
      "115/115 [==============================] - 6s 51ms/step - loss: 0.0945 - custom_metric: 1.2983\n",
      "Epoch 13/1000\n",
      "115/115 [==============================] - ETA: 0s - loss: 0.1629val_aucs: 0.456134367247002 0.8566316123752642\n",
      "115/115 [==============================] - 6s 52ms/step - loss: 0.1629 - custom_metric: 1.3128\n",
      "Test res 0.858039694530876 0.39816343525697545 0.43359375\n",
      "Repeat 4 ld 10\n",
      "Num train: 3655 Num valid: 926\n",
      "Exp_GloVe/M/models/classification/new_mimic_iii_24hm_strats_no_interp_with_ss_repeat4_10ld.h5\n",
      "Epoch 1/1000\n",
      "115/115 [==============================] - ETA: 0s - loss: 0.4996val_aucs: 0.40018854144349 0.839052699251125\n",
      "115/115 [==============================] - 16s 73ms/step - loss: 0.4996 - custom_metric: 1.2392\n",
      "Epoch 2/1000\n",
      "115/115 [==============================] - ETA: 0s - loss: 0.3980val_aucs: 0.4219728533920814 0.8533004361197505\n",
      "115/115 [==============================] - 6s 53ms/step - loss: 0.3980 - custom_metric: 1.2753\n",
      "Epoch 3/1000\n",
      "115/115 [==============================] - ETA: 0s - loss: 0.3604val_aucs: 0.4004048223113239 0.842684134623738\n",
      "115/115 [==============================] - 6s 51ms/step - loss: 0.3604 - custom_metric: 1.2431\n",
      "Epoch 4/1000\n",
      "115/115 [==============================] - ETA: 0s - loss: 0.3047val_aucs: 0.38145042887802827 0.8481226000382256\n",
      "115/115 [==============================] - 6s 52ms/step - loss: 0.3047 - custom_metric: 1.2296\n",
      "Epoch 5/1000\n",
      "115/115 [==============================] - ETA: 0s - loss: 0.2779val_aucs: 0.38121916277239415 0.8417111184473443\n",
      "115/115 [==============================] - 6s 51ms/step - loss: 0.2779 - custom_metric: 1.2229\n",
      "Epoch 6/1000\n",
      "115/115 [==============================] - ETA: 0s - loss: 0.2294val_aucs: 0.39833185133143156 0.8532309349642938\n",
      "115/115 [==============================] - 6s 51ms/step - loss: 0.2294 - custom_metric: 1.2516\n",
      "Epoch 7/1000\n",
      "115/115 [==============================] - ETA: 0s - loss: 0.1987val_aucs: 0.35039567717305903 0.8499296300801\n",
      "115/115 [==============================] - 6s 52ms/step - loss: 0.1987 - custom_metric: 1.2003\n",
      "Epoch 8/1000\n",
      "115/115 [==============================] - ETA: 0s - loss: 0.1764val_aucs: 0.393729434078755 0.8450124233315379\n",
      "115/115 [==============================] - 6s 52ms/step - loss: 0.1764 - custom_metric: 1.2387\n",
      "Epoch 9/1000\n",
      "115/115 [==============================] - ETA: 0s - loss: 0.1456val_aucs: 0.3746950777698716 0.8558024777161921\n",
      "115/115 [==============================] - 6s 50ms/step - loss: 0.1456 - custom_metric: 1.2305\n",
      "Epoch 10/1000\n",
      "115/115 [==============================] - ETA: 0s - loss: 0.1443val_aucs: 0.39694937799728547 0.8587736521119663\n",
      "115/115 [==============================] - 6s 52ms/step - loss: 0.1443 - custom_metric: 1.2557\n",
      "Epoch 11/1000\n",
      "115/115 [==============================] - ETA: 0s - loss: 0.0854val_aucs: 0.3844753252299138 0.8387051934738415\n",
      "115/115 [==============================] - 6s 51ms/step - loss: 0.0854 - custom_metric: 1.2232\n",
      "Epoch 12/1000\n",
      "115/115 [==============================] - ETA: 0s - loss: 0.0763val_aucs: 0.3499424932515851 0.8418674960471217\n",
      "115/115 [==============================] - 6s 53ms/step - loss: 0.0763 - custom_metric: 1.1918\n",
      "Test res 0.8635071355911919 0.4387797784325842 0.455609756097561\n",
      "Repeat 5 ld 10\n",
      "Num train: 3655 Num valid: 926\n",
      "Exp_GloVe/M/models/classification/new_mimic_iii_24hm_strats_no_interp_with_ss_repeat5_10ld.h5\n",
      "Epoch 1/1000\n",
      "115/115 [==============================] - ETA: 0s - loss: 0.5331val_aucs: 0.46678287015268105 0.8713644878241151\n",
      "115/115 [==============================] - 15s 72ms/step - loss: 0.5331 - custom_metric: 1.3381\n",
      "Epoch 2/1000\n",
      "115/115 [==============================] - ETA: 0s - loss: 0.4275val_aucs: 0.49506890829922895 0.8791284629793946\n",
      "115/115 [==============================] - 6s 54ms/step - loss: 0.4275 - custom_metric: 1.3742\n",
      "Epoch 3/1000\n",
      "115/115 [==============================] - ETA: 0s - loss: 0.3892val_aucs: 0.49296298551350315 0.8803115449078182\n",
      "115/115 [==============================] - 6s 52ms/step - loss: 0.3892 - custom_metric: 1.3733\n",
      "Epoch 4/1000\n",
      "115/115 [==============================] - ETA: 0s - loss: 0.3548val_aucs: 0.4884997609453995 0.8813467415951888\n",
      "115/115 [==============================] - 6s 51ms/step - loss: 0.3548 - custom_metric: 1.3698\n",
      "Epoch 5/1000\n",
      "115/115 [==============================] - ETA: 0s - loss: 0.3247val_aucs: 0.48991780897608356 0.8791161392093069\n",
      "115/115 [==============================] - 6s 51ms/step - loss: 0.3247 - custom_metric: 1.3690\n",
      "Epoch 6/1000\n",
      "115/115 [==============================] - ETA: 0s - loss: 0.2801val_aucs: 0.4944119640686557 0.8813590653652765\n",
      "115/115 [==============================] - 6s 53ms/step - loss: 0.2801 - custom_metric: 1.3758\n",
      "Epoch 7/1000\n",
      "115/115 [==============================] - ETA: 0s - loss: 0.2402val_aucs: 0.4236864896525005 0.8644015577245391\n",
      "115/115 [==============================] - 6s 51ms/step - loss: 0.2402 - custom_metric: 1.2881\n",
      "Epoch 8/1000\n",
      "115/115 [==============================] - ETA: 0s - loss: 0.2337val_aucs: 0.4706035535301373 0.8618258897762004\n",
      "115/115 [==============================] - 6s 52ms/step - loss: 0.2337 - custom_metric: 1.3324\n",
      "Epoch 9/1000\n",
      "115/115 [==============================] - ETA: 0s - loss: 0.2135val_aucs: 0.47021171756180147 0.8627501725327813\n",
      "115/115 [==============================] - 6s 50ms/step - loss: 0.2135 - custom_metric: 1.3330\n",
      "Epoch 10/1000\n",
      "114/115 [============================>.] - ETA: 0s - loss: 0.2078val_aucs: 0.40080330833013805 0.8485531893916988\n",
      "115/115 [==============================] - 6s 51ms/step - loss: 0.2075 - custom_metric: 1.2494\n",
      "Epoch 11/1000\n",
      "115/115 [==============================] - ETA: 0s - loss: 0.1505val_aucs: 0.4205359242658816 0.8509932958690724\n",
      "115/115 [==============================] - 6s 51ms/step - loss: 0.1505 - custom_metric: 1.2715\n",
      "Epoch 12/1000\n",
      "115/115 [==============================] - ETA: 0s - loss: 0.1367val_aucs: 0.4294876678438649 0.8582889677610175\n",
      "115/115 [==============================] - 6s 50ms/step - loss: 0.1367 - custom_metric: 1.2878\n",
      "Epoch 13/1000\n",
      "115/115 [==============================] - ETA: 0s - loss: 0.1147val_aucs: 0.4136565729019246 0.8410849847185251\n",
      "115/115 [==============================] - 6s 51ms/step - loss: 0.1147 - custom_metric: 1.2547\n",
      "Epoch 14/1000\n",
      "115/115 [==============================] - ETA: 0s - loss: 0.0842val_aucs: 0.43642477256521683 0.8529774228531992\n",
      "115/115 [==============================] - 6s 52ms/step - loss: 0.0842 - custom_metric: 1.2894\n",
      "Epoch 15/1000\n",
      "115/115 [==============================] - ETA: 0s - loss: 0.0579val_aucs: 0.416210812423889 0.8508823819382825\n",
      "115/115 [==============================] - 10s 88ms/step - loss: 0.0579 - custom_metric: 1.2671\n",
      "Epoch 16/1000\n",
      "115/115 [==============================] - ETA: 0s - loss: 0.1076val_aucs: 0.43287527502850015 0.848343685300207\n",
      "115/115 [==============================] - 6s 52ms/step - loss: 0.1076 - custom_metric: 1.2812\n",
      "Test res 0.855236226214696 0.4150148069035296 0.4482421875\n",
      "Repeat 6 ld 10\n",
      "Num train: 3655 Num valid: 926\n",
      "Exp_GloVe/M/models/classification/new_mimic_iii_24hm_strats_no_interp_with_ss_repeat6_10ld.h5\n",
      "Epoch 1/1000\n",
      "115/115 [==============================] - ETA: 0s - loss: 0.5725val_aucs: 0.3709407797670876 0.8418899233643675\n",
      "115/115 [==============================] - 16s 73ms/step - loss: 0.5725 - custom_metric: 1.2128\n",
      "Epoch 2/1000\n",
      "115/115 [==============================] - ETA: 0s - loss: 0.4857val_aucs: 0.4009761275787925 0.8476787636962442\n",
      "115/115 [==============================] - 6s 54ms/step - loss: 0.4857 - custom_metric: 1.2487\n",
      "Epoch 3/1000\n",
      "115/115 [==============================] - ETA: 0s - loss: 0.4462val_aucs: 0.41070489274794286 0.8464500601684717\n",
      "115/115 [==============================] - 6s 53ms/step - loss: 0.4462 - custom_metric: 1.2572\n",
      "Epoch 4/1000\n",
      "115/115 [==============================] - ETA: 0s - loss: 0.3942val_aucs: 0.3991029906811113 0.8611058331749952\n",
      "115/115 [==============================] - 6s 54ms/step - loss: 0.3942 - custom_metric: 1.2602\n",
      "Epoch 5/1000\n",
      "115/115 [==============================] - ETA: 0s - loss: 0.3555val_aucs: 0.36654764837697185 0.8463613908417253\n",
      "115/115 [==============================] - 6s 52ms/step - loss: 0.3555 - custom_metric: 1.2129\n",
      "Epoch 6/1000\n",
      "115/115 [==============================] - ETA: 0s - loss: 0.3144val_aucs: 0.40483915322800945 0.8510988662993224\n",
      "115/115 [==============================] - 6s 51ms/step - loss: 0.3144 - custom_metric: 1.2559\n",
      "Epoch 7/1000\n",
      "115/115 [==============================] - ETA: 0s - loss: 0.2732val_aucs: 0.40647302619618014 0.8412312369371081\n",
      "115/115 [==============================] - 6s 51ms/step - loss: 0.2732 - custom_metric: 1.2477\n",
      "Epoch 8/1000\n",
      "115/115 [==============================] - ETA: 0s - loss: 0.2152val_aucs: 0.3709212299980088 0.8381024764076257\n",
      "115/115 [==============================] - 6s 52ms/step - loss: 0.2152 - custom_metric: 1.2090\n",
      "Epoch 9/1000\n",
      "115/115 [==============================] - ETA: 0s - loss: 0.1874val_aucs: 0.3952323109291052 0.8395971879156374\n",
      "115/115 [==============================] - 6s 51ms/step - loss: 0.1874 - custom_metric: 1.2348\n",
      "Epoch 10/1000\n",
      "115/115 [==============================] - ETA: 0s - loss: 0.1978val_aucs: 0.34286278544455295 0.812882386471594\n",
      "115/115 [==============================] - 6s 51ms/step - loss: 0.1978 - custom_metric: 1.1557\n",
      "Epoch 11/1000\n",
      "115/115 [==============================] - ETA: 0s - loss: 0.1789val_aucs: 0.3553640891689563 0.8089176008613591\n",
      "115/115 [==============================] - 6s 50ms/step - loss: 0.1789 - custom_metric: 1.1643\n",
      "Epoch 12/1000\n",
      "115/115 [==============================] - ETA: 0s - loss: 0.1444val_aucs: 0.3849868260670645 0.8187345620368611\n",
      "115/115 [==============================] - 6s 50ms/step - loss: 0.1444 - custom_metric: 1.2037\n",
      "Epoch 13/1000\n",
      "115/115 [==============================] - ETA: 0s - loss: 0.1103val_aucs: 0.4247992985566483 0.8427766166318322\n",
      "115/115 [==============================] - 6s 53ms/step - loss: 0.1103 - custom_metric: 1.2676\n",
      "Epoch 14/1000\n",
      "115/115 [==============================] - ETA: 0s - loss: 0.0783val_aucs: 0.3975606972876393 0.8258914434099689\n",
      "115/115 [==============================] - 6s 50ms/step - loss: 0.0783 - custom_metric: 1.2235\n",
      "Epoch 15/1000\n",
      "115/115 [==============================] - ETA: 0s - loss: 0.0784val_aucs: 0.4303101623085879 0.8400785356894039\n",
      "115/115 [==============================] - 6s 53ms/step - loss: 0.0784 - custom_metric: 1.2704\n",
      "Epoch 16/1000\n",
      "115/115 [==============================] - ETA: 0s - loss: 0.0489val_aucs: 0.40734929060282393 0.8280321742985624\n",
      "115/115 [==============================] - 6s 51ms/step - loss: 0.0489 - custom_metric: 1.2354\n",
      "Epoch 17/1000\n",
      "115/115 [==============================] - ETA: 0s - loss: 0.0525val_aucs: 0.4119977585044316 0.8239280511748686\n",
      "115/115 [==============================] - 6s 50ms/step - loss: 0.0525 - custom_metric: 1.2359\n",
      "Epoch 18/1000\n",
      "115/115 [==============================] - ETA: 0s - loss: 0.0632val_aucs: 0.39991296032982127 0.8156944708341249\n",
      "115/115 [==============================] - 6s 51ms/step - loss: 0.0632 - custom_metric: 1.2156\n",
      "Epoch 19/1000\n",
      "115/115 [==============================] - ETA: 0s - loss: 0.1106val_aucs: 0.3803284650681745 0.8196845905377161\n",
      "115/115 [==============================] - 6s 50ms/step - loss: 0.1106 - custom_metric: 1.2000\n",
      "Epoch 20/1000\n",
      "115/115 [==============================] - ETA: 0s - loss: 0.1478val_aucs: 0.4394753786331731 0.8261954525302425\n",
      "115/115 [==============================] - 6s 50ms/step - loss: 0.1478 - custom_metric: 1.2657\n",
      "Epoch 21/1000\n",
      "115/115 [==============================] - ETA: 0s - loss: 0.0692val_aucs: 0.3986439087724385 0.8278168345050352\n",
      "115/115 [==============================] - 6s 51ms/step - loss: 0.0692 - custom_metric: 1.2265\n",
      "Epoch 22/1000\n",
      "115/115 [==============================] - ETA: 0s - loss: 0.0273val_aucs: 0.4332781588790411 0.8280955095319525\n",
      "115/115 [==============================] - 6s 51ms/step - loss: 0.0273 - custom_metric: 1.2614\n",
      "Epoch 23/1000\n",
      "115/115 [==============================] - ETA: 0s - loss: 0.0125val_aucs: 0.4350719696280644 0.8266514662106531\n",
      "115/115 [==============================] - 6s 50ms/step - loss: 0.0125 - custom_metric: 1.2617\n",
      "Epoch 24/1000\n",
      "115/115 [==============================] - ETA: 0s - loss: 0.0066val_aucs: 0.4284851539137462 0.8235100386344923\n",
      "115/115 [==============================] - 6s 52ms/step - loss: 0.0066 - custom_metric: 1.2520\n",
      "Epoch 25/1000\n",
      "115/115 [==============================] - ETA: 0s - loss: 0.0052val_aucs: 0.42612320912804985 0.8236620431946292\n",
      "115/115 [==============================] - 6s 51ms/step - loss: 0.0052 - custom_metric: 1.2498\n",
      "Test res 0.8128803404739109 0.31580933709437703 0.36744639376218324\n",
      "Repeat 7 ld 10\n",
      "Num train: 3655 Num valid: 926\n",
      "Exp_GloVe/M/models/classification/new_mimic_iii_24hm_strats_no_interp_with_ss_repeat7_10ld.h5\n",
      "Epoch 1/1000\n",
      "115/115 [==============================] - ETA: 0s - loss: 0.5401val_aucs: 0.5069604290708556 0.8626735734432631\n",
      "115/115 [==============================] - 14s 71ms/step - loss: 0.5401 - custom_metric: 1.3696\n",
      "Epoch 2/1000\n",
      "115/115 [==============================] - ETA: 0s - loss: 0.4458val_aucs: 0.529353786805605 0.8778612497287915\n",
      "115/115 [==============================] - 7s 57ms/step - loss: 0.4458 - custom_metric: 1.4072\n",
      "Epoch 3/1000\n",
      "115/115 [==============================] - ETA: 0s - loss: 0.4010val_aucs: 0.568091486264758 0.881156433065741\n",
      "115/115 [==============================] - 6s 53ms/step - loss: 0.4010 - custom_metric: 1.4492\n",
      "Epoch 4/1000\n",
      "115/115 [==============================] - ETA: 0s - loss: 0.3698val_aucs: 0.525403736464117 0.8744711434150575\n",
      "115/115 [==============================] - 6s 51ms/step - loss: 0.3698 - custom_metric: 1.3999\n",
      "Epoch 5/1000\n",
      "115/115 [==============================] - ETA: 0s - loss: 0.3230val_aucs: 0.5440240857652403 0.8764238446517683\n",
      "115/115 [==============================] - 6s 51ms/step - loss: 0.3230 - custom_metric: 1.4204\n",
      "Epoch 6/1000\n",
      "115/115 [==============================] - ETA: 0s - loss: 0.2861val_aucs: 0.529019817774054 0.8735625949229768\n",
      "115/115 [==============================] - 6s 53ms/step - loss: 0.2861 - custom_metric: 1.4026\n",
      "Epoch 7/1000\n",
      "115/115 [==============================] - ETA: 0s - loss: 0.2401val_aucs: 0.5234769982767965 0.8621040355825559\n",
      "115/115 [==============================] - 6s 50ms/step - loss: 0.2401 - custom_metric: 1.3856\n",
      "Epoch 8/1000\n",
      "115/115 [==============================] - ETA: 0s - loss: 0.2091val_aucs: 0.5224501051375097 0.8666874593187243\n",
      "115/115 [==============================] - 6s 50ms/step - loss: 0.2091 - custom_metric: 1.3891\n",
      "Epoch 9/1000\n",
      "115/115 [==============================] - ETA: 0s - loss: 0.1688val_aucs: 0.4783789522945774 0.8601242134953351\n",
      "115/115 [==============================] - 6s 51ms/step - loss: 0.1688 - custom_metric: 1.3385\n",
      "Epoch 10/1000\n",
      "114/115 [============================>.] - ETA: 0s - loss: 0.1359val_aucs: 0.4798822827572939 0.8650602082881319\n",
      "115/115 [==============================] - 6s 52ms/step - loss: 0.1358 - custom_metric: 1.3449\n",
      "Epoch 11/1000\n",
      "115/115 [==============================] - ETA: 0s - loss: 0.1377val_aucs: 0.49910259953316477 0.8624566066391842\n",
      "115/115 [==============================] - 6s 50ms/step - loss: 0.1377 - custom_metric: 1.3616\n",
      "Epoch 12/1000\n",
      "115/115 [==============================] - ETA: 0s - loss: 0.1235val_aucs: 0.43804505818551465 0.8527066608808852\n",
      "115/115 [==============================] - 6s 51ms/step - loss: 0.1235 - custom_metric: 1.2908\n",
      "Epoch 13/1000\n",
      "115/115 [==============================] - ETA: 0s - loss: 0.1323val_aucs: 0.5200220183066129 0.8742406161857237\n",
      "115/115 [==============================] - 6s 52ms/step - loss: 0.1323 - custom_metric: 1.3943\n",
      "Test res 0.8474839374401627 0.4131260106420431 0.4327485380116959\n",
      "Repeat 8 ld 10\n",
      "Num train: 3655 Num valid: 926\n",
      "Exp_GloVe/M/models/classification/new_mimic_iii_24hm_strats_no_interp_with_ss_repeat8_10ld.h5\n",
      "Epoch 1/1000\n",
      "115/115 [==============================] - ETA: 0s - loss: 0.5057val_aucs: 0.419860768259105 0.8553683609366077\n",
      "115/115 [==============================] - 16s 74ms/step - loss: 0.5057 - custom_metric: 1.2752\n",
      "Epoch 2/1000\n",
      "115/115 [==============================] - ETA: 0s - loss: 0.4072val_aucs: 0.445449966266621 0.8705739577384352\n",
      "115/115 [==============================] - 6s 53ms/step - loss: 0.4072 - custom_metric: 1.3160\n",
      "Epoch 3/1000\n",
      "115/115 [==============================] - ETA: 0s - loss: 0.3802val_aucs: 0.45048123113390587 0.8700861412526175\n",
      "115/115 [==============================] - 6s 53ms/step - loss: 0.3802 - custom_metric: 1.3206\n",
      "Epoch 4/1000\n",
      "115/115 [==============================] - ETA: 0s - loss: 0.3555val_aucs: 0.4895244871895055 0.8792713687416713\n",
      "115/115 [==============================] - 6s 53ms/step - loss: 0.3555 - custom_metric: 1.3688\n",
      "Epoch 5/1000\n",
      "115/115 [==============================] - ETA: 0s - loss: 0.2994val_aucs: 0.49742086259645135 0.8654578336188845\n",
      "115/115 [==============================] - 6s 51ms/step - loss: 0.2994 - custom_metric: 1.3629\n",
      "Epoch 6/1000\n",
      "115/115 [==============================] - ETA: 0s - loss: 0.2961val_aucs: 0.5066771625835857 0.8658980582524272\n",
      "115/115 [==============================] - 6s 53ms/step - loss: 0.2961 - custom_metric: 1.3726\n",
      "Epoch 7/1000\n",
      "115/115 [==============================] - ETA: 0s - loss: 0.2532val_aucs: 0.4778772580101701 0.8734532648010661\n",
      "115/115 [==============================] - 6s 52ms/step - loss: 0.2532 - custom_metric: 1.3513\n",
      "Epoch 8/1000\n",
      "114/115 [============================>.] - ETA: 0s - loss: 0.2217val_aucs: 0.5058539640070198 0.8600323624595468\n",
      "115/115 [==============================] - 6s 51ms/step - loss: 0.2225 - custom_metric: 1.3659\n",
      "Epoch 9/1000\n",
      "115/115 [==============================] - ETA: 0s - loss: 0.1853val_aucs: 0.5158843176852255 0.8698243860651056\n",
      "115/115 [==============================] - 6s 53ms/step - loss: 0.1853 - custom_metric: 1.3857\n",
      "Epoch 10/1000\n",
      "115/115 [==============================] - ETA: 0s - loss: 0.1563val_aucs: 0.4726534235495295 0.8495740529221398\n",
      "115/115 [==============================] - 6s 51ms/step - loss: 0.1563 - custom_metric: 1.3222\n",
      "Epoch 11/1000\n",
      "115/115 [==============================] - ETA: 0s - loss: 0.1578val_aucs: 0.45575624909130014 0.8551185037121646\n",
      "115/115 [==============================] - 6s 51ms/step - loss: 0.1578 - custom_metric: 1.3109\n",
      "Epoch 12/1000\n",
      "115/115 [==============================] - ETA: 0s - loss: 0.1830val_aucs: 0.4457870647021114 0.850847134970493\n",
      "115/115 [==============================] - 6s 52ms/step - loss: 0.1830 - custom_metric: 1.2966\n",
      "Epoch 13/1000\n",
      "115/115 [==============================] - ETA: 0s - loss: 0.1074val_aucs: 0.44109738885930216 0.8587235865219875\n",
      "115/115 [==============================] - 6s 50ms/step - loss: 0.1074 - custom_metric: 1.2998\n",
      "Epoch 14/1000\n",
      "115/115 [==============================] - ETA: 0s - loss: 0.1124val_aucs: 0.4844806534730657 0.8621383019227108\n",
      "115/115 [==============================] - 6s 52ms/step - loss: 0.1124 - custom_metric: 1.3466\n",
      "Epoch 15/1000\n",
      "115/115 [==============================] - ETA: 0s - loss: 0.1053val_aucs: 0.4732586219557933 0.8639824861983627\n",
      "115/115 [==============================] - 6s 51ms/step - loss: 0.1053 - custom_metric: 1.3372\n",
      "Epoch 16/1000\n",
      "115/115 [==============================] - ETA: 0s - loss: 0.0871val_aucs: 0.4345600445224469 0.8538335237007425\n",
      "115/115 [==============================] - 6s 51ms/step - loss: 0.0871 - custom_metric: 1.2884\n",
      "Epoch 17/1000\n",
      "115/115 [==============================] - ETA: 0s - loss: 0.0765val_aucs: 0.4756961505381034 0.8596754235674854\n",
      "115/115 [==============================] - 6s 50ms/step - loss: 0.0765 - custom_metric: 1.3354\n",
      "Epoch 18/1000\n",
      "115/115 [==============================] - ETA: 0s - loss: 0.0386val_aucs: 0.48534974874789144 0.8585808109651627\n",
      "115/115 [==============================] - 6s 50ms/step - loss: 0.0386 - custom_metric: 1.3439\n",
      "Epoch 19/1000\n",
      "114/115 [============================>.] - ETA: 0s - loss: 0.0176val_aucs: 0.4590960987562535 0.8522986864648773\n",
      "115/115 [==============================] - 6s 52ms/step - loss: 0.0176 - custom_metric: 1.3114\n",
      "Test res 0.8295941445518189 0.38040568451300444 0.40794573643410853\n",
      "Repeat 9 ld 10\n",
      "Num train: 3655 Num valid: 926\n",
      "Exp_GloVe/M/models/classification/new_mimic_iii_24hm_strats_no_interp_with_ss_repeat9_10ld.h5\n",
      "Epoch 1/1000\n",
      "115/115 [==============================] - ETA: 0s - loss: 0.5543val_aucs: 0.382259177088471 0.8512343441665661\n",
      "115/115 [==============================] - 15s 73ms/step - loss: 0.5543 - custom_metric: 1.2335\n",
      "Epoch 2/1000\n",
      "115/115 [==============================] - ETA: 0s - loss: 0.4655val_aucs: 0.40556672075817296 0.8632086236290658\n",
      "115/115 [==============================] - 6s 55ms/step - loss: 0.4655 - custom_metric: 1.2688\n",
      "Epoch 3/1000\n",
      "115/115 [==============================] - ETA: 0s - loss: 0.4457val_aucs: 0.40687017391993774 0.8643093981984884\n",
      "115/115 [==============================] - 6s 52ms/step - loss: 0.4457 - custom_metric: 1.2712\n",
      "Epoch 4/1000\n",
      "115/115 [==============================] - ETA: 0s - loss: 0.4016val_aucs: 0.38797361272444786 0.8754245365336341\n",
      "115/115 [==============================] - 6s 52ms/step - loss: 0.4016 - custom_metric: 1.2634\n",
      "Epoch 5/1000\n",
      "115/115 [==============================] - ETA: 0s - loss: 0.3699val_aucs: 0.39352816948356306 0.8551944478004645\n",
      "115/115 [==============================] - 6s 51ms/step - loss: 0.3699 - custom_metric: 1.2487\n",
      "Epoch 6/1000\n",
      "115/115 [==============================] - ETA: 0s - loss: 0.3260val_aucs: 0.38029204472633826 0.874229793403407\n",
      "115/115 [==============================] - 6s 51ms/step - loss: 0.3260 - custom_metric: 1.2545\n",
      "Epoch 7/1000\n",
      "115/115 [==============================] - ETA: 0s - loss: 0.2888val_aucs: 0.40131909288678275 0.880512262897185\n",
      "115/115 [==============================] - 6s 53ms/step - loss: 0.2888 - custom_metric: 1.2818\n",
      "Epoch 8/1000\n",
      "115/115 [==============================] - ETA: 0s - loss: 0.2601val_aucs: 0.3691133207710512 0.8670210623817003\n",
      "115/115 [==============================] - 6s 51ms/step - loss: 0.2601 - custom_metric: 1.2361\n",
      "Epoch 9/1000\n",
      "115/115 [==============================] - ETA: 0s - loss: 0.2310val_aucs: 0.3615810742693234 0.84167639912475\n",
      "115/115 [==============================] - 6s 51ms/step - loss: 0.2310 - custom_metric: 1.2033\n",
      "Epoch 10/1000\n",
      "115/115 [==============================] - ETA: 0s - loss: 0.1907val_aucs: 0.374835238759695 0.863490529311479\n",
      "115/115 [==============================] - 6s 50ms/step - loss: 0.1907 - custom_metric: 1.2383\n",
      "Epoch 11/1000\n",
      "115/115 [==============================] - ETA: 0s - loss: 0.1432val_aucs: 0.3623852495108822 0.8558656518062099\n",
      "115/115 [==============================] - 6s 50ms/step - loss: 0.1432 - custom_metric: 1.2183\n",
      "Epoch 12/1000\n",
      "115/115 [==============================] - ETA: 0s - loss: 0.1589val_aucs: 0.3175223708032858 0.8455693823580739\n",
      "115/115 [==============================] - 6s 52ms/step - loss: 0.1589 - custom_metric: 1.1631\n",
      "Epoch 13/1000\n",
      "115/115 [==============================] - ETA: 0s - loss: 0.1582val_aucs: 0.395243875357691 0.8673298162243432\n",
      "115/115 [==============================] - 6s 50ms/step - loss: 0.1582 - custom_metric: 1.2626\n",
      "Epoch 14/1000\n",
      "115/115 [==============================] - ETA: 0s - loss: 0.0919val_aucs: 0.36718678555399803 0.8686856483159491\n",
      "115/115 [==============================] - 6s 51ms/step - loss: 0.0919 - custom_metric: 1.2359\n",
      "Epoch 15/1000\n",
      "115/115 [==============================] - ETA: 0s - loss: 0.0785val_aucs: 0.3581507186942682 0.8526975688990913\n",
      "115/115 [==============================] - 6s 51ms/step - loss: 0.0785 - custom_metric: 1.2108\n",
      "Epoch 16/1000\n",
      "115/115 [==============================] - ETA: 0s - loss: 0.0634val_aucs: 0.33368798643843284 0.8459721047615213\n",
      "115/115 [==============================] - 6s 52ms/step - loss: 0.0634 - custom_metric: 1.1797\n",
      "Epoch 17/1000\n",
      "115/115 [==============================] - ETA: 0s - loss: 0.0638val_aucs: 0.3317570270908964 0.8577450230222974\n",
      "115/115 [==============================] - 6s 51ms/step - loss: 0.0638 - custom_metric: 1.1895\n",
      "Test res 0.8494959647259455 0.40841016565918753 0.43816942551119764\n",
      "gen_res {10: [(0.8495673253126496, 0.015550036538510871), (0.39991067148684534, 0.03244069307475938), (0.43139881908261096, 0.025470753193600273)]}\n",
      "Repeat 0 ld 20\n",
      "Num train: 7310 Num valid: 1852\n",
      "Exp_GloVe/M/models/classification/new_mimic_iii_24hm_strats_no_interp_with_ss_repeat0_20ld.h5\n",
      "Epoch 1/1000\n",
      "229/229 [==============================] - ETA: 0s - loss: 0.5018val_aucs: 0.44579983432761894 0.870656734479304\n",
      "229/229 [==============================] - 21s 63ms/step - loss: 0.5018 - custom_metric: 1.3165\n",
      "Epoch 2/1000\n",
      "229/229 [==============================] - ETA: 0s - loss: 0.4310val_aucs: 0.47355332935374095 0.8714204945594494\n",
      "229/229 [==============================] - 12s 51ms/step - loss: 0.4310 - custom_metric: 1.3450\n",
      "Epoch 3/1000\n",
      "229/229 [==============================] - ETA: 0s - loss: 0.4048val_aucs: 0.48510982283195203 0.8767635087997725\n",
      "229/229 [==============================] - 12s 51ms/step - loss: 0.4048 - custom_metric: 1.3619\n",
      "Epoch 4/1000\n",
      "229/229 [==============================] - ETA: 0s - loss: 0.3637val_aucs: 0.4820064037911313 0.8754508994845446\n",
      "229/229 [==============================] - 11s 50ms/step - loss: 0.3637 - custom_metric: 1.3575\n",
      "Epoch 5/1000\n",
      "229/229 [==============================] - ETA: 0s - loss: 0.3402val_aucs: 0.46925507070232697 0.8674165401999\n",
      "229/229 [==============================] - 12s 51ms/step - loss: 0.3402 - custom_metric: 1.3367\n",
      "Epoch 6/1000\n",
      "229/229 [==============================] - ETA: 0s - loss: 0.3175val_aucs: 0.4712194174341977 0.861928047849073\n",
      "229/229 [==============================] - 12s 50ms/step - loss: 0.3175 - custom_metric: 1.3331\n",
      "Epoch 7/1000\n",
      "229/229 [==============================] - ETA: 0s - loss: 0.2735val_aucs: 0.44429232470886376 0.8631249359400365\n",
      "229/229 [==============================] - 11s 50ms/step - loss: 0.2735 - custom_metric: 1.3074\n",
      "Epoch 8/1000\n",
      "228/229 [============================>.] - ETA: 0s - loss: 0.2421val_aucs: 0.4337367950062753 0.8410122631434513\n",
      "229/229 [==============================] - 11s 50ms/step - loss: 0.2416 - custom_metric: 1.2747\n",
      "Epoch 9/1000\n",
      "229/229 [==============================] - ETA: 0s - loss: 0.2347val_aucs: 0.4394382518372962 0.8432043537630889\n",
      "229/229 [==============================] - 12s 50ms/step - loss: 0.2347 - custom_metric: 1.2826\n",
      "Epoch 10/1000\n",
      "229/229 [==============================] - ETA: 0s - loss: 0.2060val_aucs: 0.42479943183619306 0.8491689562937469\n",
      "229/229 [==============================] - 12s 51ms/step - loss: 0.2060 - custom_metric: 1.2740\n",
      "Epoch 11/1000\n",
      "229/229 [==============================] - ETA: 0s - loss: 0.1748val_aucs: 0.38616951974062397 0.8365288922833781\n",
      "229/229 [==============================] - 11s 50ms/step - loss: 0.1748 - custom_metric: 1.2227\n",
      "Epoch 12/1000\n",
      "229/229 [==============================] - ETA: 0s - loss: 0.1673val_aucs: 0.4152614995889167 0.8613527480484443\n",
      "229/229 [==============================] - 11s 50ms/step - loss: 0.1673 - custom_metric: 1.2766\n",
      "Epoch 13/1000\n",
      "229/229 [==============================] - ETA: 0s - loss: 0.1236val_aucs: 0.3830804280965916 0.833751582901032\n",
      "229/229 [==============================] - 12s 50ms/step - loss: 0.1236 - custom_metric: 1.2168\n",
      "Test res 0.8703935607647201 0.4557517063206976 0.4580078125\n",
      "Repeat 1 ld 20\n",
      "Num train: 7310 Num valid: 1852\n",
      "Exp_GloVe/M/models/classification/new_mimic_iii_24hm_strats_no_interp_with_ss_repeat1_20ld.h5\n",
      "Epoch 1/1000\n",
      "229/229 [==============================] - ETA: 0s - loss: 0.5408val_aucs: 0.46164950676055044 0.8745965089805213\n",
      "229/229 [==============================] - 22s 61ms/step - loss: 0.5408 - custom_metric: 1.3362\n",
      "Epoch 2/1000\n",
      "229/229 [==============================] - ETA: 0s - loss: 0.4591val_aucs: 0.45581667518396507 0.8818416392613205\n",
      "229/229 [==============================] - 12s 52ms/step - loss: 0.4591 - custom_metric: 1.3377\n",
      "Epoch 3/1000\n",
      "229/229 [==============================] - ETA: 0s - loss: 0.4279val_aucs: 0.515805154282826 0.887093346825196\n",
      "229/229 [==============================] - 12s 51ms/step - loss: 0.4279 - custom_metric: 1.4029\n",
      "Epoch 4/1000\n",
      "229/229 [==============================] - ETA: 0s - loss: 0.3976val_aucs: 0.49056778480023966 0.8808263765916181\n",
      "229/229 [==============================] - 12s 51ms/step - loss: 0.3976 - custom_metric: 1.3714\n",
      "Epoch 5/1000\n",
      "229/229 [==============================] - ETA: 0s - loss: 0.3865val_aucs: 0.5178616531770486 0.8902369508390252\n",
      "229/229 [==============================] - 12s 51ms/step - loss: 0.3865 - custom_metric: 1.4081\n",
      "Epoch 6/1000\n",
      "229/229 [==============================] - ETA: 0s - loss: 0.3421val_aucs: 0.4819597455593074 0.885639598617084\n",
      "229/229 [==============================] - 11s 50ms/step - loss: 0.3421 - custom_metric: 1.3676\n",
      "Epoch 7/1000\n",
      "229/229 [==============================] - ETA: 0s - loss: 0.3226val_aucs: 0.5071306562454647 0.8787587486297326\n",
      "229/229 [==============================] - 11s 50ms/step - loss: 0.3226 - custom_metric: 1.3859\n",
      "Epoch 8/1000\n",
      "229/229 [==============================] - ETA: 0s - loss: 0.2916val_aucs: 0.44684148077431396 0.8651420861792731\n",
      "229/229 [==============================] - 12s 50ms/step - loss: 0.2916 - custom_metric: 1.3120\n",
      "Epoch 9/1000\n",
      "229/229 [==============================] - ETA: 0s - loss: 0.2581val_aucs: 0.4244615659513333 0.8517311746352981\n",
      "229/229 [==============================] - 11s 50ms/step - loss: 0.2581 - custom_metric: 1.2762\n",
      "Epoch 10/1000\n",
      "229/229 [==============================] - ETA: 0s - loss: 0.2667val_aucs: 0.4580152249187457 0.8637895269415633\n",
      "229/229 [==============================] - 12s 50ms/step - loss: 0.2667 - custom_metric: 1.3218\n",
      "Epoch 11/1000\n",
      "229/229 [==============================] - ETA: 0s - loss: 0.2493val_aucs: 0.42077399992385145 0.8622480816257695\n",
      "229/229 [==============================] - 11s 50ms/step - loss: 0.2493 - custom_metric: 1.2830\n",
      "Epoch 12/1000\n",
      "229/229 [==============================] - ETA: 0s - loss: 0.1973val_aucs: 0.41959394002171496 0.857954296315035\n",
      "229/229 [==============================] - 11s 50ms/step - loss: 0.1973 - custom_metric: 1.2775\n",
      "Epoch 13/1000\n",
      "229/229 [==============================] - ETA: 0s - loss: 0.1865val_aucs: 0.4195881474355588 0.85409225061135\n",
      "229/229 [==============================] - 11s 50ms/step - loss: 0.1865 - custom_metric: 1.2737\n",
      "Epoch 14/1000\n",
      "229/229 [==============================] - ETA: 0s - loss: 0.1510val_aucs: 0.4486591579621188 0.8644337633864575\n",
      "229/229 [==============================] - 11s 50ms/step - loss: 0.1510 - custom_metric: 1.3131\n",
      "Epoch 15/1000\n",
      "229/229 [==============================] - ETA: 0s - loss: 0.1599val_aucs: 0.42234581079360584 0.8518424825027406\n",
      "229/229 [==============================] - 12s 51ms/step - loss: 0.1599 - custom_metric: 1.2742\n",
      "Test res 0.8688662151448061 0.4546787977984077 0.46830652790917693\n",
      "Repeat 2 ld 20\n",
      "Num train: 7310 Num valid: 1852\n",
      "Exp_GloVe/M/models/classification/new_mimic_iii_24hm_strats_no_interp_with_ss_repeat2_20ld.h5\n",
      "Epoch 1/1000\n",
      "229/229 [==============================] - ETA: 0s - loss: 0.5148val_aucs: 0.5249856401939006 0.8747271998018143\n",
      "229/229 [==============================] - 20s 61ms/step - loss: 0.5148 - custom_metric: 1.3997\n",
      "Epoch 2/1000\n",
      "229/229 [==============================] - ETA: 0s - loss: 0.4449val_aucs: 0.5632023206360045 0.8898418053769658\n",
      "229/229 [==============================] - 12s 51ms/step - loss: 0.4449 - custom_metric: 1.4530\n",
      "Epoch 3/1000\n",
      "229/229 [==============================] - ETA: 0s - loss: 0.4222val_aucs: 0.5665055648716475 0.8931743915818282\n",
      "229/229 [==============================] - 12s 52ms/step - loss: 0.4222 - custom_metric: 1.4597\n",
      "Epoch 4/1000\n",
      "228/229 [============================>.] - ETA: 0s - loss: 0.3745val_aucs: 0.5782189272456049 0.8899214335429226\n",
      "229/229 [==============================] - 12s 51ms/step - loss: 0.3760 - custom_metric: 1.4681\n",
      "Epoch 5/1000\n",
      "229/229 [==============================] - ETA: 0s - loss: 0.3494val_aucs: 0.5613263732968288 0.890363812242683\n",
      "229/229 [==============================] - 12s 51ms/step - loss: 0.3494 - custom_metric: 1.4517\n",
      "Epoch 6/1000\n",
      "229/229 [==============================] - ETA: 0s - loss: 0.3253val_aucs: 0.5358711435104889 0.8840790855147519\n",
      "229/229 [==============================] - 12s 50ms/step - loss: 0.3253 - custom_metric: 1.4200\n",
      "Epoch 7/1000\n",
      "229/229 [==============================] - ETA: 0s - loss: 0.3026val_aucs: 0.5720684891479206 0.8878864915240242\n",
      "229/229 [==============================] - 11s 50ms/step - loss: 0.3026 - custom_metric: 1.4600\n",
      "Epoch 8/1000\n",
      "229/229 [==============================] - ETA: 0s - loss: 0.2614val_aucs: 0.515893829350579 0.8678703299555263\n",
      "229/229 [==============================] - 12s 51ms/step - loss: 0.2614 - custom_metric: 1.3838\n",
      "Epoch 9/1000\n",
      "229/229 [==============================] - ETA: 0s - loss: 0.2495val_aucs: 0.524222092048688 0.8840967806627423\n",
      "229/229 [==============================] - 11s 50ms/step - loss: 0.2495 - custom_metric: 1.4083\n",
      "Epoch 10/1000\n",
      "229/229 [==============================] - ETA: 0s - loss: 0.2113val_aucs: 0.49226407557477075 0.8705216529627576\n",
      "229/229 [==============================] - 11s 50ms/step - loss: 0.2113 - custom_metric: 1.3628\n",
      "Epoch 11/1000\n",
      "229/229 [==============================] - ETA: 0s - loss: 0.1824val_aucs: 0.5177699541724144 0.8745325531739196\n",
      "229/229 [==============================] - 11s 50ms/step - loss: 0.1824 - custom_metric: 1.3923\n",
      "Epoch 12/1000\n",
      "229/229 [==============================] - ETA: 0s - loss: 0.1823val_aucs: 0.5125806009093078 0.8734030128938646\n",
      "229/229 [==============================] - 11s 50ms/step - loss: 0.1823 - custom_metric: 1.3860\n",
      "Epoch 13/1000\n",
      "229/229 [==============================] - ETA: 0s - loss: 0.1823val_aucs: 0.5378834195020452 0.8724931873680238\n",
      "229/229 [==============================] - 12s 50ms/step - loss: 0.1823 - custom_metric: 1.4104\n",
      "Epoch 14/1000\n",
      "229/229 [==============================] - ETA: 0s - loss: 0.1361val_aucs: 0.4898036005207397 0.8638771248923546\n",
      "229/229 [==============================] - 12s 51ms/step - loss: 0.1361 - custom_metric: 1.3537\n",
      "Test res 0.8739001166826232 0.46441320516350126 0.486328125\n",
      "Repeat 3 ld 20\n",
      "Num train: 7310 Num valid: 1852\n",
      "Exp_GloVe/M/models/classification/new_mimic_iii_24hm_strats_no_interp_with_ss_repeat3_20ld.h5\n",
      "Epoch 1/1000\n",
      "229/229 [==============================] - ETA: 0s - loss: 0.5079val_aucs: 0.4781789209639542 0.8765961336607293\n",
      "229/229 [==============================] - 22s 62ms/step - loss: 0.5079 - custom_metric: 1.3548\n",
      "Epoch 2/1000\n",
      "229/229 [==============================] - ETA: 0s - loss: 0.4182val_aucs: 0.5003468824759717 0.8731824328370911\n",
      "229/229 [==============================] - 12s 52ms/step - loss: 0.4182 - custom_metric: 1.3735\n",
      "Epoch 3/1000\n",
      "228/229 [============================>.] - ETA: 0s - loss: 0.3982val_aucs: 0.5362708893322992 0.8849964223911037\n",
      "229/229 [==============================] - 12s 52ms/step - loss: 0.3978 - custom_metric: 1.4213\n",
      "Epoch 4/1000\n",
      "229/229 [==============================] - ETA: 0s - loss: 0.3555val_aucs: 0.5210621931289876 0.8861500830572638\n",
      "229/229 [==============================] - 11s 50ms/step - loss: 0.3555 - custom_metric: 1.4072\n",
      "Epoch 5/1000\n",
      "228/229 [============================>.] - ETA: 0s - loss: 0.3228val_aucs: 0.5195531994175503 0.8790452985175776\n",
      "229/229 [==============================] - 12s 50ms/step - loss: 0.3228 - custom_metric: 1.3986\n",
      "Epoch 6/1000\n",
      "229/229 [==============================] - ETA: 0s - loss: 0.3132val_aucs: 0.4977107360246451 0.8795181102660038\n",
      "229/229 [==============================] - 12s 50ms/step - loss: 0.3132 - custom_metric: 1.3772\n",
      "Epoch 7/1000\n",
      "228/229 [============================>.] - ETA: 0s - loss: 0.2840val_aucs: 0.4836230468775752 0.8665945891423511\n",
      "229/229 [==============================] - 12s 50ms/step - loss: 0.2844 - custom_metric: 1.3502\n",
      "Epoch 8/1000\n",
      "229/229 [==============================] - ETA: 0s - loss: 0.2522val_aucs: 0.44677929991824 0.8535828098256586\n",
      "229/229 [==============================] - 11s 50ms/step - loss: 0.2522 - custom_metric: 1.3004\n",
      "Epoch 9/1000\n",
      "229/229 [==============================] - ETA: 0s - loss: 0.2406val_aucs: 0.4808244571011087 0.8585378769491665\n",
      "229/229 [==============================] - 11s 50ms/step - loss: 0.2406 - custom_metric: 1.3394\n",
      "Epoch 10/1000\n",
      "229/229 [==============================] - ETA: 0s - loss: 0.2072val_aucs: 0.46002328582889285 0.8569051003779341\n",
      "229/229 [==============================] - 11s 50ms/step - loss: 0.2072 - custom_metric: 1.3169\n",
      "Epoch 11/1000\n",
      "229/229 [==============================] - ETA: 0s - loss: 0.1784val_aucs: 0.4556098144898025 0.8602967366533123\n",
      "229/229 [==============================] - 11s 50ms/step - loss: 0.1784 - custom_metric: 1.3159\n",
      "Epoch 12/1000\n",
      "229/229 [==============================] - ETA: 0s - loss: 0.1610val_aucs: 0.445287465480785 0.8589917762276558\n",
      "229/229 [==============================] - 11s 50ms/step - loss: 0.1610 - custom_metric: 1.3043\n",
      "Epoch 13/1000\n",
      "229/229 [==============================] - ETA: 0s - loss: 0.1654val_aucs: 0.4161615315326932 0.8389004289978597\n",
      "229/229 [==============================] - 12s 51ms/step - loss: 0.1654 - custom_metric: 1.2551\n",
      "Test res 0.8739205922391096 0.44222740851733805 0.4580078125\n",
      "Repeat 4 ld 20\n",
      "Num train: 7310 Num valid: 1852\n",
      "Exp_GloVe/M/models/classification/new_mimic_iii_24hm_strats_no_interp_with_ss_repeat4_20ld.h5\n",
      "Epoch 1/1000\n",
      "229/229 [==============================] - ETA: 0s - loss: 0.5076val_aucs: 0.41299575020486573 0.8846991290577987\n",
      "229/229 [==============================] - 20s 61ms/step - loss: 0.5076 - custom_metric: 1.2977\n",
      "Epoch 2/1000\n",
      "229/229 [==============================] - ETA: 0s - loss: 0.4217val_aucs: 0.43517938617584534 0.8933626003845719\n",
      "229/229 [==============================] - 12s 52ms/step - loss: 0.4217 - custom_metric: 1.3285\n",
      "Epoch 3/1000\n",
      "229/229 [==============================] - ETA: 0s - loss: 0.3951val_aucs: 0.4757227329244196 0.9020472797194887\n",
      "229/229 [==============================] - 12s 52ms/step - loss: 0.3951 - custom_metric: 1.3778\n",
      "Epoch 4/1000\n",
      "228/229 [============================>.] - ETA: 0s - loss: 0.3618val_aucs: 0.45728791424079246 0.8978905101232892\n",
      "229/229 [==============================] - 12s 51ms/step - loss: 0.3617 - custom_metric: 1.3552\n",
      "Epoch 5/1000\n",
      "229/229 [==============================] - ETA: 0s - loss: 0.3406val_aucs: 0.4710643625562498 0.8995270614183916\n",
      "229/229 [==============================] - 12s 51ms/step - loss: 0.3406 - custom_metric: 1.3706\n",
      "Epoch 6/1000\n",
      "229/229 [==============================] - ETA: 0s - loss: 0.3105val_aucs: 0.4537813593871545 0.888753393281303\n",
      "229/229 [==============================] - 11s 50ms/step - loss: 0.3105 - custom_metric: 1.3425\n",
      "Epoch 7/1000\n",
      "229/229 [==============================] - ETA: 0s - loss: 0.2896val_aucs: 0.46577517172963273 0.8865477604343401\n",
      "229/229 [==============================] - 11s 50ms/step - loss: 0.2896 - custom_metric: 1.3523\n",
      "Epoch 8/1000\n",
      "229/229 [==============================] - ETA: 0s - loss: 0.2637val_aucs: 0.4473274657687151 0.8800581099423143\n",
      "229/229 [==============================] - 11s 50ms/step - loss: 0.2637 - custom_metric: 1.3274\n",
      "Epoch 9/1000\n",
      "229/229 [==============================] - ETA: 0s - loss: 0.2492val_aucs: 0.41993018946977523 0.8788952041624251\n",
      "229/229 [==============================] - 12s 50ms/step - loss: 0.2492 - custom_metric: 1.2988\n",
      "Epoch 10/1000\n",
      "229/229 [==============================] - ETA: 0s - loss: 0.2375val_aucs: 0.407168100187576 0.8669444915733514\n",
      "229/229 [==============================] - 11s 50ms/step - loss: 0.2375 - custom_metric: 1.2741\n",
      "Epoch 11/1000\n",
      "229/229 [==============================] - ETA: 0s - loss: 0.2094val_aucs: 0.38581351557465876 0.8663648060174188\n",
      "229/229 [==============================] - 11s 50ms/step - loss: 0.2094 - custom_metric: 1.2522\n",
      "Epoch 12/1000\n",
      "229/229 [==============================] - ETA: 0s - loss: 0.2027val_aucs: 0.44311925947515474 0.8851621705689402\n",
      "229/229 [==============================] - 11s 50ms/step - loss: 0.2027 - custom_metric: 1.3283\n",
      "Epoch 13/1000\n",
      "229/229 [==============================] - ETA: 0s - loss: 0.1684val_aucs: 0.40833391380352163 0.8708715077479923\n",
      "229/229 [==============================] - 12s 51ms/step - loss: 0.1684 - custom_metric: 1.2792\n",
      "Test res 0.8748482564923408 0.4474720626975332 0.46484375\n",
      "Repeat 5 ld 20\n",
      "Num train: 7310 Num valid: 1852\n",
      "Exp_GloVe/M/models/classification/new_mimic_iii_24hm_strats_no_interp_with_ss_repeat5_20ld.h5\n",
      "Epoch 1/1000\n",
      "229/229 [==============================] - ETA: 0s - loss: 0.4925val_aucs: 0.43508138607444546 0.8871294666858118\n",
      "229/229 [==============================] - 22s 61ms/step - loss: 0.4925 - custom_metric: 1.3222\n",
      "Epoch 2/1000\n",
      "229/229 [==============================] - ETA: 0s - loss: 0.4177val_aucs: 0.5040120418451598 0.8963692159737133\n",
      "229/229 [==============================] - 12s 53ms/step - loss: 0.4177 - custom_metric: 1.4004\n",
      "Epoch 3/1000\n",
      "228/229 [============================>.] - ETA: 0s - loss: 0.3889val_aucs: 0.45601493333166293 0.8818300513913052\n",
      "229/229 [==============================] - 12s 51ms/step - loss: 0.3888 - custom_metric: 1.3378\n",
      "Epoch 4/1000\n",
      "229/229 [==============================] - ETA: 0s - loss: 0.3625val_aucs: 0.5124622309840912 0.8949066174236227\n",
      "229/229 [==============================] - 12s 51ms/step - loss: 0.3625 - custom_metric: 1.4074\n",
      "Epoch 5/1000\n",
      "229/229 [==============================] - ETA: 0s - loss: 0.3301val_aucs: 0.46086558769920316 0.8766541889423541\n",
      "229/229 [==============================] - 12s 50ms/step - loss: 0.3301 - custom_metric: 1.3375\n",
      "Epoch 6/1000\n",
      "229/229 [==============================] - ETA: 0s - loss: 0.3224val_aucs: 0.5075633073101777 0.8815528923053291\n",
      "229/229 [==============================] - 11s 50ms/step - loss: 0.3224 - custom_metric: 1.3891\n",
      "Epoch 7/1000\n",
      "229/229 [==============================] - ETA: 0s - loss: 0.2845val_aucs: 0.4834222238826433 0.8717521463132833\n",
      "229/229 [==============================] - 12s 50ms/step - loss: 0.2845 - custom_metric: 1.3552\n",
      "Epoch 8/1000\n",
      "229/229 [==============================] - ETA: 0s - loss: 0.2590val_aucs: 0.48358845144163454 0.8666697833150231\n",
      "229/229 [==============================] - 11s 50ms/step - loss: 0.2590 - custom_metric: 1.3503\n",
      "Epoch 9/1000\n",
      "229/229 [==============================] - ETA: 0s - loss: 0.2270val_aucs: 0.4783636578897912 0.867494582040759\n",
      "229/229 [==============================] - 11s 50ms/step - loss: 0.2270 - custom_metric: 1.3459\n",
      "Epoch 10/1000\n",
      "229/229 [==============================] - ETA: 0s - loss: 0.2038val_aucs: 0.4415145632641052 0.8610698340718678\n",
      "229/229 [==============================] - 11s 50ms/step - loss: 0.2038 - custom_metric: 1.3026\n",
      "Epoch 11/1000\n",
      "229/229 [==============================] - ETA: 0s - loss: 0.1722val_aucs: 0.4390087313807091 0.8590362210193444\n",
      "229/229 [==============================] - 11s 50ms/step - loss: 0.1722 - custom_metric: 1.2980\n",
      "Epoch 12/1000\n",
      "229/229 [==============================] - ETA: 0s - loss: 0.1683val_aucs: 0.4356017924317599 0.8690773941703093\n",
      "229/229 [==============================] - 11s 50ms/step - loss: 0.1683 - custom_metric: 1.3047\n",
      "Epoch 13/1000\n",
      "229/229 [==============================] - ETA: 0s - loss: 0.1311val_aucs: 0.40723838127772394 0.8575302120100045\n",
      "229/229 [==============================] - 11s 50ms/step - loss: 0.1311 - custom_metric: 1.2648\n",
      "Epoch 14/1000\n",
      "229/229 [==============================] - ETA: 0s - loss: 0.1270val_aucs: 0.42589604099317074 0.8493690456711424\n",
      "229/229 [==============================] - 12s 51ms/step - loss: 0.1270 - custom_metric: 1.2753\n",
      "Test res 0.879624764390857 0.4727348687848902 0.4736328125\n",
      "Repeat 6 ld 20\n",
      "Num train: 7310 Num valid: 1852\n",
      "Exp_GloVe/M/models/classification/new_mimic_iii_24hm_strats_no_interp_with_ss_repeat6_20ld.h5\n",
      "Epoch 1/1000\n",
      "229/229 [==============================] - ETA: 0s - loss: 0.5132val_aucs: 0.43094197107495513 0.8613872518247149\n",
      "229/229 [==============================] - 21s 64ms/step - loss: 0.5132 - custom_metric: 1.2923\n",
      "Epoch 2/1000\n",
      "229/229 [==============================] - ETA: 0s - loss: 0.4450val_aucs: 0.4577370926188323 0.8688141617242424\n",
      "229/229 [==============================] - 12s 51ms/step - loss: 0.4450 - custom_metric: 1.3266\n",
      "Epoch 3/1000\n",
      "229/229 [==============================] - ETA: 0s - loss: 0.4230val_aucs: 0.47806875572096125 0.8714438749855553\n",
      "229/229 [==============================] - 12s 52ms/step - loss: 0.4230 - custom_metric: 1.3495\n",
      "Epoch 4/1000\n",
      "228/229 [============================>.] - ETA: 0s - loss: 0.3887val_aucs: 0.47173328648245505 0.8700665548570053\n",
      "229/229 [==============================] - 12s 51ms/step - loss: 0.3886 - custom_metric: 1.3418\n",
      "Epoch 5/1000\n",
      "228/229 [============================>.] - ETA: 0s - loss: 0.3577val_aucs: 0.4956617665023305 0.866549859925606\n",
      "229/229 [==============================] - 12s 51ms/step - loss: 0.3572 - custom_metric: 1.3622\n",
      "Epoch 6/1000\n",
      "229/229 [==============================] - ETA: 0s - loss: 0.3378val_aucs: 0.46232320888206563 0.8638732990408731\n",
      "229/229 [==============================] - 11s 50ms/step - loss: 0.3378 - custom_metric: 1.3262\n",
      "Epoch 7/1000\n",
      "229/229 [==============================] - ETA: 0s - loss: 0.3149val_aucs: 0.46248986873683007 0.8601754599655825\n",
      "229/229 [==============================] - 12s 50ms/step - loss: 0.3149 - custom_metric: 1.3227\n",
      "Epoch 8/1000\n",
      "229/229 [==============================] - ETA: 0s - loss: 0.2828val_aucs: 0.4378225662336773 0.8481356207466262\n",
      "229/229 [==============================] - 12s 50ms/step - loss: 0.2828 - custom_metric: 1.2860\n",
      "Epoch 9/1000\n",
      "229/229 [==============================] - ETA: 0s - loss: 0.2694val_aucs: 0.46515980565748055 0.8606845374734139\n",
      "229/229 [==============================] - 11s 50ms/step - loss: 0.2694 - custom_metric: 1.3258\n",
      "Epoch 10/1000\n",
      "229/229 [==============================] - ETA: 0s - loss: 0.2321val_aucs: 0.4617252281064584 0.8562465059480866\n",
      "229/229 [==============================] - 11s 50ms/step - loss: 0.2321 - custom_metric: 1.3180\n",
      "Epoch 11/1000\n",
      "228/229 [============================>.] - ETA: 0s - loss: 0.2141val_aucs: 0.3985494187435567 0.8236249441732486\n",
      "229/229 [==============================] - 12s 50ms/step - loss: 0.2145 - custom_metric: 1.2222\n",
      "Epoch 12/1000\n",
      "228/229 [============================>.] - ETA: 0s - loss: 0.2039val_aucs: 0.41042038691770477 0.8302773066988979\n",
      "229/229 [==============================] - 11s 50ms/step - loss: 0.2038 - custom_metric: 1.2407\n",
      "Epoch 13/1000\n",
      "229/229 [==============================] - ETA: 0s - loss: 0.1776val_aucs: 0.4357500281234324 0.8430261066189446\n",
      "229/229 [==============================] - 11s 50ms/step - loss: 0.1776 - custom_metric: 1.2788\n",
      "Epoch 14/1000\n",
      "229/229 [==============================] - ETA: 0s - loss: 0.1439val_aucs: 0.38432408815407526 0.8313610483873486\n",
      "229/229 [==============================] - 11s 50ms/step - loss: 0.1439 - custom_metric: 1.2157\n",
      "Epoch 15/1000\n",
      "229/229 [==============================] - ETA: 0s - loss: 0.1471val_aucs: 0.38414322208463864 0.8374200076830103\n",
      "229/229 [==============================] - 12s 51ms/step - loss: 0.1471 - custom_metric: 1.2216\n",
      "Test res 0.8751864304092867 0.46520423001047917 0.4747572815533981\n",
      "Repeat 7 ld 20\n",
      "Num train: 7310 Num valid: 1852\n",
      "Exp_GloVe/M/models/classification/new_mimic_iii_24hm_strats_no_interp_with_ss_repeat7_20ld.h5\n",
      "Epoch 1/1000\n",
      "229/229 [==============================] - ETA: 0s - loss: 0.5228val_aucs: 0.40907346155381097 0.8537283725297651\n",
      "229/229 [==============================] - 22s 61ms/step - loss: 0.5228 - custom_metric: 1.2628\n",
      "Epoch 2/1000\n",
      "229/229 [==============================] - ETA: 0s - loss: 0.4455val_aucs: 0.4501761933903844 0.8707426988173289\n",
      "229/229 [==============================] - 12s 54ms/step - loss: 0.4455 - custom_metric: 1.3209\n",
      "Epoch 3/1000\n",
      "229/229 [==============================] - ETA: 0s - loss: 0.4051val_aucs: 0.44872200286234126 0.8741812723383292\n",
      "229/229 [==============================] - 12s 51ms/step - loss: 0.4051 - custom_metric: 1.3229\n",
      "Epoch 4/1000\n",
      "228/229 [============================>.] - ETA: 0s - loss: 0.3796val_aucs: 0.4668775917931001 0.8790349511160486\n",
      "229/229 [==============================] - 12s 51ms/step - loss: 0.3793 - custom_metric: 1.3459\n",
      "Epoch 5/1000\n",
      "229/229 [==============================] - ETA: 0s - loss: 0.3576val_aucs: 0.4347448926602492 0.8702963455237378\n",
      "229/229 [==============================] - 11s 50ms/step - loss: 0.3576 - custom_metric: 1.3050\n",
      "Epoch 6/1000\n",
      "229/229 [==============================] - ETA: 0s - loss: 0.3442val_aucs: 0.4295328801616406 0.8719131363427464\n",
      "229/229 [==============================] - 12s 50ms/step - loss: 0.3442 - custom_metric: 1.3014\n",
      "Epoch 7/1000\n",
      "229/229 [==============================] - ETA: 0s - loss: 0.2940val_aucs: 0.409080934028563 0.8629100250949739\n",
      "229/229 [==============================] - 12s 50ms/step - loss: 0.2940 - custom_metric: 1.2720\n",
      "Epoch 8/1000\n",
      "229/229 [==============================] - ETA: 0s - loss: 0.2680val_aucs: 0.39928215240848175 0.8625231855738617\n",
      "229/229 [==============================] - 12s 50ms/step - loss: 0.2680 - custom_metric: 1.2618\n",
      "Epoch 9/1000\n",
      "229/229 [==============================] - ETA: 0s - loss: 0.2459val_aucs: 0.39459860360226007 0.8518437697346016\n",
      "229/229 [==============================] - 12s 50ms/step - loss: 0.2459 - custom_metric: 1.2464\n",
      "Epoch 10/1000\n",
      "229/229 [==============================] - ETA: 0s - loss: 0.2137val_aucs: 0.37494106824153434 0.8462858446492159\n",
      "229/229 [==============================] - 11s 50ms/step - loss: 0.2137 - custom_metric: 1.2212\n",
      "Epoch 11/1000\n",
      "229/229 [==============================] - ETA: 0s - loss: 0.2067val_aucs: 0.38670027347464675 0.8505906741918525\n",
      "229/229 [==============================] - 11s 50ms/step - loss: 0.2067 - custom_metric: 1.2373\n",
      "Epoch 12/1000\n",
      "229/229 [==============================] - ETA: 0s - loss: 0.1646val_aucs: 0.3871440281514571 0.8469305771844035\n",
      "229/229 [==============================] - 12s 50ms/step - loss: 0.1646 - custom_metric: 1.2341\n",
      "Epoch 13/1000\n",
      "229/229 [==============================] - ETA: 0s - loss: 0.1523val_aucs: 0.40016692652972974 0.852389312648991\n",
      "229/229 [==============================] - 12s 50ms/step - loss: 0.1523 - custom_metric: 1.2526\n",
      "Epoch 14/1000\n",
      "229/229 [==============================] - ETA: 0s - loss: 0.1247val_aucs: 0.39209921761253197 0.8488019546967939\n",
      "229/229 [==============================] - 12s 51ms/step - loss: 0.1247 - custom_metric: 1.2409\n",
      "Test res 0.8761013792484443 0.442782270684245 0.4606413994169096\n",
      "Repeat 8 ld 20\n",
      "Num train: 7310 Num valid: 1852\n",
      "Exp_GloVe/M/models/classification/new_mimic_iii_24hm_strats_no_interp_with_ss_repeat8_20ld.h5\n",
      "Epoch 1/1000\n",
      "229/229 [==============================] - ETA: 0s - loss: 0.4835val_aucs: 0.5031684577846481 0.8766061845131613\n",
      "229/229 [==============================] - 22s 66ms/step - loss: 0.4835 - custom_metric: 1.3798\n",
      "Epoch 2/1000\n",
      "229/229 [==============================] - ETA: 0s - loss: 0.4210val_aucs: 0.5484046151475437 0.8933333333333333\n",
      "229/229 [==============================] - 12s 51ms/step - loss: 0.4210 - custom_metric: 1.4417\n",
      "Epoch 3/1000\n",
      "229/229 [==============================] - ETA: 0s - loss: 0.3928val_aucs: 0.5382841621321229 0.8961274384530199\n",
      "229/229 [==============================] - 12s 51ms/step - loss: 0.3928 - custom_metric: 1.4344\n",
      "Epoch 4/1000\n",
      "229/229 [==============================] - ETA: 0s - loss: 0.3526val_aucs: 0.5604512826780594 0.9030377374563421\n",
      "229/229 [==============================] - 12s 51ms/step - loss: 0.3526 - custom_metric: 1.4635\n",
      "Epoch 5/1000\n",
      "229/229 [==============================] - ETA: 0s - loss: 0.3234val_aucs: 0.5500943883610654 0.8986250958343981\n",
      "229/229 [==============================] - 11s 50ms/step - loss: 0.3234 - custom_metric: 1.4487\n",
      "Epoch 6/1000\n",
      "229/229 [==============================] - ETA: 0s - loss: 0.2879val_aucs: 0.5560621196758433 0.8941954169861146\n",
      "229/229 [==============================] - 12s 51ms/step - loss: 0.2879 - custom_metric: 1.4503\n",
      "Epoch 7/1000\n",
      "229/229 [==============================] - ETA: 0s - loss: 0.2531val_aucs: 0.4910797393164268 0.8817991311014567\n",
      "229/229 [==============================] - 11s 50ms/step - loss: 0.2531 - custom_metric: 1.3729\n",
      "Epoch 8/1000\n",
      "229/229 [==============================] - ETA: 0s - loss: 0.2312val_aucs: 0.4926358711330483 0.8876906039696737\n",
      "229/229 [==============================] - 12s 50ms/step - loss: 0.2312 - custom_metric: 1.3803\n",
      "Epoch 9/1000\n",
      "229/229 [==============================] - ETA: 0s - loss: 0.1904val_aucs: 0.4655822146096271 0.8815435727063635\n",
      "229/229 [==============================] - 12s 50ms/step - loss: 0.1904 - custom_metric: 1.3471\n",
      "Epoch 10/1000\n",
      "229/229 [==============================] - ETA: 0s - loss: 0.1686val_aucs: 0.5020999205063147 0.8843615299429253\n",
      "229/229 [==============================] - 11s 50ms/step - loss: 0.1686 - custom_metric: 1.3865\n",
      "Epoch 11/1000\n",
      "229/229 [==============================] - ETA: 0s - loss: 0.1734val_aucs: 0.5076657645263412 0.8896601073345259\n",
      "229/229 [==============================] - 11s 50ms/step - loss: 0.1734 - custom_metric: 1.3973\n",
      "Epoch 12/1000\n",
      "229/229 [==============================] - ETA: 0s - loss: 0.1281val_aucs: 0.4794372552271216 0.8785041315273874\n",
      "229/229 [==============================] - 11s 50ms/step - loss: 0.1281 - custom_metric: 1.3579\n",
      "Epoch 13/1000\n",
      "229/229 [==============================] - ETA: 0s - loss: 0.1317val_aucs: 0.43600220448952404 0.88631058863617\n",
      "229/229 [==============================] - 11s 50ms/step - loss: 0.1317 - custom_metric: 1.3223\n",
      "Epoch 14/1000\n",
      "229/229 [==============================] - ETA: 0s - loss: 0.1148val_aucs: 0.5172070711507507 0.8839560439560441\n",
      "229/229 [==============================] - 13s 57ms/step - loss: 0.1148 - custom_metric: 1.4012\n",
      "Test res 0.872197840623504 0.44016841698162157 0.4534220532319392\n",
      "Repeat 9 ld 20\n",
      "Num train: 7310 Num valid: 1852\n",
      "Exp_GloVe/M/models/classification/new_mimic_iii_24hm_strats_no_interp_with_ss_repeat9_20ld.h5\n",
      "Epoch 1/1000\n",
      "229/229 [==============================] - ETA: 0s - loss: 0.4924val_aucs: 0.5300683167016764 0.8940818186978867\n",
      "229/229 [==============================] - 21s 62ms/step - loss: 0.4924 - custom_metric: 1.4242\n",
      "Epoch 2/1000\n",
      "229/229 [==============================] - ETA: 0s - loss: 0.4137val_aucs: 0.5518419652387433 0.8896940230476147\n",
      "229/229 [==============================] - 12s 52ms/step - loss: 0.4137 - custom_metric: 1.4415\n",
      "Epoch 3/1000\n",
      "229/229 [==============================] - ETA: 0s - loss: 0.3787val_aucs: 0.582176132267137 0.8978485108542844\n",
      "229/229 [==============================] - 12s 51ms/step - loss: 0.3787 - custom_metric: 1.4800\n",
      "Epoch 4/1000\n",
      "229/229 [==============================] - ETA: 0s - loss: 0.3504val_aucs: 0.5493368563926955 0.8882781742228693\n",
      "229/229 [==============================] - 11s 50ms/step - loss: 0.3504 - custom_metric: 1.4376\n",
      "Epoch 5/1000\n",
      "229/229 [==============================] - ETA: 0s - loss: 0.3256val_aucs: 0.5057770322026972 0.881890158181035\n",
      "229/229 [==============================] - 12s 50ms/step - loss: 0.3256 - custom_metric: 1.3877\n",
      "Epoch 6/1000\n",
      "229/229 [==============================] - ETA: 0s - loss: 0.2828val_aucs: 0.558350134826901 0.8865617914494752\n",
      "229/229 [==============================] - 12s 51ms/step - loss: 0.2828 - custom_metric: 1.4449\n",
      "Epoch 7/1000\n",
      "229/229 [==============================] - ETA: 0s - loss: 0.2713val_aucs: 0.5338658646361921 0.8878373910981845\n",
      "229/229 [==============================] - 11s 50ms/step - loss: 0.2713 - custom_metric: 1.4217\n",
      "Epoch 8/1000\n",
      "229/229 [==============================] - ETA: 0s - loss: 0.2404val_aucs: 0.5069743142898718 0.8706234743728023\n",
      "229/229 [==============================] - 12s 51ms/step - loss: 0.2404 - custom_metric: 1.3776\n",
      "Epoch 9/1000\n",
      "229/229 [==============================] - ETA: 0s - loss: 0.2181val_aucs: 0.5033046761328885 0.8732147448633739\n",
      "229/229 [==============================] - 12s 50ms/step - loss: 0.2181 - custom_metric: 1.3765\n",
      "Epoch 10/1000\n",
      "229/229 [==============================] - ETA: 0s - loss: 0.2062val_aucs: 0.5192077125975721 0.8872964299906166\n",
      "229/229 [==============================] - 11s 50ms/step - loss: 0.2062 - custom_metric: 1.4065\n",
      "Epoch 11/1000\n",
      "229/229 [==============================] - ETA: 0s - loss: 0.1763val_aucs: 0.4403666792865598 0.8659885730314192\n",
      "229/229 [==============================] - 12s 51ms/step - loss: 0.1763 - custom_metric: 1.3064\n",
      "Epoch 12/1000\n",
      "229/229 [==============================] - ETA: 0s - loss: 0.1651val_aucs: 0.45931509504780194 0.8650702748549923\n",
      "229/229 [==============================] - 11s 50ms/step - loss: 0.1651 - custom_metric: 1.3244\n",
      "Epoch 13/1000\n",
      "229/229 [==============================] - ETA: 0s - loss: 0.1642val_aucs: 0.45341209619833533 0.854771978214628\n",
      "229/229 [==============================] - 12s 51ms/step - loss: 0.1642 - custom_metric: 1.3082\n",
      "Test res 0.8769452713618955 0.47018940583069513 0.4771178188899708\n",
      "gen_res {10: [(0.8495673253126496, 0.015550036538510871), (0.39991067148684534, 0.03244069307475938), (0.43139881908261096, 0.025470753193600273)], 20: [(0.8741984427357586, 0.002984760713655759), (0.45556223727894085, 0.01150887998551242), (0.4675065393501395, 0.00985098073216147)]}\n",
      "Repeat 0 ld 30\n",
      "Num train: 10965 Num valid: 2778\n",
      "Exp_GloVe/M/models/classification/new_mimic_iii_24hm_strats_no_interp_with_ss_repeat0_30ld.h5\n",
      "Epoch 1/1000\n",
      "343/343 [==============================] - ETA: 0s - loss: 0.4814val_aucs: 0.4537709067499367 0.8660402877697841\n",
      "343/343 [==============================] - 27s 57ms/step - loss: 0.4814 - custom_metric: 1.3198\n",
      "Epoch 2/1000\n",
      "343/343 [==============================] - ETA: 0s - loss: 0.4189val_aucs: 0.47807100665843416 0.8733913669064748\n",
      "343/343 [==============================] - 17s 51ms/step - loss: 0.4189 - custom_metric: 1.3515\n",
      "Epoch 3/1000\n",
      "343/343 [==============================] - ETA: 0s - loss: 0.3813val_aucs: 0.4733954898528726 0.8751812949640287\n",
      "343/343 [==============================] - 17s 50ms/step - loss: 0.3813 - custom_metric: 1.3486\n",
      "Epoch 4/1000\n",
      "342/343 [============================>.] - ETA: 0s - loss: 0.3630val_aucs: 0.49361614729313036 0.8684172661870504\n",
      "343/343 [==============================] - 17s 51ms/step - loss: 0.3626 - custom_metric: 1.3620\n",
      "Epoch 5/1000\n",
      "343/343 [==============================] - ETA: 0s - loss: 0.3343val_aucs: 0.4850229284044772 0.8622776978417267\n",
      "343/343 [==============================] - 17s 50ms/step - loss: 0.3343 - custom_metric: 1.3473\n",
      "Epoch 6/1000\n",
      "343/343 [==============================] - ETA: 0s - loss: 0.3111val_aucs: 0.47269242006954665 0.8659453237410072\n",
      "343/343 [==============================] - 17s 50ms/step - loss: 0.3111 - custom_metric: 1.3386\n",
      "Epoch 7/1000\n",
      "343/343 [==============================] - ETA: 0s - loss: 0.2801val_aucs: 0.49961527523809357 0.8699438848920863\n",
      "343/343 [==============================] - 17s 50ms/step - loss: 0.2801 - custom_metric: 1.3696\n",
      "Epoch 8/1000\n",
      "343/343 [==============================] - ETA: 0s - loss: 0.2531val_aucs: 0.47493809760277533 0.8649496402877698\n",
      "343/343 [==============================] - 17s 50ms/step - loss: 0.2531 - custom_metric: 1.3399\n",
      "Epoch 9/1000\n",
      "343/343 [==============================] - ETA: 0s - loss: 0.2483val_aucs: 0.4616846484577599 0.8623151079136692\n",
      "343/343 [==============================] - 17s 50ms/step - loss: 0.2483 - custom_metric: 1.3240\n",
      "Epoch 10/1000\n",
      "343/343 [==============================] - ETA: 0s - loss: 0.2273val_aucs: 0.4618042783680078 0.8603438848920864\n",
      "343/343 [==============================] - 17s 50ms/step - loss: 0.2273 - custom_metric: 1.3221\n",
      "Epoch 11/1000\n",
      "343/343 [==============================] - ETA: 0s - loss: 0.2061val_aucs: 0.4754133193662206 0.8517064748201438\n",
      "343/343 [==============================] - 17s 50ms/step - loss: 0.2061 - custom_metric: 1.3271\n",
      "Epoch 12/1000\n",
      "343/343 [==============================] - ETA: 0s - loss: 0.1743val_aucs: 0.44389638825073235 0.8539856115107914\n",
      "343/343 [==============================] - 17s 50ms/step - loss: 0.1743 - custom_metric: 1.2979\n",
      "Epoch 13/1000\n",
      "343/343 [==============================] - ETA: 0s - loss: 0.1590val_aucs: 0.4663338461455155 0.8579741007194246\n",
      "343/343 [==============================] - 17s 50ms/step - loss: 0.1590 - custom_metric: 1.3243\n",
      "Epoch 14/1000\n",
      "343/343 [==============================] - ETA: 0s - loss: 0.1546val_aucs: 0.45736189216382533 0.8551136690647483\n",
      "343/343 [==============================] - 17s 50ms/step - loss: 0.1546 - custom_metric: 1.3125\n",
      "Epoch 15/1000\n",
      "343/343 [==============================] - ETA: 0s - loss: 0.1228val_aucs: 0.4044351889021357 0.8430201438848922\n",
      "343/343 [==============================] - 17s 50ms/step - loss: 0.1228 - custom_metric: 1.2475\n",
      "Epoch 16/1000\n",
      "343/343 [==============================] - ETA: 0s - loss: 0.1356val_aucs: 0.43658414280544805 0.8476028776978417\n",
      "343/343 [==============================] - 17s 50ms/step - loss: 0.1356 - custom_metric: 1.2842\n",
      "Epoch 17/1000\n",
      "343/343 [==============================] - ETA: 0s - loss: 0.1445val_aucs: 0.44675744196446526 0.8516446043165466\n",
      "343/343 [==============================] - 17s 50ms/step - loss: 0.1445 - custom_metric: 1.2984\n",
      "Test res 0.8775530868537579 0.4480171819400565 0.4859359844810863\n",
      "Repeat 1 ld 30\n",
      "Num train: 10965 Num valid: 2778\n",
      "Exp_GloVe/M/models/classification/new_mimic_iii_24hm_strats_no_interp_with_ss_repeat1_30ld.h5\n",
      "Epoch 1/1000\n",
      "342/343 [============================>.] - ETA: 0s - loss: 0.4797val_aucs: 0.4410133908704751 0.8621084515366431\n",
      "343/343 [==============================] - 26s 58ms/step - loss: 0.4794 - custom_metric: 1.3031\n",
      "Epoch 2/1000\n",
      "343/343 [==============================] - ETA: 0s - loss: 0.4086val_aucs: 0.479894695921461 0.8748721358428804\n",
      "343/343 [==============================] - 18s 51ms/step - loss: 0.4086 - custom_metric: 1.3548\n",
      "Epoch 3/1000\n",
      "343/343 [==============================] - ETA: 0s - loss: 0.3778val_aucs: 0.49104546954598505 0.8747741066557555\n",
      "343/343 [==============================] - 17s 50ms/step - loss: 0.3778 - custom_metric: 1.3658\n",
      "Epoch 4/1000\n",
      "342/343 [============================>.] - ETA: 0s - loss: 0.3604val_aucs: 0.48073529200560655 0.8744104041643935\n",
      "343/343 [==============================] - 17s 50ms/step - loss: 0.3612 - custom_metric: 1.3551\n",
      "Epoch 5/1000\n",
      "343/343 [==============================] - ETA: 0s - loss: 0.3400val_aucs: 0.45819285323228587 0.8693682942353155\n",
      "343/343 [==============================] - 17s 50ms/step - loss: 0.3400 - custom_metric: 1.3276\n",
      "Epoch 6/1000\n",
      "343/343 [==============================] - ETA: 0s - loss: 0.3107val_aucs: 0.4770562126619393 0.8718751420712857\n",
      "343/343 [==============================] - 17s 50ms/step - loss: 0.3107 - custom_metric: 1.3489\n",
      "Epoch 7/1000\n",
      "343/343 [==============================] - ETA: 0s - loss: 0.2931val_aucs: 0.45916552147490913 0.8746632910529187\n",
      "343/343 [==============================] - 17s 50ms/step - loss: 0.2931 - custom_metric: 1.3338\n",
      "Epoch 8/1000\n",
      "343/343 [==============================] - ETA: 0s - loss: 0.2732val_aucs: 0.4488618893069359 0.8628656914893617\n",
      "343/343 [==============================] - 17s 50ms/step - loss: 0.2732 - custom_metric: 1.3117\n",
      "Epoch 9/1000\n",
      "343/343 [==============================] - ETA: 0s - loss: 0.2537val_aucs: 0.44751434749295427 0.8649470358246953\n",
      "343/343 [==============================] - 17s 50ms/step - loss: 0.2537 - custom_metric: 1.3125\n",
      "Epoch 10/1000\n",
      "342/343 [============================>.] - ETA: 0s - loss: 0.2365val_aucs: 0.44140279261890647 0.8559539234406256\n",
      "343/343 [==============================] - 18s 53ms/step - loss: 0.2363 - custom_metric: 1.2974\n",
      "Epoch 11/1000\n",
      "343/343 [==============================] - ETA: 0s - loss: 0.2099val_aucs: 0.4231668228115141 0.8579713356973994\n",
      "343/343 [==============================] - 18s 52ms/step - loss: 0.2099 - custom_metric: 1.2811\n",
      "Epoch 12/1000\n",
      "343/343 [==============================] - ETA: 0s - loss: 0.2140val_aucs: 0.37296604572567915 0.8470702059465356\n",
      "343/343 [==============================] - 17s 50ms/step - loss: 0.2140 - custom_metric: 1.2200\n",
      "Epoch 13/1000\n",
      "343/343 [==============================] - ETA: 0s - loss: 0.1928val_aucs: 0.3877300835433653 0.8538739998181488\n",
      "343/343 [==============================] - 17s 50ms/step - loss: 0.1928 - custom_metric: 1.2416\n",
      "Test res 0.8815083285962183 0.4811239415202225 0.4951267056530214\n",
      "Repeat 2 ld 30\n",
      "Num train: 10965 Num valid: 2778\n",
      "Exp_GloVe/M/models/classification/new_mimic_iii_24hm_strats_no_interp_with_ss_repeat2_30ld.h5\n",
      "Epoch 1/1000\n",
      "343/343 [==============================] - ETA: 0s - loss: 0.4811val_aucs: 0.4458346434812497 0.8679699327560297\n",
      "343/343 [==============================] - 28s 58ms/step - loss: 0.4811 - custom_metric: 1.3138\n",
      "Epoch 2/1000\n",
      "342/343 [============================>.] - ETA: 0s - loss: 0.4186val_aucs: 0.4960744828581076 0.8912696834931831\n",
      "343/343 [==============================] - 17s 51ms/step - loss: 0.4184 - custom_metric: 1.3873\n",
      "Epoch 3/1000\n",
      "343/343 [==============================] - ETA: 0s - loss: 0.3885val_aucs: 0.5406016254182328 0.8922747980387059\n",
      "343/343 [==============================] - 17s 50ms/step - loss: 0.3885 - custom_metric: 1.4329\n",
      "Epoch 4/1000\n",
      "343/343 [==============================] - ETA: 0s - loss: 0.3649val_aucs: 0.5318338101809335 0.8919286382574678\n",
      "343/343 [==============================] - 17s 50ms/step - loss: 0.3649 - custom_metric: 1.4238\n",
      "Epoch 5/1000\n",
      "343/343 [==============================] - ETA: 0s - loss: 0.3431val_aucs: 0.5236224113047131 0.8934995641723236\n",
      "343/343 [==============================] - 17s 50ms/step - loss: 0.3431 - custom_metric: 1.4171\n",
      "Epoch 6/1000\n",
      "342/343 [============================>.] - ETA: 0s - loss: 0.3107val_aucs: 0.5087075967866116 0.8781920728019896\n",
      "343/343 [==============================] - 17s 50ms/step - loss: 0.3105 - custom_metric: 1.3869\n",
      "Epoch 7/1000\n",
      "343/343 [==============================] - ETA: 0s - loss: 0.2904val_aucs: 0.48947247767863516 0.869969040247678\n",
      "343/343 [==============================] - 17s 50ms/step - loss: 0.2904 - custom_metric: 1.3594\n",
      "Epoch 8/1000\n",
      "343/343 [==============================] - ETA: 0s - loss: 0.2594val_aucs: 0.49512771866812266 0.8777930854236147\n",
      "343/343 [==============================] - 18s 52ms/step - loss: 0.2594 - custom_metric: 1.3729\n",
      "Epoch 9/1000\n",
      "343/343 [==============================] - ETA: 0s - loss: 0.2414val_aucs: 0.4715979062496779 0.8657664658754576\n",
      "343/343 [==============================] - 17s 50ms/step - loss: 0.2414 - custom_metric: 1.3374\n",
      "Epoch 10/1000\n",
      "343/343 [==============================] - ETA: 0s - loss: 0.2300val_aucs: 0.4451202691866408 0.8611065157280269\n",
      "343/343 [==============================] - 17s 50ms/step - loss: 0.2300 - custom_metric: 1.3062\n",
      "Epoch 11/1000\n",
      "343/343 [==============================] - ETA: 0s - loss: 0.1988val_aucs: 0.5098158440425566 0.8814520916252967\n",
      "343/343 [==============================] - 17s 50ms/step - loss: 0.1988 - custom_metric: 1.3913\n",
      "Epoch 12/1000\n",
      "343/343 [==============================] - ETA: 0s - loss: 0.1932val_aucs: 0.4454841237414401 0.862564835448986\n",
      "343/343 [==============================] - 20s 57ms/step - loss: 0.1932 - custom_metric: 1.3080\n",
      "Epoch 13/1000\n",
      "343/343 [==============================] - ETA: 0s - loss: 0.1766val_aucs: 0.44831247645267397 0.869255867686332\n",
      "343/343 [==============================] - 17s 50ms/step - loss: 0.1766 - custom_metric: 1.3176\n",
      "Test res 0.8752791781354715 0.45658047254258594 0.4649805447470817\n",
      "Repeat 3 ld 30\n",
      "Num train: 10965 Num valid: 2778\n",
      "Exp_GloVe/M/models/classification/new_mimic_iii_24hm_strats_no_interp_with_ss_repeat3_30ld.h5\n",
      "Epoch 1/1000\n",
      "343/343 [==============================] - ETA: 0s - loss: 0.4899val_aucs: 0.49950721422112165 0.8967339596836534\n",
      "343/343 [==============================] - 27s 58ms/step - loss: 0.4899 - custom_metric: 1.3962\n",
      "Epoch 2/1000\n",
      "343/343 [==============================] - ETA: 0s - loss: 0.4192val_aucs: 0.5115380994021961 0.8980317535826843\n",
      "343/343 [==============================] - 18s 52ms/step - loss: 0.4192 - custom_metric: 1.4096\n",
      "Epoch 3/1000\n",
      "343/343 [==============================] - ETA: 0s - loss: 0.3984val_aucs: 0.5244596423673239 0.9053101028720938\n",
      "343/343 [==============================] - 17s 51ms/step - loss: 0.3984 - custom_metric: 1.4298\n",
      "Epoch 4/1000\n",
      "343/343 [==============================] - ETA: 0s - loss: 0.3752val_aucs: 0.5621744343971785 0.906723850865196\n",
      "343/343 [==============================] - 17s 51ms/step - loss: 0.3752 - custom_metric: 1.4689\n",
      "Epoch 5/1000\n",
      "343/343 [==============================] - ETA: 0s - loss: 0.3479val_aucs: 0.5066435115284951 0.8994559077124339\n",
      "343/343 [==============================] - 17s 50ms/step - loss: 0.3479 - custom_metric: 1.4061\n",
      "Epoch 6/1000\n",
      "343/343 [==============================] - ETA: 0s - loss: 0.3364val_aucs: 0.5146864923649872 0.8995510495332105\n",
      "343/343 [==============================] - 17s 50ms/step - loss: 0.3364 - custom_metric: 1.4142\n",
      "Epoch 7/1000\n",
      "343/343 [==============================] - ETA: 0s - loss: 0.3081val_aucs: 0.5105872874424029 0.9009216863887733\n",
      "343/343 [==============================] - 17s 50ms/step - loss: 0.3081 - custom_metric: 1.4115\n",
      "Epoch 8/1000\n",
      "343/343 [==============================] - ETA: 0s - loss: 0.2893val_aucs: 0.45638737618533887 0.888438782184694\n",
      "343/343 [==============================] - 17s 50ms/step - loss: 0.2893 - custom_metric: 1.3448\n",
      "Epoch 9/1000\n",
      "343/343 [==============================] - ETA: 0s - loss: 0.2532val_aucs: 0.47841599665087714 0.8887598858298151\n",
      "343/343 [==============================] - 17s 50ms/step - loss: 0.2532 - custom_metric: 1.3672\n",
      "Epoch 10/1000\n",
      "343/343 [==============================] - ETA: 0s - loss: 0.2328val_aucs: 0.40428986409531115 0.8681527620859844\n",
      "343/343 [==============================] - 17s 50ms/step - loss: 0.2328 - custom_metric: 1.2724\n",
      "Epoch 11/1000\n",
      "343/343 [==============================] - ETA: 0s - loss: 0.2134val_aucs: 0.4181719764363301 0.8708509246595707\n",
      "343/343 [==============================] - 17s 50ms/step - loss: 0.2134 - custom_metric: 1.2890\n",
      "Epoch 12/1000\n",
      "343/343 [==============================] - ETA: 0s - loss: 0.1928val_aucs: 0.39715292697886717 0.8648317179045013\n",
      "343/343 [==============================] - 17s 50ms/step - loss: 0.1928 - custom_metric: 1.2620\n",
      "Epoch 13/1000\n",
      "343/343 [==============================] - ETA: 0s - loss: 0.1689val_aucs: 0.4233617984113316 0.8728920140334185\n",
      "343/343 [==============================] - 17s 50ms/step - loss: 0.1689 - custom_metric: 1.2963\n",
      "Epoch 14/1000\n",
      "343/343 [==============================] - ETA: 0s - loss: 0.1741val_aucs: 0.4331059558442161 0.8753211036451211\n",
      "343/343 [==============================] - 17s 51ms/step - loss: 0.1741 - custom_metric: 1.3084\n",
      "Test res 0.8809635292005745 0.4918100765125399 0.4814453125\n",
      "Repeat 4 ld 30\n",
      "Num train: 10965 Num valid: 2778\n",
      "Exp_GloVe/M/models/classification/new_mimic_iii_24hm_strats_no_interp_with_ss_repeat4_30ld.h5\n",
      "Epoch 1/1000\n",
      "342/343 [============================>.] - ETA: 0s - loss: 0.5139val_aucs: 0.5145424603672287 0.8977890479978786\n",
      "343/343 [==============================] - 28s 57ms/step - loss: 0.5135 - custom_metric: 1.4123\n",
      "Epoch 2/1000\n",
      "342/343 [============================>.] - ETA: 0s - loss: 0.4349val_aucs: 0.5260657051744453 0.9044622839372242\n",
      "343/343 [==============================] - 18s 52ms/step - loss: 0.4347 - custom_metric: 1.4305\n",
      "Epoch 3/1000\n",
      "342/343 [============================>.] - ETA: 0s - loss: 0.4134val_aucs: 0.5333671646572454 0.9054732889756756\n",
      "343/343 [==============================] - 18s 51ms/step - loss: 0.4141 - custom_metric: 1.4388\n",
      "Epoch 4/1000\n",
      "343/343 [==============================] - ETA: 0s - loss: 0.3930val_aucs: 0.5695024438062659 0.9149339456618693\n",
      "343/343 [==============================] - 17s 51ms/step - loss: 0.3930 - custom_metric: 1.4844\n",
      "Epoch 5/1000\n",
      "343/343 [==============================] - ETA: 0s - loss: 0.3733val_aucs: 0.5484221665212627 0.9056616282153275\n",
      "343/343 [==============================] - 17s 50ms/step - loss: 0.3733 - custom_metric: 1.4541\n",
      "Epoch 6/1000\n",
      "343/343 [==============================] - ETA: 0s - loss: 0.3437val_aucs: 0.5586050421935724 0.9072391576866518\n",
      "343/343 [==============================] - 17s 50ms/step - loss: 0.3437 - custom_metric: 1.4658\n",
      "Epoch 7/1000\n",
      "343/343 [==============================] - ETA: 0s - loss: 0.3289val_aucs: 0.5591990982444531 0.9019852462573227\n",
      "343/343 [==============================] - 17s 50ms/step - loss: 0.3289 - custom_metric: 1.4612\n",
      "Epoch 8/1000\n",
      "343/343 [==============================] - ETA: 0s - loss: 0.3224val_aucs: 0.4895163925477004 0.9044517369398037\n",
      "343/343 [==============================] - 17s 50ms/step - loss: 0.3224 - custom_metric: 1.3940\n",
      "Epoch 9/1000\n",
      "343/343 [==============================] - ETA: 0s - loss: 0.2825val_aucs: 0.5252800812378564 0.9013418794146717\n",
      "343/343 [==============================] - 17s 50ms/step - loss: 0.2825 - custom_metric: 1.4266\n",
      "Epoch 10/1000\n",
      "343/343 [==============================] - ETA: 0s - loss: 0.2497val_aucs: 0.51315494094924 0.8882229213374799\n",
      "343/343 [==============================] - 17s 50ms/step - loss: 0.2497 - custom_metric: 1.4014\n",
      "Epoch 11/1000\n",
      "343/343 [==============================] - ETA: 0s - loss: 0.2346val_aucs: 0.4704325648435798 0.8876458499071865\n",
      "343/343 [==============================] - 17s 50ms/step - loss: 0.2346 - custom_metric: 1.3581\n",
      "Epoch 12/1000\n",
      "343/343 [==============================] - ETA: 0s - loss: 0.2211val_aucs: 0.4820315350250761 0.8809153588389864\n",
      "343/343 [==============================] - 17s 50ms/step - loss: 0.2211 - custom_metric: 1.3629\n",
      "Epoch 13/1000\n",
      "343/343 [==============================] - ETA: 0s - loss: 0.2090val_aucs: 0.499612322097349 0.8908430968395169\n",
      "343/343 [==============================] - 17s 50ms/step - loss: 0.2090 - custom_metric: 1.3905\n",
      "Epoch 14/1000\n",
      "343/343 [==============================] - ETA: 0s - loss: 0.1746val_aucs: 0.5234388524978973 0.8852818157710758\n",
      "343/343 [==============================] - 17s 50ms/step - loss: 0.1746 - custom_metric: 1.4087\n",
      "Test res 0.8801743320667783 0.4823702743481985 0.474609375\n",
      "Repeat 5 ld 30\n",
      "Num train: 10965 Num valid: 2778\n",
      "Exp_GloVe/M/models/classification/new_mimic_iii_24hm_strats_no_interp_with_ss_repeat5_30ld.h5\n",
      "Epoch 1/1000\n",
      "343/343 [==============================] - ETA: 0s - loss: 0.4946val_aucs: 0.48275228065228465 0.9003658430808883\n",
      "343/343 [==============================] - 28s 58ms/step - loss: 0.4946 - custom_metric: 1.3831\n",
      "Epoch 2/1000\n",
      "343/343 [==============================] - ETA: 0s - loss: 0.4316val_aucs: 0.5149957795806306 0.9052169037470262\n",
      "343/343 [==============================] - 17s 50ms/step - loss: 0.4316 - custom_metric: 1.4202\n",
      "Epoch 3/1000\n",
      "343/343 [==============================] - ETA: 0s - loss: 0.4017val_aucs: 0.5118939793691454 0.9076966445281522\n",
      "343/343 [==============================] - 17s 50ms/step - loss: 0.4017 - custom_metric: 1.4196\n",
      "Epoch 4/1000\n",
      "342/343 [============================>.] - ETA: 0s - loss: 0.3895val_aucs: 0.5190541348866939 0.9078097120341\n",
      "343/343 [==============================] - 18s 51ms/step - loss: 0.3893 - custom_metric: 1.4269\n",
      "Epoch 5/1000\n",
      "342/343 [============================>.] - ETA: 0s - loss: 0.3619val_aucs: 0.518241954085981 0.8997741747620936\n",
      "343/343 [==============================] - 17s 50ms/step - loss: 0.3620 - custom_metric: 1.4180\n",
      "Epoch 6/1000\n",
      "342/343 [============================>.] - ETA: 0s - loss: 0.3443val_aucs: 0.5086492573248825 0.8989780556106265\n",
      "343/343 [==============================] - 17s 50ms/step - loss: 0.3443 - custom_metric: 1.4076\n",
      "Epoch 7/1000\n",
      "343/343 [==============================] - ETA: 0s - loss: 0.3203val_aucs: 0.5059959455080699 0.9022694042426644\n",
      "343/343 [==============================] - 17s 50ms/step - loss: 0.3203 - custom_metric: 1.4083\n",
      "Epoch 8/1000\n",
      "343/343 [==============================] - ETA: 0s - loss: 0.2916val_aucs: 0.5271674186779218 0.8955271733743062\n",
      "343/343 [==============================] - 17s 50ms/step - loss: 0.2916 - custom_metric: 1.4227\n",
      "Epoch 9/1000\n",
      "343/343 [==============================] - ETA: 0s - loss: 0.2662val_aucs: 0.4795000830255227 0.8802398270222046\n",
      "343/343 [==============================] - 17s 50ms/step - loss: 0.2662 - custom_metric: 1.3597\n",
      "Epoch 10/1000\n",
      "343/343 [==============================] - ETA: 0s - loss: 0.2583val_aucs: 0.4528425004615323 0.8766356066613799\n",
      "343/343 [==============================] - 17s 50ms/step - loss: 0.2583 - custom_metric: 1.3295\n",
      "Epoch 11/1000\n",
      "343/343 [==============================] - ETA: 0s - loss: 0.2374val_aucs: 0.4729599865581478 0.8818847269032514\n",
      "343/343 [==============================] - 17s 50ms/step - loss: 0.2374 - custom_metric: 1.3548\n",
      "Epoch 12/1000\n",
      "342/343 [============================>.] - ETA: 0s - loss: 0.2084val_aucs: 0.4665930068931198 0.8680517694290246\n",
      "343/343 [==============================] - 17s 50ms/step - loss: 0.2084 - custom_metric: 1.3346\n",
      "Epoch 13/1000\n",
      "343/343 [==============================] - ETA: 0s - loss: 0.2093val_aucs: 0.47213753545579096 0.878094642149088\n",
      "343/343 [==============================] - 17s 50ms/step - loss: 0.2093 - custom_metric: 1.3502\n",
      "Epoch 14/1000\n",
      "342/343 [============================>.] - ETA: 0s - loss: 0.1791val_aucs: 0.47345778799331606 0.8660707647700239\n",
      "343/343 [==============================] - 17s 51ms/step - loss: 0.1791 - custom_metric: 1.3395\n",
      "Test res 0.8804423842149355 0.47447109917459596 0.4848780487804878\n",
      "Repeat 6 ld 30\n",
      "Num train: 10965 Num valid: 2778\n",
      "Exp_GloVe/M/models/classification/new_mimic_iii_24hm_strats_no_interp_with_ss_repeat6_30ld.h5\n",
      "Epoch 1/1000\n",
      "343/343 [==============================] - ETA: 0s - loss: 0.4896val_aucs: 0.4522797234584412 0.8833856281530699\n",
      "343/343 [==============================] - 30s 68ms/step - loss: 0.4896 - custom_metric: 1.3357\n",
      "Epoch 2/1000\n",
      "342/343 [============================>.] - ETA: 0s - loss: 0.4299val_aucs: 0.4460312984519665 0.8889442598744923\n",
      "343/343 [==============================] - 17s 51ms/step - loss: 0.4301 - custom_metric: 1.3350\n",
      "Epoch 3/1000\n",
      "343/343 [==============================] - ETA: 0s - loss: 0.4087val_aucs: 0.4619388771690425 0.8875584471514703\n",
      "343/343 [==============================] - 17s 51ms/step - loss: 0.4087 - custom_metric: 1.3495\n",
      "Epoch 4/1000\n",
      "342/343 [============================>.] - ETA: 0s - loss: 0.3855val_aucs: 0.46452784026820704 0.8943706164636397\n",
      "343/343 [==============================] - 17s 51ms/step - loss: 0.3855 - custom_metric: 1.3589\n",
      "Epoch 5/1000\n",
      "343/343 [==============================] - ETA: 0s - loss: 0.3616val_aucs: 0.4629193278589572 0.8957625815183954\n",
      "343/343 [==============================] - 17s 50ms/step - loss: 0.3616 - custom_metric: 1.3587\n",
      "Epoch 6/1000\n",
      "342/343 [============================>.] - ETA: 0s - loss: 0.3472val_aucs: 0.4733092921704214 0.8917835609696074\n",
      "343/343 [==============================] - 17s 51ms/step - loss: 0.3473 - custom_metric: 1.3651\n",
      "Epoch 7/1000\n",
      "343/343 [==============================] - ETA: 0s - loss: 0.3251val_aucs: 0.4587664531835439 0.8854251261228004\n",
      "343/343 [==============================] - 17s 50ms/step - loss: 0.3251 - custom_metric: 1.3442\n",
      "Epoch 8/1000\n",
      "343/343 [==============================] - ETA: 0s - loss: 0.3047val_aucs: 0.45366248894433436 0.8860449735449736\n",
      "343/343 [==============================] - 17s 50ms/step - loss: 0.3047 - custom_metric: 1.3397\n",
      "Epoch 9/1000\n",
      "343/343 [==============================] - ETA: 0s - loss: 0.2803val_aucs: 0.4327354221004699 0.8822197612895288\n",
      "343/343 [==============================] - 17s 50ms/step - loss: 0.2803 - custom_metric: 1.3150\n",
      "Epoch 10/1000\n",
      "343/343 [==============================] - ETA: 0s - loss: 0.2575val_aucs: 0.4209047619433289 0.8686200319921249\n",
      "343/343 [==============================] - 17s 50ms/step - loss: 0.2575 - custom_metric: 1.2895\n",
      "Epoch 11/1000\n",
      "342/343 [============================>.] - ETA: 0s - loss: 0.2664val_aucs: 0.4073639794793498 0.8691475944382923\n",
      "343/343 [==============================] - 17s 50ms/step - loss: 0.2663 - custom_metric: 1.2765\n",
      "Epoch 12/1000\n",
      "343/343 [==============================] - ETA: 0s - loss: 0.2201val_aucs: 0.4098884233301719 0.8679401993355481\n",
      "343/343 [==============================] - 17s 50ms/step - loss: 0.2201 - custom_metric: 1.2778\n",
      "Epoch 13/1000\n",
      "343/343 [==============================] - ETA: 0s - loss: 0.2054val_aucs: 0.40141742048750256 0.8628799064845576\n",
      "343/343 [==============================] - 17s 50ms/step - loss: 0.2054 - custom_metric: 1.2643\n",
      "Epoch 14/1000\n",
      "343/343 [==============================] - ETA: 0s - loss: 0.2060val_aucs: 0.41741678574395474 0.8781115417743324\n",
      "343/343 [==============================] - 17s 50ms/step - loss: 0.2060 - custom_metric: 1.2955\n",
      "Epoch 15/1000\n",
      "343/343 [==============================] - ETA: 0s - loss: 0.1693val_aucs: 0.3958281868972094 0.8708533284114679\n",
      "343/343 [==============================] - 17s 50ms/step - loss: 0.1693 - custom_metric: 1.2667\n",
      "Epoch 16/1000\n",
      "342/343 [============================>.] - ETA: 0s - loss: 0.1746val_aucs: 0.4030759836914339 0.8596437799926172\n",
      "343/343 [==============================] - 17s 51ms/step - loss: 0.1746 - custom_metric: 1.2627\n",
      "Test res 0.8762891185974149 0.4603371424288571 0.4647342995169082\n",
      "Repeat 7 ld 30\n",
      "Num train: 10965 Num valid: 2778\n",
      "Exp_GloVe/M/models/classification/new_mimic_iii_24hm_strats_no_interp_with_ss_repeat7_30ld.h5\n",
      "Epoch 1/1000\n",
      "342/343 [============================>.] - ETA: 0s - loss: 0.4977val_aucs: 0.4542554708514523 0.8844398525301778\n",
      "343/343 [==============================] - 28s 58ms/step - loss: 0.4978 - custom_metric: 1.3387\n",
      "Epoch 2/1000\n",
      "343/343 [==============================] - ETA: 0s - loss: 0.4384val_aucs: 0.4855853592370358 0.8927157043467919\n",
      "343/343 [==============================] - 17s 51ms/step - loss: 0.4384 - custom_metric: 1.3783\n",
      "Epoch 3/1000\n",
      "343/343 [==============================] - ETA: 0s - loss: 0.4102val_aucs: 0.503713144298786 0.8994276624843908\n",
      "343/343 [==============================] - 17s 51ms/step - loss: 0.4102 - custom_metric: 1.4031\n",
      "Epoch 4/1000\n",
      "342/343 [============================>.] - ETA: 0s - loss: 0.3870val_aucs: 0.482420795418666 0.8972126419694357\n",
      "343/343 [==============================] - 17s 50ms/step - loss: 0.3870 - custom_metric: 1.3796\n",
      "Epoch 5/1000\n",
      "343/343 [==============================] - ETA: 0s - loss: 0.3634val_aucs: 0.49462504435079424 0.89418742938693\n",
      "343/343 [==============================] - 17s 50ms/step - loss: 0.3634 - custom_metric: 1.3888\n",
      "Epoch 6/1000\n",
      "343/343 [==============================] - ETA: 0s - loss: 0.3405val_aucs: 0.48028963467809016 0.8928435511684606\n",
      "343/343 [==============================] - 17s 50ms/step - loss: 0.3405 - custom_metric: 1.3731\n",
      "Epoch 7/1000\n",
      "342/343 [============================>.] - ETA: 0s - loss: 0.3186val_aucs: 0.45506230546387144 0.8766188975441518\n",
      "343/343 [==============================] - 17s 50ms/step - loss: 0.3193 - custom_metric: 1.3317\n",
      "Epoch 8/1000\n",
      "343/343 [==============================] - ETA: 0s - loss: 0.3050val_aucs: 0.45206515221380106 0.8715020514955105\n",
      "343/343 [==============================] - 17s 50ms/step - loss: 0.3050 - custom_metric: 1.3236\n",
      "Epoch 9/1000\n",
      "343/343 [==============================] - ETA: 0s - loss: 0.2754val_aucs: 0.424785461559735 0.8634700005946363\n",
      "343/343 [==============================] - 17s 50ms/step - loss: 0.2754 - custom_metric: 1.2883\n",
      "Epoch 10/1000\n",
      "343/343 [==============================] - ETA: 0s - loss: 0.2591val_aucs: 0.4513140701452782 0.8623981685199501\n",
      "343/343 [==============================] - 17s 50ms/step - loss: 0.2591 - custom_metric: 1.3137\n",
      "Epoch 11/1000\n",
      "343/343 [==============================] - ETA: 0s - loss: 0.2332val_aucs: 0.39078858541636585 0.853499435095439\n",
      "343/343 [==============================] - 17s 50ms/step - loss: 0.2332 - custom_metric: 1.2443\n",
      "Epoch 12/1000\n",
      "343/343 [==============================] - ETA: 0s - loss: 0.2050val_aucs: 0.4222901721853569 0.8674050068383183\n",
      "343/343 [==============================] - 17s 50ms/step - loss: 0.2050 - custom_metric: 1.2897\n",
      "Epoch 13/1000\n",
      "343/343 [==============================] - ETA: 0s - loss: 0.1861val_aucs: 0.41495749332574167 0.8497948504489504\n",
      "343/343 [==============================] - 17s 50ms/step - loss: 0.1861 - custom_metric: 1.2648\n",
      "Test res 0.8788292095500239 0.47176976056918984 0.4873046875\n",
      "Repeat 8 ld 30\n",
      "Num train: 10965 Num valid: 2778\n",
      "Exp_GloVe/M/models/classification/new_mimic_iii_24hm_strats_no_interp_with_ss_repeat8_30ld.h5\n",
      "Epoch 1/1000\n",
      "343/343 [==============================] - ETA: 0s - loss: 0.4938val_aucs: 0.4947439565897475 0.8793136690647483\n",
      "343/343 [==============================] - 26s 58ms/step - loss: 0.4938 - custom_metric: 1.3741\n",
      "Epoch 2/1000\n",
      "343/343 [==============================] - ETA: 0s - loss: 0.4397val_aucs: 0.5149078661599358 0.8923985611510791\n",
      "343/343 [==============================] - 17s 50ms/step - loss: 0.4397 - custom_metric: 1.4073\n",
      "Epoch 3/1000\n",
      "342/343 [============================>.] - ETA: 0s - loss: 0.4128val_aucs: 0.5124479531109852 0.8970935251798562\n",
      "343/343 [==============================] - 18s 51ms/step - loss: 0.4124 - custom_metric: 1.4095\n",
      "Epoch 4/1000\n",
      "343/343 [==============================] - ETA: 0s - loss: 0.3918val_aucs: 0.5242858361332643 0.8921726618705036\n",
      "343/343 [==============================] - 17s 50ms/step - loss: 0.3918 - custom_metric: 1.4165\n",
      "Epoch 5/1000\n",
      "343/343 [==============================] - ETA: 0s - loss: 0.3637val_aucs: 0.5200768413739069 0.8872791366906474\n",
      "343/343 [==============================] - 17s 50ms/step - loss: 0.3637 - custom_metric: 1.4074\n",
      "Epoch 6/1000\n",
      "343/343 [==============================] - ETA: 0s - loss: 0.3413val_aucs: 0.529601883494567 0.8909812949640289\n",
      "343/343 [==============================] - 17s 50ms/step - loss: 0.3413 - custom_metric: 1.4206\n",
      "Epoch 7/1000\n",
      "343/343 [==============================] - ETA: 0s - loss: 0.3231val_aucs: 0.49239403944523064 0.8774892086330937\n",
      "343/343 [==============================] - 17s 50ms/step - loss: 0.3231 - custom_metric: 1.3699\n",
      "Epoch 8/1000\n",
      "343/343 [==============================] - ETA: 0s - loss: 0.2923val_aucs: 0.5152038298890572 0.8809381294964028\n",
      "343/343 [==============================] - 17s 50ms/step - loss: 0.2923 - custom_metric: 1.3961\n",
      "Epoch 9/1000\n",
      "343/343 [==============================] - ETA: 0s - loss: 0.2571val_aucs: 0.5163177239344956 0.8726014388489208\n",
      "343/343 [==============================] - 17s 50ms/step - loss: 0.2571 - custom_metric: 1.3889\n",
      "Epoch 10/1000\n",
      "343/343 [==============================] - ETA: 0s - loss: 0.2378val_aucs: 0.4773101486248363 0.8612776978417265\n",
      "343/343 [==============================] - 17s 50ms/step - loss: 0.2378 - custom_metric: 1.3386\n",
      "Epoch 11/1000\n",
      "343/343 [==============================] - ETA: 0s - loss: 0.2225val_aucs: 0.4727927774581504 0.8610086330935252\n",
      "343/343 [==============================] - 17s 50ms/step - loss: 0.2225 - custom_metric: 1.3338\n",
      "Epoch 12/1000\n",
      "343/343 [==============================] - ETA: 0s - loss: 0.2020val_aucs: 0.4713919507145848 0.8541467625899283\n",
      "343/343 [==============================] - 17s 50ms/step - loss: 0.2020 - custom_metric: 1.3255\n",
      "Epoch 13/1000\n",
      "343/343 [==============================] - ETA: 0s - loss: 0.1877val_aucs: 0.4356917855367267 0.8617021582733814\n",
      "343/343 [==============================] - 17s 50ms/step - loss: 0.1877 - custom_metric: 1.2974\n",
      "Epoch 14/1000\n",
      "343/343 [==============================] - ETA: 0s - loss: 0.1603val_aucs: 0.43856119203705457 0.8627755395683453\n",
      "343/343 [==============================] - 17s 50ms/step - loss: 0.1603 - custom_metric: 1.3013\n",
      "Epoch 15/1000\n",
      "343/343 [==============================] - ETA: 0s - loss: 0.1556val_aucs: 0.4758164351665024 0.8584\n",
      "343/343 [==============================] - 17s 50ms/step - loss: 0.1556 - custom_metric: 1.3342\n",
      "Epoch 16/1000\n",
      "343/343 [==============================] - ETA: 0s - loss: 0.1373val_aucs: 0.45875984920620405 0.8507597122302158\n",
      "343/343 [==============================] - 17s 50ms/step - loss: 0.1373 - custom_metric: 1.3095\n",
      "Test res 0.870945278841551 0.43829899752189594 0.4609375\n",
      "Repeat 9 ld 30\n",
      "Num train: 10965 Num valid: 2778\n",
      "Exp_GloVe/M/models/classification/new_mimic_iii_24hm_strats_no_interp_with_ss_repeat9_30ld.h5\n",
      "Epoch 1/1000\n",
      "343/343 [==============================] - ETA: 0s - loss: 0.4847val_aucs: 0.48113235574516355 0.8829872204472843\n",
      "343/343 [==============================] - 28s 58ms/step - loss: 0.4847 - custom_metric: 1.3641\n",
      "Epoch 2/1000\n",
      "343/343 [==============================] - ETA: 0s - loss: 0.4277val_aucs: 0.5148221103168384 0.8980579977146054\n",
      "343/343 [==============================] - 17s 51ms/step - loss: 0.4277 - custom_metric: 1.4129\n",
      "Epoch 3/1000\n",
      "343/343 [==============================] - ETA: 0s - loss: 0.4065val_aucs: 0.5228068050071277 0.8970071243674355\n",
      "343/343 [==============================] - 17s 50ms/step - loss: 0.4065 - custom_metric: 1.4198\n",
      "Epoch 4/1000\n",
      "342/343 [============================>.] - ETA: 0s - loss: 0.3797val_aucs: 0.523889332534671 0.895170646673352\n",
      "343/343 [==============================] - 17s 50ms/step - loss: 0.3799 - custom_metric: 1.4191\n",
      "Epoch 5/1000\n",
      "343/343 [==============================] - ETA: 0s - loss: 0.3540val_aucs: 0.5259764087547525 0.8975230871481543\n",
      "343/343 [==============================] - 17s 50ms/step - loss: 0.3540 - custom_metric: 1.4235\n",
      "Epoch 6/1000\n",
      "343/343 [==============================] - ETA: 0s - loss: 0.3285val_aucs: 0.49479221902020565 0.8873976819570438\n",
      "343/343 [==============================] - 17s 50ms/step - loss: 0.3285 - custom_metric: 1.3822\n",
      "Epoch 7/1000\n",
      "343/343 [==============================] - ETA: 0s - loss: 0.3070val_aucs: 0.4541475623834637 0.8830353186726055\n",
      "343/343 [==============================] - 17s 50ms/step - loss: 0.3070 - custom_metric: 1.3372\n",
      "Epoch 8/1000\n",
      "342/343 [============================>.] - ETA: 0s - loss: 0.2817val_aucs: 0.4815861712478372 0.8855597467409807\n",
      "343/343 [==============================] - 17s 50ms/step - loss: 0.2813 - custom_metric: 1.3671\n",
      "Epoch 9/1000\n",
      "343/343 [==============================] - ETA: 0s - loss: 0.2635val_aucs: 0.47137407519049035 0.8825747417271053\n",
      "343/343 [==============================] - 17s 50ms/step - loss: 0.2635 - custom_metric: 1.3539\n",
      "Epoch 10/1000\n",
      "343/343 [==============================] - ETA: 0s - loss: 0.2270val_aucs: 0.44878545753560956 0.8739389239989739\n",
      "343/343 [==============================] - 17s 50ms/step - loss: 0.2270 - custom_metric: 1.3227\n",
      "Epoch 11/1000\n",
      "343/343 [==============================] - ETA: 0s - loss: 0.2132val_aucs: 0.4256761376082994 0.8716272941395956\n",
      "343/343 [==============================] - 17s 50ms/step - loss: 0.2132 - custom_metric: 1.2973\n",
      "Epoch 12/1000\n",
      "343/343 [==============================] - ETA: 0s - loss: 0.2021val_aucs: 0.4472519864148467 0.8701916349898556\n",
      "343/343 [==============================] - 17s 50ms/step - loss: 0.2021 - custom_metric: 1.3174\n",
      "Epoch 13/1000\n",
      "343/343 [==============================] - ETA: 0s - loss: 0.1733val_aucs: 0.46404008043518025 0.8704700216879271\n",
      "343/343 [==============================] - 17s 50ms/step - loss: 0.1733 - custom_metric: 1.3345\n",
      "Epoch 14/1000\n",
      "342/343 [============================>.] - ETA: 0s - loss: 0.1742val_aucs: 0.45123575957000767 0.8640962780718733\n",
      "343/343 [==============================] - 17s 50ms/step - loss: 0.1741 - custom_metric: 1.3153\n",
      "Epoch 15/1000\n",
      "343/343 [==============================] - ETA: 0s - loss: 0.1449val_aucs: 0.4046075622399657 0.8482384389356592\n",
      "343/343 [==============================] - 17s 50ms/step - loss: 0.1449 - custom_metric: 1.2528\n",
      "Test res 0.8772688599509335 0.4619571250460892 0.4692682926829268\n",
      "gen_res {10: [(0.8495673253126496, 0.015550036538510871), (0.39991067148684534, 0.03244069307475938), (0.43139881908261096, 0.025470753193600273)], 20: [(0.8741984427357586, 0.002984760713655759), (0.45556223727894085, 0.01150887998551242), (0.4675065393501395, 0.00985098073216147)], 30: [(0.877925330600766, 0.003053670631961259), (0.4666736071604231, 0.015783723597634962), (0.47692207508615125, 0.011021763012031695)]}\n",
      "Repeat 0 ld 40\n",
      "Num train: 14620 Num valid: 3704\n",
      "Exp_GloVe/M/models/classification/new_mimic_iii_24hm_strats_no_interp_with_ss_repeat0_40ld.h5\n",
      "Epoch 1/1000\n",
      "457/457 [==============================] - ETA: 0s - loss: 0.4781val_aucs: 0.5400143907145283 0.8937946294629462\n",
      "457/457 [==============================] - 32s 56ms/step - loss: 0.4781 - custom_metric: 1.4338\n",
      "Epoch 2/1000\n",
      "457/457 [==============================] - ETA: 0s - loss: 0.4176val_aucs: 0.5580005828708116 0.9016486648664866\n",
      "457/457 [==============================] - 24s 53ms/step - loss: 0.4176 - custom_metric: 1.4596\n",
      "Epoch 3/1000\n",
      "457/457 [==============================] - ETA: 0s - loss: 0.3913val_aucs: 0.5670424692900559 0.9022682268226823\n",
      "457/457 [==============================] - 23s 50ms/step - loss: 0.3913 - custom_metric: 1.4693\n",
      "Epoch 4/1000\n",
      "457/457 [==============================] - ETA: 0s - loss: 0.3720val_aucs: 0.5474173753945191 0.9030400540054007\n",
      "457/457 [==============================] - 23s 50ms/step - loss: 0.3720 - custom_metric: 1.4505\n",
      "Epoch 5/1000\n",
      "457/457 [==============================] - ETA: 0s - loss: 0.3507val_aucs: 0.551069983106291 0.8953022802280227\n",
      "457/457 [==============================] - 23s 50ms/step - loss: 0.3507 - custom_metric: 1.4464\n",
      "Epoch 6/1000\n",
      "457/457 [==============================] - ETA: 0s - loss: 0.3332val_aucs: 0.5332496613508854 0.8940466546654664\n",
      "457/457 [==============================] - 23s 50ms/step - loss: 0.3332 - custom_metric: 1.4273\n",
      "Epoch 7/1000\n",
      "457/457 [==============================] - ETA: 0s - loss: 0.3084val_aucs: 0.5195417643952236 0.8877516501650164\n",
      "457/457 [==============================] - 23s 50ms/step - loss: 0.3084 - custom_metric: 1.4073\n",
      "Epoch 8/1000\n",
      "457/457 [==============================] - ETA: 0s - loss: 0.2940val_aucs: 0.5499971689214506 0.899324182418242\n",
      "457/457 [==============================] - 23s 50ms/step - loss: 0.2940 - custom_metric: 1.4493\n",
      "Epoch 9/1000\n",
      "457/457 [==============================] - ETA: 0s - loss: 0.2823val_aucs: 0.5147209484355043 0.8875746324632463\n",
      "457/457 [==============================] - 23s 50ms/step - loss: 0.2823 - custom_metric: 1.4023\n",
      "Epoch 10/1000\n",
      "457/457 [==============================] - ETA: 0s - loss: 0.2597val_aucs: 0.5267818936230457 0.8881023102310232\n",
      "457/457 [==============================] - 23s 50ms/step - loss: 0.2597 - custom_metric: 1.4149\n",
      "Epoch 11/1000\n",
      "457/457 [==============================] - ETA: 0s - loss: 0.2345val_aucs: 0.5167176222173667 0.8866396639663967\n",
      "457/457 [==============================] - 23s 50ms/step - loss: 0.2345 - custom_metric: 1.4034\n",
      "Epoch 12/1000\n",
      "457/457 [==============================] - ETA: 0s - loss: 0.2210val_aucs: 0.479380317815435 0.8811708670867086\n",
      "457/457 [==============================] - 23s 50ms/step - loss: 0.2210 - custom_metric: 1.3606\n",
      "Epoch 13/1000\n",
      "456/457 [============================>.] - ETA: 0s - loss: 0.1907val_aucs: 0.498153586359635 0.8798709870987098\n",
      "457/457 [==============================] - 23s 50ms/step - loss: 0.1910 - custom_metric: 1.3780\n",
      "Test res 0.8838150543022977 0.4769430607739424 0.4892578125\n",
      "Repeat 1 ld 40\n",
      "Num train: 14620 Num valid: 3704\n",
      "Exp_GloVe/M/models/classification/new_mimic_iii_24hm_strats_no_interp_with_ss_repeat1_40ld.h5\n",
      "Epoch 1/1000\n",
      "457/457 [==============================] - ETA: 0s - loss: 0.4813val_aucs: 0.49699285846977104 0.8852673075249027\n",
      "457/457 [==============================] - 32s 55ms/step - loss: 0.4813 - custom_metric: 1.3823\n",
      "Epoch 2/1000\n",
      "456/457 [============================>.] - ETA: 0s - loss: 0.4277val_aucs: 0.5110881023485415 0.8915054157909792\n",
      "457/457 [==============================] - 23s 50ms/step - loss: 0.4276 - custom_metric: 1.4026\n",
      "Epoch 3/1000\n",
      "457/457 [==============================] - ETA: 0s - loss: 0.3941val_aucs: 0.5160358586717033 0.8926184199072289\n",
      "457/457 [==============================] - 23s 50ms/step - loss: 0.3941 - custom_metric: 1.4087\n",
      "Epoch 4/1000\n",
      "456/457 [============================>.] - ETA: 0s - loss: 0.3760val_aucs: 0.5025194088587309 0.8942988074067626\n",
      "457/457 [==============================] - 23s 50ms/step - loss: 0.3781 - custom_metric: 1.3968\n",
      "Epoch 5/1000\n",
      "456/457 [============================>.] - ETA: 0s - loss: 0.3586val_aucs: 0.5155133389015538 0.8969771678708666\n",
      "457/457 [==============================] - 23s 50ms/step - loss: 0.3586 - custom_metric: 1.4125\n",
      "Epoch 6/1000\n",
      "457/457 [==============================] - ETA: 0s - loss: 0.3410val_aucs: 0.5105820176082906 0.8939568229017696\n",
      "457/457 [==============================] - 23s 50ms/step - loss: 0.3410 - custom_metric: 1.4045\n",
      "Epoch 7/1000\n",
      "456/457 [============================>.] - ETA: 0s - loss: 0.3179val_aucs: 0.4977496695621816 0.8881236553791055\n",
      "457/457 [==============================] - 23s 50ms/step - loss: 0.3178 - custom_metric: 1.3859\n",
      "Epoch 8/1000\n",
      "456/457 [============================>.] - ETA: 0s - loss: 0.2988val_aucs: 0.4727506000752757 0.8831081417183788\n",
      "457/457 [==============================] - 24s 52ms/step - loss: 0.2985 - custom_metric: 1.3559\n",
      "Epoch 9/1000\n",
      "457/457 [==============================] - ETA: 0s - loss: 0.2740val_aucs: 0.4778158347529419 0.8854079875144567\n",
      "457/457 [==============================] - 23s 50ms/step - loss: 0.2740 - custom_metric: 1.3632\n",
      "Epoch 10/1000\n",
      "457/457 [==============================] - ETA: 0s - loss: 0.2666val_aucs: 0.47503669359681266 0.8715133125240944\n",
      "457/457 [==============================] - 23s 50ms/step - loss: 0.2666 - custom_metric: 1.3466\n",
      "Epoch 11/1000\n",
      "457/457 [==============================] - ETA: 0s - loss: 0.2406val_aucs: 0.4674478871141817 0.8718148806784972\n",
      "457/457 [==============================] - 23s 50ms/step - loss: 0.2406 - custom_metric: 1.3393\n",
      "Epoch 12/1000\n",
      "456/457 [============================>.] - ETA: 0s - loss: 0.2192val_aucs: 0.45699327095859066 0.8771210811684679\n",
      "457/457 [==============================] - 23s 50ms/step - loss: 0.2202 - custom_metric: 1.3341\n",
      "Epoch 13/1000\n",
      "457/457 [==============================] - ETA: 0s - loss: 0.2130val_aucs: 0.46174869042778555 0.8791978597987887\n",
      "457/457 [==============================] - 23s 50ms/step - loss: 0.2130 - custom_metric: 1.3409\n",
      "Epoch 14/1000\n",
      "457/457 [==============================] - ETA: 0s - loss: 0.2010val_aucs: 0.438325479331675 0.8702988789126136\n",
      "457/457 [==============================] - 23s 50ms/step - loss: 0.2010 - custom_metric: 1.3086\n",
      "Epoch 15/1000\n",
      "457/457 [==============================] - ETA: 0s - loss: 0.1744val_aucs: 0.45312714551531164 0.8735729919291656\n",
      "457/457 [==============================] - 23s 50ms/step - loss: 0.1744 - custom_metric: 1.3267\n",
      "Test res 0.8829039387865005 0.46480356016758595 0.47189922480620156\n",
      "Repeat 2 ld 40\n",
      "Num train: 14620 Num valid: 3704\n",
      "Exp_GloVe/M/models/classification/new_mimic_iii_24hm_strats_no_interp_with_ss_repeat2_40ld.h5\n",
      "Epoch 1/1000\n",
      "457/457 [==============================] - ETA: 0s - loss: 0.4786val_aucs: 0.5053936271044179 0.8862784178089198\n",
      "457/457 [==============================] - 34s 56ms/step - loss: 0.4786 - custom_metric: 1.3917\n",
      "Epoch 2/1000\n",
      "456/457 [============================>.] - ETA: 0s - loss: 0.4220val_aucs: 0.5183736200957184 0.8902094414893618\n",
      "457/457 [==============================] - 23s 51ms/step - loss: 0.4218 - custom_metric: 1.4086\n",
      "Epoch 3/1000\n",
      "457/457 [==============================] - ETA: 0s - loss: 0.4023val_aucs: 0.5370031929434996 0.8947182513297872\n",
      "457/457 [==============================] - 23s 51ms/step - loss: 0.4023 - custom_metric: 1.4317\n",
      "Epoch 4/1000\n",
      "457/457 [==============================] - ETA: 0s - loss: 0.3795val_aucs: 0.5495632720379449 0.8983392044292144\n",
      "457/457 [==============================] - 23s 50ms/step - loss: 0.3795 - custom_metric: 1.4479\n",
      "Epoch 5/1000\n",
      "456/457 [============================>.] - ETA: 0s - loss: 0.3585val_aucs: 0.5222514630465543 0.8977965809124387\n",
      "457/457 [==============================] - 23s 50ms/step - loss: 0.3583 - custom_metric: 1.4200\n",
      "Epoch 6/1000\n",
      "457/457 [==============================] - ETA: 0s - loss: 0.3375val_aucs: 0.525117131501025 0.8904419944251228\n",
      "457/457 [==============================] - 23s 50ms/step - loss: 0.3375 - custom_metric: 1.4156\n",
      "Epoch 7/1000\n",
      "457/457 [==============================] - ETA: 0s - loss: 0.3205val_aucs: 0.49222848796064866 0.8866468264116203\n",
      "457/457 [==============================] - 23s 50ms/step - loss: 0.3205 - custom_metric: 1.3789\n",
      "Epoch 8/1000\n",
      "457/457 [==============================] - ETA: 0s - loss: 0.2987val_aucs: 0.5075759013206248 0.8919348084594926\n",
      "457/457 [==============================] - 23s 50ms/step - loss: 0.2987 - custom_metric: 1.3995\n",
      "Epoch 9/1000\n",
      "456/457 [============================>.] - ETA: 0s - loss: 0.2873val_aucs: 0.508466709199511 0.8883721933817513\n",
      "457/457 [==============================] - 23s 51ms/step - loss: 0.2873 - custom_metric: 1.3968\n",
      "Epoch 10/1000\n",
      "457/457 [==============================] - ETA: 0s - loss: 0.2612val_aucs: 0.4912908338700698 0.8912711135689444\n",
      "457/457 [==============================] - 23s 50ms/step - loss: 0.2612 - custom_metric: 1.3826\n",
      "Epoch 11/1000\n",
      "457/457 [==============================] - ETA: 0s - loss: 0.2489val_aucs: 0.4803911636375497 0.8810583635945172\n",
      "457/457 [==============================] - 23s 50ms/step - loss: 0.2489 - custom_metric: 1.3614\n",
      "Epoch 12/1000\n",
      "457/457 [==============================] - ETA: 0s - loss: 0.2117val_aucs: 0.49764564317740567 0.8851044650163665\n",
      "457/457 [==============================] - 23s 50ms/step - loss: 0.2117 - custom_metric: 1.3828\n",
      "Epoch 13/1000\n",
      "457/457 [==============================] - ETA: 0s - loss: 0.2088val_aucs: 0.4909836777985677 0.8765599427168576\n",
      "457/457 [==============================] - 23s 50ms/step - loss: 0.2088 - custom_metric: 1.3675\n",
      "Epoch 14/1000\n",
      "457/457 [==============================] - ETA: 0s - loss: 0.1907val_aucs: 0.44088997569961336 0.8700804265548281\n",
      "457/457 [==============================] - 23s 50ms/step - loss: 0.1907 - custom_metric: 1.3110\n",
      "Test res 0.8857225066568932 0.4851614521318192 0.48828125\n",
      "Repeat 3 ld 40\n",
      "Num train: 14620 Num valid: 3704\n",
      "Exp_GloVe/M/models/classification/new_mimic_iii_24hm_strats_no_interp_with_ss_repeat3_40ld.h5\n",
      "Epoch 1/1000\n",
      "456/457 [============================>.] - ETA: 0s - loss: 0.4977val_aucs: 0.4897102728201693 0.8896122505395625\n",
      "457/457 [==============================] - 34s 57ms/step - loss: 0.4975 - custom_metric: 1.3793\n",
      "Epoch 2/1000\n",
      "457/457 [==============================] - ETA: 0s - loss: 0.4282val_aucs: 0.5115572335901083 0.8959855541832255\n",
      "457/457 [==============================] - 23s 51ms/step - loss: 0.4282 - custom_metric: 1.4075\n",
      "Epoch 3/1000\n",
      "456/457 [============================>.] - ETA: 0s - loss: 0.4074val_aucs: 0.5099262209090424 0.9008743743147155\n",
      "457/457 [==============================] - 23s 51ms/step - loss: 0.4075 - custom_metric: 1.4108\n",
      "Epoch 4/1000\n",
      "457/457 [==============================] - ETA: 0s - loss: 0.3842val_aucs: 0.5165593717131343 0.9008659205361725\n",
      "457/457 [==============================] - 23s 50ms/step - loss: 0.3842 - custom_metric: 1.4174\n",
      "Epoch 5/1000\n",
      "457/457 [==============================] - ETA: 0s - loss: 0.3638val_aucs: 0.49684178024571557 0.8905235678665114\n",
      "457/457 [==============================] - 23s 50ms/step - loss: 0.3638 - custom_metric: 1.3874\n",
      "Epoch 6/1000\n",
      "456/457 [============================>.] - ETA: 0s - loss: 0.3519val_aucs: 0.46593809903933286 0.8851647176480237\n",
      "457/457 [==============================] - 23s 50ms/step - loss: 0.3518 - custom_metric: 1.3511\n",
      "Epoch 7/1000\n",
      "456/457 [============================>.] - ETA: 0s - loss: 0.3310val_aucs: 0.5023819662071398 0.8953278502125702\n",
      "457/457 [==============================] - 23s 50ms/step - loss: 0.3306 - custom_metric: 1.3977\n",
      "Epoch 8/1000\n",
      "457/457 [==============================] - ETA: 0s - loss: 0.3069val_aucs: 0.5089953137604907 0.8885191769739361\n",
      "457/457 [==============================] - 23s 50ms/step - loss: 0.3069 - custom_metric: 1.3975\n",
      "Epoch 9/1000\n",
      "456/457 [============================>.] - ETA: 0s - loss: 0.2770val_aucs: 0.49044295851575326 0.8927688914475658\n",
      "457/457 [==============================] - 23s 50ms/step - loss: 0.2767 - custom_metric: 1.3832\n",
      "Epoch 10/1000\n",
      "457/457 [==============================] - ETA: 0s - loss: 0.2629val_aucs: 0.48027203094455734 0.884984652165055\n",
      "457/457 [==============================] - 23s 50ms/step - loss: 0.2629 - custom_metric: 1.3653\n",
      "Epoch 11/1000\n",
      "457/457 [==============================] - ETA: 0s - loss: 0.2551val_aucs: 0.45492298432606787 0.8799309833519738\n",
      "457/457 [==============================] - 23s 50ms/step - loss: 0.2551 - custom_metric: 1.3349\n",
      "Epoch 12/1000\n",
      "457/457 [==============================] - ETA: 0s - loss: 0.2308val_aucs: 0.47380763092657746 0.8784557989961984\n",
      "457/457 [==============================] - 23s 50ms/step - loss: 0.2308 - custom_metric: 1.3523\n",
      "Epoch 13/1000\n",
      "457/457 [==============================] - ETA: 0s - loss: 0.2105val_aucs: 0.46233776083563816 0.8715456804150467\n",
      "457/457 [==============================] - 23s 50ms/step - loss: 0.2105 - custom_metric: 1.3339\n",
      "Epoch 14/1000\n",
      "457/457 [==============================] - ETA: 0s - loss: 0.2043val_aucs: 0.48290944070608177 0.8866931608086208\n",
      "457/457 [==============================] - 23s 50ms/step - loss: 0.2043 - custom_metric: 1.3696\n",
      "Test res 0.8853154731629967 0.48087202913701077 0.478515625\n",
      "Repeat 4 ld 40\n",
      "Num train: 14620 Num valid: 3704\n",
      "Exp_GloVe/M/models/classification/new_mimic_iii_24hm_strats_no_interp_with_ss_repeat4_40ld.h5\n",
      "Epoch 1/1000\n",
      "457/457 [==============================] - ETA: 0s - loss: 0.4880val_aucs: 0.47260035081354074 0.8762669770851736\n",
      "457/457 [==============================] - 34s 57ms/step - loss: 0.4880 - custom_metric: 1.3489\n",
      "Epoch 2/1000\n",
      "456/457 [============================>.] - ETA: 0s - loss: 0.4297val_aucs: 0.48515646394091644 0.8782045173790582\n",
      "457/457 [==============================] - 23s 50ms/step - loss: 0.4297 - custom_metric: 1.3634\n",
      "Epoch 3/1000\n",
      "457/457 [==============================] - ETA: 0s - loss: 0.4038val_aucs: 0.5071811391178733 0.8873496732278632\n",
      "457/457 [==============================] - 23s 50ms/step - loss: 0.4038 - custom_metric: 1.3945\n",
      "Epoch 4/1000\n",
      "456/457 [============================>.] - ETA: 0s - loss: 0.3926val_aucs: 0.478684565173943 0.8804090381919496\n",
      "457/457 [==============================] - 23s 50ms/step - loss: 0.3927 - custom_metric: 1.3591\n",
      "Epoch 5/1000\n",
      "457/457 [==============================] - ETA: 0s - loss: 0.3676val_aucs: 0.5111555541792212 0.8867015372412069\n",
      "457/457 [==============================] - 23s 50ms/step - loss: 0.3676 - custom_metric: 1.3979\n",
      "Epoch 6/1000\n",
      "457/457 [==============================] - ETA: 0s - loss: 0.3489val_aucs: 0.5151121672228731 0.889362328619539\n",
      "457/457 [==============================] - 23s 50ms/step - loss: 0.3489 - custom_metric: 1.4045\n",
      "Epoch 7/1000\n",
      "457/457 [==============================] - ETA: 0s - loss: 0.3262val_aucs: 0.4816021344030638 0.8770455987274215\n",
      "457/457 [==============================] - 23s 50ms/step - loss: 0.3262 - custom_metric: 1.3586\n",
      "Epoch 8/1000\n",
      "457/457 [==============================] - ETA: 0s - loss: 0.3054val_aucs: 0.485567474925757 0.8733001453369833\n",
      "457/457 [==============================] - 23s 50ms/step - loss: 0.3054 - custom_metric: 1.3589\n",
      "Epoch 9/1000\n",
      "457/457 [==============================] - ETA: 0s - loss: 0.2777val_aucs: 0.4617668897043227 0.8600438328787796\n",
      "457/457 [==============================] - 23s 50ms/step - loss: 0.2777 - custom_metric: 1.3218\n",
      "Epoch 10/1000\n",
      "457/457 [==============================] - ETA: 0s - loss: 0.2527val_aucs: 0.47001442511681596 0.8697839003020913\n",
      "457/457 [==============================] - 23s 50ms/step - loss: 0.2527 - custom_metric: 1.3398\n",
      "Epoch 11/1000\n",
      "457/457 [==============================] - ETA: 0s - loss: 0.2377val_aucs: 0.4477297664224605 0.8635909824110487\n",
      "457/457 [==============================] - 23s 50ms/step - loss: 0.2377 - custom_metric: 1.3113\n",
      "Epoch 12/1000\n",
      "457/457 [==============================] - ETA: 0s - loss: 0.2059val_aucs: 0.4426713531914256 0.8662281661872179\n",
      "457/457 [==============================] - 23s 50ms/step - loss: 0.2059 - custom_metric: 1.3089\n",
      "Epoch 13/1000\n",
      "457/457 [==============================] - ETA: 0s - loss: 0.2158val_aucs: 0.3636097572655192 0.850968212149073\n",
      "457/457 [==============================] - 23s 50ms/step - loss: 0.2158 - custom_metric: 1.2146\n",
      "Epoch 14/1000\n",
      "457/457 [==============================] - ETA: 0s - loss: 0.1793val_aucs: 0.37855431724890426 0.8526705348795882\n",
      "457/457 [==============================] - 23s 50ms/step - loss: 0.1793 - custom_metric: 1.2312\n",
      "Epoch 15/1000\n",
      "457/457 [==============================] - ETA: 0s - loss: 0.1683val_aucs: 0.3926960997834059 0.8598129076067127\n",
      "457/457 [==============================] - 23s 50ms/step - loss: 0.1683 - custom_metric: 1.2525\n",
      "Epoch 16/1000\n",
      "457/457 [==============================] - ETA: 0s - loss: 0.1564val_aucs: 0.4255567849055255 0.8592154206574244\n",
      "457/457 [==============================] - 23s 50ms/step - loss: 0.1564 - custom_metric: 1.2848\n",
      "Test res 0.8854396354415988 0.4903848742113686 0.4931640625\n",
      "Repeat 5 ld 40\n",
      "Num train: 14620 Num valid: 3704\n",
      "Exp_GloVe/M/models/classification/new_mimic_iii_24hm_strats_no_interp_with_ss_repeat5_40ld.h5\n",
      "Epoch 1/1000\n",
      "456/457 [============================>.] - ETA: 0s - loss: 0.4955val_aucs: 0.49102054415980173 0.8849820355036926\n",
      "457/457 [==============================] - 32s 56ms/step - loss: 0.4953 - custom_metric: 1.3760\n",
      "Epoch 2/1000\n",
      "457/457 [==============================] - ETA: 0s - loss: 0.4299val_aucs: 0.49261942518533375 0.8939337152173239\n",
      "457/457 [==============================] - 23s 50ms/step - loss: 0.4299 - custom_metric: 1.3866\n",
      "Epoch 3/1000\n",
      "457/457 [==============================] - ETA: 0s - loss: 0.4080val_aucs: 0.5054481348481406 0.8915724139764224\n",
      "457/457 [==============================] - 23s 50ms/step - loss: 0.4080 - custom_metric: 1.3970\n",
      "Epoch 4/1000\n",
      "456/457 [============================>.] - ETA: 0s - loss: 0.3889val_aucs: 0.512628137335396 0.8934446201760564\n",
      "457/457 [==============================] - 23s 51ms/step - loss: 0.3887 - custom_metric: 1.4061\n",
      "Epoch 5/1000\n",
      "456/457 [============================>.] - ETA: 0s - loss: 0.3694val_aucs: 0.4814122333136315 0.8881505207710335\n",
      "457/457 [==============================] - 23s 51ms/step - loss: 0.3693 - custom_metric: 1.3696\n",
      "Epoch 6/1000\n",
      "457/457 [==============================] - ETA: 0s - loss: 0.3554val_aucs: 0.4754142617721111 0.883038059923003\n",
      "457/457 [==============================] - 23s 50ms/step - loss: 0.3554 - custom_metric: 1.3585\n",
      "Epoch 7/1000\n",
      "457/457 [==============================] - ETA: 0s - loss: 0.3484val_aucs: 0.44450134647502765 0.8744425689894251\n",
      "457/457 [==============================] - 23s 50ms/step - loss: 0.3484 - custom_metric: 1.3189\n",
      "Epoch 8/1000\n",
      "457/457 [==============================] - ETA: 0s - loss: 0.3313val_aucs: 0.4842578169313419 0.8857564359856993\n",
      "457/457 [==============================] - 23s 50ms/step - loss: 0.3313 - custom_metric: 1.3700\n",
      "Epoch 9/1000\n",
      "457/457 [==============================] - ETA: 0s - loss: 0.2899val_aucs: 0.4635571606762796 0.8715895145820318\n",
      "457/457 [==============================] - 23s 50ms/step - loss: 0.2899 - custom_metric: 1.3351\n",
      "Epoch 10/1000\n",
      "457/457 [==============================] - ETA: 0s - loss: 0.2725val_aucs: 0.4332385452148554 0.8700734971624512\n",
      "457/457 [==============================] - 23s 50ms/step - loss: 0.2725 - custom_metric: 1.3033\n",
      "Epoch 11/1000\n",
      "457/457 [==============================] - ETA: 0s - loss: 0.2572val_aucs: 0.43311322163010835 0.8645401664872432\n",
      "457/457 [==============================] - 23s 50ms/step - loss: 0.2572 - custom_metric: 1.2977\n",
      "Epoch 12/1000\n",
      "457/457 [==============================] - ETA: 0s - loss: 0.2420val_aucs: 0.4145086225139921 0.8568847658413188\n",
      "457/457 [==============================] - 23s 50ms/step - loss: 0.2420 - custom_metric: 1.2714\n",
      "Epoch 13/1000\n",
      "457/457 [==============================] - ETA: 0s - loss: 0.2202val_aucs: 0.40510519199845635 0.8559030315918182\n",
      "457/457 [==============================] - 23s 50ms/step - loss: 0.2202 - custom_metric: 1.2610\n",
      "Epoch 14/1000\n",
      "457/457 [==============================] - ETA: 0s - loss: 0.2084val_aucs: 0.44371294787757143 0.8609215720152578\n",
      "457/457 [==============================] - 23s 50ms/step - loss: 0.2084 - custom_metric: 1.3046\n",
      "Test res 0.8811123743417903 0.46820995372226853 0.48046875\n",
      "Repeat 6 ld 40\n",
      "Num train: 14620 Num valid: 3704\n",
      "Exp_GloVe/M/models/classification/new_mimic_iii_24hm_strats_no_interp_with_ss_repeat6_40ld.h5\n",
      "Epoch 1/1000\n",
      "457/457 [==============================] - ETA: 0s - loss: 0.4931val_aucs: 0.43100307756891426 0.8708169423937058\n",
      "457/457 [==============================] - 34s 56ms/step - loss: 0.4931 - custom_metric: 1.3018\n",
      "Epoch 2/1000\n",
      "456/457 [============================>.] - ETA: 0s - loss: 0.4224val_aucs: 0.4802955736176158 0.8728009196888866\n",
      "457/457 [==============================] - 23s 51ms/step - loss: 0.4225 - custom_metric: 1.3531\n",
      "Epoch 3/1000\n",
      "457/457 [==============================] - ETA: 0s - loss: 0.4013val_aucs: 0.4832279145269148 0.8826516498715669\n",
      "457/457 [==============================] - 23s 51ms/step - loss: 0.4013 - custom_metric: 1.3659\n",
      "Epoch 4/1000\n",
      "457/457 [==============================] - ETA: 0s - loss: 0.3745val_aucs: 0.4644229458291877 0.8824127463131612\n",
      "457/457 [==============================] - 23s 50ms/step - loss: 0.3745 - custom_metric: 1.3468\n",
      "Epoch 5/1000\n",
      "457/457 [==============================] - ETA: 0s - loss: 0.3577val_aucs: 0.466861586500762 0.8729594402830917\n",
      "457/457 [==============================] - 23s 50ms/step - loss: 0.3577 - custom_metric: 1.3398\n",
      "Epoch 6/1000\n",
      "457/457 [==============================] - ETA: 0s - loss: 0.3351val_aucs: 0.445689569918648 0.8671318999119829\n",
      "457/457 [==============================] - 23s 50ms/step - loss: 0.3351 - custom_metric: 1.3128\n",
      "Epoch 7/1000\n",
      "457/457 [==============================] - ETA: 0s - loss: 0.3107val_aucs: 0.4525787854085169 0.8617215426344057\n",
      "457/457 [==============================] - 23s 50ms/step - loss: 0.3107 - custom_metric: 1.3143\n",
      "Epoch 8/1000\n",
      "457/457 [==============================] - ETA: 0s - loss: 0.2817val_aucs: 0.4259588377573832 0.8536814499470101\n",
      "457/457 [==============================] - 23s 50ms/step - loss: 0.2817 - custom_metric: 1.2796\n",
      "Epoch 9/1000\n",
      "457/457 [==============================] - ETA: 0s - loss: 0.2680val_aucs: 0.4266599635505809 0.8575178279535126\n",
      "457/457 [==============================] - 23s 50ms/step - loss: 0.2680 - custom_metric: 1.2842\n",
      "Epoch 10/1000\n",
      "457/457 [==============================] - ETA: 0s - loss: 0.2480val_aucs: 0.42903218013184585 0.8576467101363368\n",
      "457/457 [==============================] - 23s 50ms/step - loss: 0.2480 - custom_metric: 1.2867\n",
      "Epoch 11/1000\n",
      "457/457 [==============================] - ETA: 0s - loss: 0.2252val_aucs: 0.41222442046927443 0.849137791668912\n",
      "457/457 [==============================] - 23s 50ms/step - loss: 0.2252 - custom_metric: 1.2614\n",
      "Epoch 12/1000\n",
      "457/457 [==============================] - ETA: 0s - loss: 0.2152val_aucs: 0.42397879848262787 0.8484749690143881\n",
      "457/457 [==============================] - 23s 50ms/step - loss: 0.2152 - custom_metric: 1.2725\n",
      "Epoch 13/1000\n",
      "457/457 [==============================] - ETA: 0s - loss: 0.1952val_aucs: 0.3648555382793837 0.827888846975984\n",
      "457/457 [==============================] - 23s 50ms/step - loss: 0.1952 - custom_metric: 1.1927\n",
      "Test res 0.8827492969123982 0.4697603539952252 0.48346303501945526\n",
      "Repeat 7 ld 40\n",
      "Num train: 14620 Num valid: 3704\n",
      "Exp_GloVe/M/models/classification/new_mimic_iii_24hm_strats_no_interp_with_ss_repeat7_40ld.h5\n",
      "Epoch 1/1000\n",
      "457/457 [==============================] - ETA: 0s - loss: 0.4836val_aucs: 0.4836406667998839 0.8848763269925095\n",
      "457/457 [==============================] - 32s 56ms/step - loss: 0.4836 - custom_metric: 1.3685\n",
      "Epoch 2/1000\n",
      "456/457 [============================>.] - ETA: 0s - loss: 0.4242val_aucs: 0.5094754210049852 0.8901259183416861\n",
      "457/457 [==============================] - 23s 50ms/step - loss: 0.4243 - custom_metric: 1.3996\n",
      "Epoch 3/1000\n",
      "456/457 [============================>.] - ETA: 0s - loss: 0.4029val_aucs: 0.5073135176755244 0.8893310700364643\n",
      "457/457 [==============================] - 23s 50ms/step - loss: 0.4029 - custom_metric: 1.3966\n",
      "Epoch 4/1000\n",
      "456/457 [============================>.] - ETA: 0s - loss: 0.3867val_aucs: 0.5190741277850263 0.8885928041529702\n",
      "457/457 [==============================] - 23s 50ms/step - loss: 0.3865 - custom_metric: 1.4077\n",
      "Epoch 5/1000\n",
      "457/457 [==============================] - ETA: 0s - loss: 0.3716val_aucs: 0.4935676318912619 0.887252788705071\n",
      "457/457 [==============================] - 23s 50ms/step - loss: 0.3716 - custom_metric: 1.3808\n",
      "Epoch 6/1000\n",
      "456/457 [============================>.] - ETA: 0s - loss: 0.3545val_aucs: 0.5128612340935631 0.8863187296797256\n",
      "457/457 [==============================] - 23s 50ms/step - loss: 0.3543 - custom_metric: 1.3992\n",
      "Epoch 7/1000\n",
      "457/457 [==============================] - ETA: 0s - loss: 0.3287val_aucs: 0.48453383398713157 0.8712525372276411\n",
      "457/457 [==============================] - 23s 50ms/step - loss: 0.3287 - custom_metric: 1.3558\n",
      "Epoch 8/1000\n",
      "457/457 [==============================] - ETA: 0s - loss: 0.3204val_aucs: 0.4683300080751619 0.8667043882811518\n",
      "457/457 [==============================] - 24s 54ms/step - loss: 0.3204 - custom_metric: 1.3350\n",
      "Epoch 9/1000\n",
      "457/457 [==============================] - ETA: 0s - loss: 0.3099val_aucs: 0.4883083397996061 0.8658852903666182\n",
      "457/457 [==============================] - 23s 50ms/step - loss: 0.3099 - custom_metric: 1.3542\n",
      "Epoch 10/1000\n",
      "457/457 [==============================] - ETA: 0s - loss: 0.2811val_aucs: 0.46562808845742226 0.845701532216055\n",
      "457/457 [==============================] - 23s 50ms/step - loss: 0.2811 - custom_metric: 1.3113\n",
      "Epoch 11/1000\n",
      "457/457 [==============================] - ETA: 0s - loss: 0.2607val_aucs: 0.45923690629728053 0.8655251387616533\n",
      "457/457 [==============================] - 23s 50ms/step - loss: 0.2607 - custom_metric: 1.3248\n",
      "Epoch 12/1000\n",
      "457/457 [==============================] - ETA: 0s - loss: 0.2501val_aucs: 0.49606609199443763 0.8693934005137324\n",
      "457/457 [==============================] - 23s 50ms/step - loss: 0.2501 - custom_metric: 1.3655\n",
      "Epoch 13/1000\n",
      "457/457 [==============================] - ETA: 0s - loss: 0.2207val_aucs: 0.4572064537631108 0.8399813188194932\n",
      "457/457 [==============================] - 23s 50ms/step - loss: 0.2207 - custom_metric: 1.2972\n",
      "Epoch 14/1000\n",
      "456/457 [============================>.] - ETA: 0s - loss: 0.1997val_aucs: 0.44324728712361894 0.853892511361391\n",
      "457/457 [==============================] - 23s 51ms/step - loss: 0.1998 - custom_metric: 1.2971\n",
      "Test res 0.8859699897528721 0.47508645486791445 0.4826923076923077\n",
      "Repeat 8 ld 40\n",
      "Num train: 14620 Num valid: 3704\n",
      "Exp_GloVe/M/models/classification/new_mimic_iii_24hm_strats_no_interp_with_ss_repeat8_40ld.h5\n",
      "Epoch 1/1000\n",
      "456/457 [============================>.] - ETA: 0s - loss: 0.4731val_aucs: 0.45516769926579514 0.876678083644384\n",
      "457/457 [==============================] - 32s 55ms/step - loss: 0.4730 - custom_metric: 1.3318\n",
      "Epoch 2/1000\n",
      "456/457 [============================>.] - ETA: 0s - loss: 0.4150val_aucs: 0.4936034548529566 0.8885333601968103\n",
      "457/457 [==============================] - 23s 51ms/step - loss: 0.4166 - custom_metric: 1.3821\n",
      "Epoch 3/1000\n",
      "457/457 [==============================] - ETA: 0s - loss: 0.3992val_aucs: 0.5058128815397095 0.88892835934849\n",
      "457/457 [==============================] - 23s 50ms/step - loss: 0.3992 - custom_metric: 1.3947\n",
      "Epoch 4/1000\n",
      "456/457 [============================>.] - ETA: 0s - loss: 0.3825val_aucs: 0.5155968287737418 0.8915802440334804\n",
      "457/457 [==============================] - 23s 50ms/step - loss: 0.3826 - custom_metric: 1.4072\n",
      "Epoch 5/1000\n",
      "457/457 [==============================] - ETA: 0s - loss: 0.3569val_aucs: 0.5015826660912361 0.8824342905214342\n",
      "457/457 [==============================] - 23s 50ms/step - loss: 0.3569 - custom_metric: 1.3840\n",
      "Epoch 6/1000\n",
      "457/457 [==============================] - ETA: 0s - loss: 0.3425val_aucs: 0.49747285532112684 0.8837624420314445\n",
      "457/457 [==============================] - 23s 50ms/step - loss: 0.3425 - custom_metric: 1.3812\n",
      "Epoch 7/1000\n",
      "457/457 [==============================] - ETA: 0s - loss: 0.3185val_aucs: 0.48439574142190023 0.8695742845831919\n",
      "457/457 [==============================] - 23s 50ms/step - loss: 0.3185 - custom_metric: 1.3540\n",
      "Epoch 8/1000\n",
      "456/457 [============================>.] - ETA: 0s - loss: 0.2883val_aucs: 0.46507629576382087 0.875695445933718\n",
      "457/457 [==============================] - 23s 50ms/step - loss: 0.2885 - custom_metric: 1.3408\n",
      "Epoch 9/1000\n",
      "457/457 [==============================] - ETA: 0s - loss: 0.2743val_aucs: 0.4520634039193797 0.8587961981110733\n",
      "457/457 [==============================] - 23s 50ms/step - loss: 0.2743 - custom_metric: 1.3109\n",
      "Epoch 10/1000\n",
      "457/457 [==============================] - ETA: 0s - loss: 0.2478val_aucs: 0.4454216300934039 0.8691077084040266\n",
      "457/457 [==============================] - 23s 50ms/step - loss: 0.2478 - custom_metric: 1.3145\n",
      "Epoch 11/1000\n",
      "457/457 [==============================] - ETA: 0s - loss: 0.2345val_aucs: 0.4429372731931049 0.8684520274855786\n",
      "457/457 [==============================] - 23s 50ms/step - loss: 0.2345 - custom_metric: 1.3114\n",
      "Epoch 12/1000\n",
      "457/457 [==============================] - ETA: 0s - loss: 0.2151val_aucs: 0.4350381505457992 0.8608480728989932\n",
      "457/457 [==============================] - 23s 50ms/step - loss: 0.2151 - custom_metric: 1.2959\n",
      "Epoch 13/1000\n",
      "457/457 [==============================] - ETA: 0s - loss: 0.1906val_aucs: 0.4427570422197827 0.8668844022169437\n",
      "457/457 [==============================] - 23s 50ms/step - loss: 0.1906 - custom_metric: 1.3096\n",
      "Epoch 14/1000\n",
      "457/457 [==============================] - ETA: 0s - loss: 0.1775val_aucs: 0.4584563089161549 0.8653830519737585\n",
      "457/457 [==============================] - 23s 50ms/step - loss: 0.1775 - custom_metric: 1.3238\n",
      "Test res 0.8830548408329344 0.46056971644680444 0.4586977648202138\n",
      "Repeat 9 ld 40\n",
      "Num train: 14620 Num valid: 3704\n",
      "Exp_GloVe/M/models/classification/new_mimic_iii_24hm_strats_no_interp_with_ss_repeat9_40ld.h5\n",
      "Epoch 1/1000\n",
      "457/457 [==============================] - ETA: 0s - loss: 0.4828"
     ]
    }
   ],
   "source": [
    "repeats = {k:10 for k in [10,20,30,40,50]}\n",
    "lds = [10, 20, 30, 40, 50]\n",
    "batch_size, lr, patience = 32, 0.0005, 10\n",
    "d, N, he, dropout = 50,2,4,0.2\n",
    "fore_savepath = 'Exp_GloVe/M/models/forecasting/forecasting_109_epochs.h5'\n",
    "f = open('Exp_GloVe/M/logs/classification_log.csv', 'a+')\n",
    "f.write('\\nTraining on different % of labeled data\\n')\n",
    "\n",
    "train_inds = np.arange(len(train_op))\n",
    "valid_inds = np.arange(len(valid_op))\n",
    "gen_res = {}\n",
    "\n",
    "np.random.seed(2023)\n",
    "for ld in lds:\n",
    "    np.random.shuffle(train_inds)\n",
    "    np.random.shuffle(valid_inds)\n",
    "    train_starts = [int(i) for i in np.linspace(0, len(train_inds)-int(ld*len(train_inds)/100), repeats[ld])]\n",
    "    valid_starts = [int(i) for i in np.linspace(0, len(valid_inds)-int(ld*len(valid_inds)/100), repeats[ld])]\n",
    "    f.write('Training on '+str(ld)+' % of labaled data+\\n'+'val_metric,roc_auc,pr_auc,min_rp,savepath\\n')\n",
    "    all_test_res = []\n",
    "    for i in range(repeats[ld]):\n",
    "        print ('Repeat', i, 'ld', ld)\n",
    "        # Get train and validation data.\n",
    "        curr_train_ind = train_inds[np.arange(train_starts[i], train_starts[i]+int(ld*len(train_inds)/100))]\n",
    "        curr_valid_ind = valid_inds[np.arange(valid_starts[i], valid_starts[i]+int(ld*len(valid_inds)/100))]\n",
    "        curr_train_ip = [ip[curr_train_ind] for ip in train_ip]\n",
    "        curr_valid_ip = [ip[curr_valid_ind] for ip in valid_ip]\n",
    "        curr_train_op = train_op[curr_train_ind]\n",
    "        curr_valid_op = valid_op[curr_valid_ind]\n",
    "        print ('Num train:',len(curr_train_op),'Num valid:',len(curr_valid_op))\n",
    "        # Construct save_path.\n",
    "        savepath = 'Exp_GloVe/M/models/classification/new_mimic_iii_24hm_strats_no_interp_with_ss_repeat'+str(i)+'_'+str(ld)+'ld'+'.h5'\n",
    "        print (savepath)\n",
    "        # Build and compile model.\n",
    "        model, fore_model =  build_strats(D, max_len, V, d, N, he, dropout, forecast=True)\n",
    "        model.compile(loss=mortality_loss, optimizer=Adam(lr))\n",
    "        fore_model.compile(loss=forecast_loss, optimizer=Adam(lr))\n",
    "        # Load pretrained weights here.\n",
    "        fore_model.load_weights(fore_savepath)\n",
    "        # Train model.\n",
    "        es = EarlyStopping(monitor='custom_metric', patience=patience, mode='max', \n",
    "                           restore_best_weights=True)\n",
    "        cus = CustomCallback(validation_data=(curr_valid_ip, curr_valid_op), batch_size=batch_size)\n",
    "        his = model.fit(curr_train_ip, curr_train_op, batch_size=batch_size, epochs=1000,\n",
    "                        verbose=1, callbacks=[cus, es]).history\n",
    "        model.save_weights(savepath)\n",
    "        # Test and write to log.\n",
    "        rocauc, prauc, minrp = get_res(test_op, model.predict(test_ip, verbose=0, batch_size=batch_size))\n",
    "        f.write(str(np.min(his['custom_metric']))+str(rocauc)+str(prauc)+str(minrp)+savepath+'\\n')\n",
    "        print ('Test res', rocauc, prauc, minrp)\n",
    "        all_test_res.append([rocauc, prauc, minrp])\n",
    "        \n",
    "    gen_res[ld] = []\n",
    "    for i in range(len(all_test_res[0])):\n",
    "        nums = [test_res[i] for test_res in all_test_res]\n",
    "        gen_res[ld].append((np.mean(nums), np.std(nums)))\n",
    "    print ('gen_res', gen_res)\n",
    "f.close()\n",
    "\n",
    "# # save to local\n",
    "# log_path = '/content/log.csv'\n",
    "# files.download(log_path)"
   ]
  }
 ],
 "metadata": {
  "accelerator": "GPU",
  "colab": {
   "machine_shape": "hm",
   "provenance": [],
   "toc_visible": true
  },
  "gpuClass": "standard",
  "kernelspec": {
   "display_name": "Python 3 (ipykernel)",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.9.7"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}

        # values_inp[ts_ind, l] = row.value
        texts_inp[ts_ind, l] = None
    # varis_inp[ts_ind, l] = row.vind
    
data.drop(columns=['obs_ind', 'first_obs_ind'], inplace=True)
# Generate 3 sets of inputs and outputs.
# train_ip = [ip[train_ind] for ip in [demo, times_inp, values_inp, varis_inp, texts_inp]]
# valid_ip = [ip[valid_ind] for ip in [demo, times_inp, values_inp, varis_inp, texts_inp]]
# test_ip = [ip[test_ind] for ip in [demo, times_inp, values_inp, varis_inp, texts_inp]]
# del times_inp, values_inp, varis_inp

19267073it [00:21, 894379.54it/s]


In [10]:
# fore_texts_ip = np.concatenate(fore_texts_ip, axis=0)

In [11]:
train_ip = [ip[train_ind] for ip in [texts_inp]]
valid_ip = [ip[valid_ind] for ip in [texts_inp]]
test_ip = [ip[test_ind] for ip in [texts_inp]]
texts_inp.shape

(57282, 880)

In [12]:
del texts_inp

In [13]:
with open('Exp_GloVe/texts_ip/classification_train.txt', 'w') as f:
    train_ip = train_ip[0]
    for txt_l in tqdm(train_ip):
        t = ''
        for text in txt_l:
            if text:
                t = t + text
        f.write(t+'\n')

100%|██████████| 36551/36551 [00:01<00:00, 21113.11it/s]


In [14]:
with open('Exp_GloVe/texts_ip/classification_valid.txt', 'w') as f:
    valid_ip = valid_ip[0]
    for txt_l in tqdm(valid_ip):
        t = ''
        for text in txt_l:
            if text:
                t = t + text
        f.write(t+'\n')

100%|██████████| 9262/9262 [00:00<00:00, 20454.82it/s]


In [15]:
with open('Exp_GloVe/texts_ip/classification_test.txt', 'w') as f:
    test_ip = test_ip[0]
    for txt_l in tqdm(test_ip):
        t = ''
        for text in txt_l:
            if text:
                t = t + text
        f.write(t+'\n')

100%|██████████| 11469/11469 [00:00<00:00, 20515.40it/s]
